In [ ]:
!pip install kaggle


In [ ]:
from google.colab import files
uploaded = files.upload()  # Select kaggle.json from your computer


Saving kaggle.json to kaggle.json


In [ ]:
!mkdir -p ~/.kaggle  # Create directory if it doesn't exist
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # Secure the file

In [ ]:
!kaggle datasets download -d ricgomes/global-fashion-retail-stores-dataset -p /content/


Dataset URL: https://www.kaggle.com/datasets/ricgomes/global-fashion-retail-stores-dataset
License(s): Attribution 4.0 International (CC BY 4.0)
 92% 205M/224M [00:00<00:00, 631MB/s]
100% 224M/224M [00:00<00:00, 585MB/s]


In [ ]:
# Unzip the dataset (adjust the file name if needed)
!unzip /content/global-fashion-retail-stores-dataset.zip -d /content/global-fashion-dataset/

# # Example: Read a CSV file from the dataset using pandas
# import pandas as pd

# # Replace with the actual file path after unzipping (check the Files pane in Colab)
# df = pd.read_csv('/content/global-fashion-dataset/global_fashion_brands_stores.csv')  # Example file name – verify yours
# print(df.head())  # Preview the first few rows


Archive:  /content/global-fashion-retail-stores-dataset.zip
  inflating: /content/global-fashion-dataset/customers.csv  
  inflating: /content/global-fashion-dataset/discounts.csv  
  inflating: /content/global-fashion-dataset/employees.csv  
  inflating: /content/global-fashion-dataset/products.csv  
  inflating: /content/global-fashion-dataset/stores.csv  
  inflating: /content/global-fashion-dataset/transactions.csv  


In [ ]:
import polars as pl
import pandas as pd
from datetime import datetime
import os

# Exchange rates you provided
exchange_rates = {
    'EUR': 1.13,
    'GBP': 1.34,
    'CNY': 0.14,
    'USD': 1.0
}

city_translation_map = {

}

country_translation_map = {
    '中国': 'China',
    'España': 'Spain',
    'Portugal': 'Portugal',
    'Deutschland': 'Germany',
    'France': 'France',
    'United Kingdom': 'United Kingdom',
    'United States': 'United States'
}

def load_data_efficiently():
    """Load data with optimized memory usage"""
    print(" Loading data efficiently...")

    # Load with pandas first, then convert to polars for better performance
    print("Loading discounts...")
    discounts = pd.read_csv('/content/global-fashion-dataset/discounts.csv')

    print("Loading products...")
    products = pd.read_csv(r'/content/global-fashion-dataset/products.csv')

    print("Loading employees...")
    employees = pd.read_csv(r'/content/global-fashion-dataset/employees.csv')

    print("Loading stores...")
    stores = pd.read_csv(r'/content/global-fashion-dataset/stores.csv')

    print("Loading customers...")
    customers = pd.read_csv(r'/content/global-fashion-dataset/customers.csv', low_memory=False)

    print("Loading transactions (this may take a moment)...")
    transactions = pd.read_csv(r'/content/global-fashion-dataset/transactions.csv', low_memory=False)
    print(f"Intial Transactions loaded: {len(transactions)} rows")
    transactions=transactions.drop_duplicates()
    print(f"Final Transactions loaded: {len(transactions)} rows")

    # Convert to polars for better performance
    print("Converting to Polars for better performance...")
    discounts_pl = pl.from_pandas(discounts)
    products_pl = pl.from_pandas(products)
    employees_pl = pl.from_pandas(employees)
    stores_pl = pl.from_pandas(stores)
    customers_pl = pl.from_pandas(customers)
    transactions_pl = pl.from_pandas(transactions)

    print(f" Data loaded successfully!")
    print(f"Transactions shape: {transactions_pl.shape}")

    return discounts_pl, products_pl, employees_pl, stores_pl, customers_pl, transactions_pl


def clean_and_translate_data(master_data, city_translation_map, country_translation_map):
    """
    Clean and translate city and country names into English using Polars DataFrame.
    Compatible with older Polars versions.
    """
    master_data = master_data.with_columns([
        pl.col("City").replace(city_translation_map).alias("City"),
        pl.col("Country").replace(country_translation_map).alias("Country")
    ])
    return master_data


def clean_date_columns(df, date_columns):
    """Clean and convert date columns to simple YYYY-MM-DD format"""
    print(f" Converting date columns to YYYY-MM-DD format: {date_columns}")

    for col in date_columns:
        if col in df.columns:
            try:
                # Convert to date (not datetime) - this removes time component automatically
                df = df.with_columns(
                    pl.col(col).str.strptime(pl.Date, format="%Y-%m-%d", strict=False)
                    .fill_null(
                        pl.col(col).str.strptime(pl.Date, format="%Y-%m-%d %H:%M:%S", strict=False)
                    )
                    .fill_null(
                        pl.col(col).str.strptime(pl.Date, format="%d/%m/%Y", strict=False)
                    )
                    .alias(col)
                )
                print(f" Converted {col} to YYYY-MM-DD date format")
            except Exception as e:
                print(f" Could not convert {col}: {e}")
                # Keep original if conversion fails
                pass

    return df

def prepare_lookup_tables(discounts_pl, products_pl, employees_pl, stores_pl, customers_pl):
    """Prepare minimal lookup tables with only essential information"""
    print("🔧 Preparing optimized lookup tables...")

    # Clean discount dates - Start and End to YYYY-MM-DD
    discounts_clean = clean_date_columns(discounts_pl, ['Start', 'End'])
    discounts_lookup = discounts_clean.select([
        'Start', 'End', 'Discont', 'Description', 'Category', 'Sub Category'
    ])

    # Products - keep essential info only
    products_lookup = products_pl.select([
        'Product ID', 'Category', 'Sub Category', 'Description EN',
        'Color', 'Sizes', 'Production Cost'
    ])

    # Employees - minimal info
    employees_lookup = employees_pl.select([
        'Employee ID', 'Store ID', 'Name', 'Position'
    ])

    # Stores - essential location info
    stores_pl = clean_and_translate_data(stores_pl, city_translation_map, country_translation_map)
    stores_lookup = stores_pl.select([
        'Store ID', 'Country', 'City', 'Store Name', 'Number of Employees'
    ])

    # Customers - clean birth date to YYYY-MM-DD and keep essential info
    customers_pl = clean_and_translate_data(customers_pl, city_translation_map, country_translation_map)
    customers_clean = clean_date_columns(customers_pl, ['Date Of Birth'])
    customers_lookup = customers_clean.select([
        'Customer ID', 'Name', 'Email', 'Gender', 'Date Of Birth', 'Job Title'
    ])

    print(" Lookup tables prepared")
    return discounts_lookup, products_lookup, employees_lookup, stores_lookup, customers_lookup

def process_transactions(transactions_pl, exchange_rates):
    """Process transactions with USD conversion and date cleaning"""
    print(" Processing transactions...")

    # Clean transaction dates - convert Date to YYYY-MM-DD format (removes time)

    transactions_clean = clean_date_columns(transactions_pl, ['Date'])

    # Add exchange rate column
    print("💱 Adding USD conversion...")
    transactions_clean = transactions_clean.with_columns(
        pl.col('Currency').map_elements(
            lambda x: exchange_rates.get(x, 1.0),
            return_dtype=pl.Float64
        ).alias("Exchange_Rate_to_USD")
    )

    # Convert monetary columns to USD
    monetary_columns = ['Unit Price', 'Line Total', 'Invoice Total']

    for col in monetary_columns:
        if col in transactions_clean.columns:
            usd_col_name = f"{col.replace(' ', '_')}_USD"
            transactions_clean = transactions_clean.with_columns(
                (pl.col(col) * pl.col("Exchange_Rate_to_USD")).alias(usd_col_name)
            )
            print(f" Created {usd_col_name}")

    return transactions_clean

def create_master_dataset():
    """Create the master dataset with optimized merging"""
    print(" Creating master dataset...")

    # Load data
    discounts_pl, products_pl, employees_pl, stores_pl, customers_pl, transactions_pl = load_data_efficiently()

    # Prepare lookup tables
    discounts_lookup, products_lookup, employees_lookup, stores_lookup, customers_lookup = prepare_lookup_tables(
        discounts_pl, products_pl, employees_pl, stores_pl, customers_pl
    )

    # Process transactions
    transactions_processed = process_transactions(transactions_pl, exchange_rates)

    # Merge with lookup tables (left joins to keep all transactions)
    print("🔗 Merging tables...")

    # Merge with products
    master_data = transactions_processed.join(
        products_lookup,
        on='Product ID',
        how='left'
    )
    print(f" Merged with products. Shape: {master_data.shape}")

    # Merge with customers
    master_data = master_data.join(
        customers_lookup,
        on='Customer ID',
        how='left'
    )
    print(f" Merged with customers. Shape: {master_data.shape}")

    # Merge with stores
    master_data = master_data.join(
        stores_lookup,
        on='Store ID',
        how='left'
    )
    print(f" Merged with stores. Shape: {master_data.shape}")

    # Merge with employees
    master_data = master_data.join(
        employees_lookup,
        on='Employee ID',
        how='left'
    )
    print(f" Merged with employees. Shape: {master_data.shape}")

    # Create data directory if it doesn't exist
    os.makedirs("data", exist_ok=True)

    # Save lookup tables separately for future use
    print(" Saving lookup tables...")
    discounts_lookup.write_parquet("data/discounts_lookup.parquet")
    products_lookup.write_parquet("data/products_lookup.parquet")
    employees_lookup.write_parquet("data/employees_lookup.parquet")
    stores_lookup.write_parquet("data/stores_lookup.parquet")
    customers_lookup.write_parquet("data/customers_lookup.parquet")

    # Save master dataset
    print("💾 Saving master dataset...")
    master_data.write_parquet("data/master_transactions.parquet")

    # Save a sample CSV for inspection (first 10k rows)
    master_data.head(10000).write_csv("data/master_transactions_sample.csv")

    print(" Master dataset created successfully!")
    print(f"Final shape: {master_data.shape}")
    print(f"Memory usage optimized by keeping only essential columns from lookup tables")

    return master_data, discounts_lookup

def show_dataset_summary(master_data):
    """Show summary of the final dataset"""
    print("\n MASTER DATASET SUMMARY:")
    print(f"Shape: {master_data.shape}")
    print(f"Columns: {len(master_data.columns)}")

    print("\n Column List:")
    for i, col in enumerate(master_data.columns, 1):
        print(f"{i:2d}. {col}")

    print(f"\n USD Converted Columns:")
    usd_cols = [col for col in master_data.columns if 'USD' in col]
    for col in usd_cols:
        print(f"   • {col}")

    print(f"\n Date Columns (YYYY-MM-DD format):")
    date_cols = [col for col in master_data.columns if master_data[col].dtype == pl.Date]
    for col in date_cols:
        print(f"   • {col}")
        # Show sample dates
        sample_dates = master_data.select(col).drop_nulls().head(3)
        print(f"     Sample: {sample_dates.to_series().to_list()}")

    print(f"\n Sample Data:")
    print(master_data.head(3))

    return master_data

# Execute the pipeline
print(" Starting Optimized Data Pipeline...")
master_data, discounts_lookup = create_master_dataset()
final_data = show_dataset_summary(master_data)

print("\n Pipeline Complete!")
print("Files saved:")
print("   • data/master_transactions.parquet (main dataset)")
print("   • data/master_transactions_sample.csv (sample for inspection)")
print("   • data/discounts_lookup.parquet (for discount analysis)")
print("   • data/products_lookup.parquet")
print("   • data/employees_lookup.parquet")
print("   • data/stores_lookup.parquet")
print("   • data/customers_lookup.parquet")
print("\n All dates are now in simple YYYY-MM-DD format!")


 Starting Optimized Data Pipeline...
 Creating master dataset...
 Loading data efficiently...
Loading discounts...
Loading products...
Loading employees...
Loading stores...
Loading customers...
Loading transactions (this may take a moment)...
Intial Transactions loaded: 6416827 rows
Final Transactions loaded: 6416029 rows
Converting to Polars for better performance...
 Data loaded successfully!
Transactions shape: (6416029, 19)
🔧 Preparing optimized lookup tables...
 Converting date columns to YYYY-MM-DD format: ['Start', 'End']
 Converted Start to YYYY-MM-DD date format
 Converted End to YYYY-MM-DD date format
 Converting date columns to YYYY-MM-DD format: ['Date Of Birth']
 Converted Date Of Birth to YYYY-MM-DD date format
 Lookup tables prepared
 Processing transactions...
 Converting date columns to YYYY-MM-DD format: ['Date']
 Converted Date to YYYY-MM-DD date format
💱 Adding USD conversion...
 Created Unit_Price_USD
 Created Line_Total_USD
 Created Invoice_Total_USD
🔗 Merging ta

## **Geographic , Trend & Discount Analysis**

In [ ]:

import polars as pl
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
from datetime import datetime
import os

def load_master_data_safely():
    """Safely load master data and check columns"""
    print("Loading master dataset safely...")

    try:
        # Try loading with coordinates first
        if os.path.exists("data/master_transactions_with_coords.parquet"):
            master_data = pl.read_parquet("data/master_transactions_with_coords.parquet")
            print(f" Master data with coordinates loaded: {master_data.shape}")
        else:
            # Load original master data
            master_data = pl.read_parquet("data/master_transactions.parquet")
            print(f"Master data loaded: {master_data.shape}")

            # Add coordinates if needed
            print("Adding coordinates...")
            stores_original = pd.read_csv(r'/content/global-fashion-dataset/stores.csv')
            stores_coords = pl.from_pandas(stores_original).select([
                'Store ID', 'Latitude', 'Longitude'
            ])

            master_data = master_data.join(stores_coords, on='Store ID', how='left')
            master_data.write_parquet("data/master_transactions_with_coords.parquet")
            print(" Coordinates added and saved!")

        # Verify key columns exist
        required_columns = ['Line_Total_USD', 'Unit_Price_USD', 'Quantity', 'Country', 'Transaction Type']
        missing_columns = [col for col in required_columns if col not in master_data.columns]

        if missing_columns:
            print(f" Missing columns: {missing_columns}")
            return None

        print(" All required columns found!")
        return master_data

    except Exception as e:
        print(f" Errror loading master data: {e}")
        return None

def prepare_optimized_geographic_data(master_data):
    """Prepare optimized data for geographic analysis using SIMPLE discount calculation"""
    print(" Preparing optimized geographic analysis data...")

    # Filter for sales only (exclude returns)
    sales_data = master_data.filter(pl.col("Transaction Type") == "Sale")
    print(f"Sales data shape: {sales_data.shape}")

    # Verify we have the required columns
    print(" Verifying required columns in sales data...")
    required_cols = ['Line_Total_USD', 'Unit_Price_USD', 'Quantity', 'Country', 'Date']
    for col in required_cols:
        if col not in sales_data.columns:
            print(f" Missing column: {col}")
            return None
        else:
            print(f" Found column: {col}")

    # Add time-based columns
    sales_data = sales_data.with_columns([
        pl.col("Date").dt.strftime("%Y-%m").alias("Year_Month"),
        pl.col("Date").dt.strftime("%Y-W%U").alias("Year_Week"),
        pl.col("Date").dt.weekday().alias("Day_of_Week_Num"),
        pl.col("Date").dt.strftime("%A").alias("Day_of_Week_Name")
    ])

    print(" Creating optimized data aggregations with SIMPLE discount calculation...")

    # 1. Monthly trends - ALL data (cumulative)
    print("Creating monthly trends...")
    monthly_trends = sales_data.group_by("Year_Month").agg([
        pl.col("Line_Total_USD").sum().alias("Monthly_Sales_USD"),
        pl.col("Quantity").sum().alias("Monthly_Quantity"),
        pl.col("Line_Total_USD").count().alias("Monthly_Transactions")
    ]).sort("Year_Month")

    # 2. Weekly trends - ALL data (cumulative)
    print("Creating weekly trends...")
    weekly_trends = sales_data.group_by("Year_Week").agg([
        pl.col("Line_Total_USD").sum().alias("Weekly_Sales_USD"),
        pl.col("Quantity").sum().alias("Weekly_Quantity"),
        pl.col("Line_Total_USD").count().alias("Weekly_Transactions")
    ]).sort("Year_Week")

    # 3. Monthly trends by country - ALL countries
    print("Creating monthly country trends...")
    monthly_country_trends = sales_data.group_by(["Country", "Year_Month"]).agg([
        pl.col("Line_Total_USD").sum().alias("Monthly_Sales_USD"),
        pl.col("Quantity").sum().alias("Monthly_Quantity")
    ]).sort(["Country", "Year_Month"])

    # 4. COMBINED Country-wise analysis using SIMPLE discount calculation
    print("Creating country combined analysis...")
    country_combined = sales_data.group_by("Country").agg([
        pl.col("Line_Total_USD").sum().alias("Total_Sales_USD"),
        pl.col("Quantity").sum().alias("Total_Quantity"),
        pl.col("Line_Total_USD").count().alias("Total_Transactions"),
        pl.col("Store ID").n_unique().alias("Number_of_Stores"),
        pl.col("Customer ID").n_unique().alias("Unique_Customers"),
        # SIMPLE discount calculation - Unit_Price_USD * Quantity = Total without discount
        (pl.col("Unit_Price_USD") * pl.col("Quantity")).sum().alias("Total_Without_Discount_USD")
    ]).with_columns([
        # Calculate average realization
        (pl.col("Total_Sales_USD") / pl.col("Total_Quantity")).alias("Avg_Realization_USD"),
        # Calculate simple average discount percentage
        ((pl.col("Total_Without_Discount_USD") - pl.col("Total_Sales_USD")) /
         pl.col("Total_Without_Discount_USD") * 100).alias("Avg_Discount_Percent")
    ]).sort("Total_Sales_USD", descending=True)


    # 5. COMBINED City-wise analysis - FIXED to use store-level data
    print("Creating city combined analysis...")
    # First get store-level aggregated data with discount calculation
    store_level_with_discount = sales_data.filter(
        pl.col("Store ID").is_not_null() &
        pl.col("Store Name").is_not_null() &
        pl.col("Country").is_not_null() &
        pl.col("City").is_not_null()
    ).group_by([
        "Store ID"
    ]).agg([
        pl.col("Store Name").first().alias("Store_Name"),
        pl.col("Country").first().alias("Country"),
        pl.col("City").first().alias("City"),
        pl.col("Line_Total_USD").sum().alias("Total_Sales_USD"),
        pl.col("Quantity").sum().alias("Total_Quantity"),
        pl.col("Line_Total_USD").count().alias("Total_Transactions"),
        pl.col("Customer ID").n_unique().alias("Unique_Customers"),
        (pl.col("Unit_Price_USD") * pl.col("Quantity")).sum().alias("Total_Without_Discount_USD")
    ]).with_columns([
        (pl.col("Total_Sales_USD") / pl.col("Total_Quantity")).alias("Avg_Realization_USD"),
        ((pl.col("Total_Without_Discount_USD") - pl.col("Total_Sales_USD")) /
         pl.col("Total_Without_Discount_USD") * 100).alias("Avg_Discount_Percent")
    ])

    # Then aggregate by city using store-level data
    city_combined = store_level_with_discount.group_by(["Country", "City"]).agg([
        pl.col("Total_Sales_USD").sum().alias("Total_Sales_USD"),
        pl.col("Total_Quantity").sum().alias("Total_Quantity"),
        pl.col("Total_Transactions").sum().alias("Total_Transactions"),
        pl.col("Store ID").count().alias("Number_of_Stores"),
        pl.col("Unique_Customers").sum().alias("Unique_Customers"),
        pl.col("Total_Without_Discount_USD").sum().alias("Total_Without_Discount_USD")
    ]).with_columns([
        (pl.col("Total_Sales_USD") / pl.col("Total_Quantity")).alias("Avg_Realization_USD"),
        ((pl.col("Total_Without_Discount_USD") - pl.col("Total_Sales_USD")) /
         pl.col("Total_Without_Discount_USD") * 100).alias("Avg_Discount_Percent")
    ]).sort("Total_Sales_USD", descending=True)


    # 6. Store-level for heatmap - ALL stores - FIXED
    print("Creating store heatmap data...")
    store_heatmap = sales_data.filter(
        pl.col("Store ID").is_not_null() &
        pl.col("Store Name").is_not_null() &
        pl.col("Country").is_not_null() &
        pl.col("City").is_not_null()
    ).group_by([
        "Store ID"
    ]).agg([
        pl.col("Store Name").first().alias("Store Name"),
        pl.col("Country").first().alias("Country"),
        pl.col("City").first().alias("City"),
        pl.col("Latitude").first().alias("Latitude"),
        pl.col("Longitude").first().alias("Longitude"),
        pl.col("Line_Total_USD").sum().alias("Total_Sales_USD"),
        pl.col("Quantity").sum().alias("Total_Quantity"),
        pl.col("Line_Total_USD").count().alias("Total_Transactions")
    ]).sort("Total_Sales_USD", descending=True)


    # 7. Day of week analysis - Sales and Quantity separately
    print("Creating day of week analysis...")
    day_of_week_sales = sales_data.group_by(["Day_of_Week_Num", "Day_of_Week_Name"]).agg([
        pl.col("Line_Total_USD").sum().alias("Total_Sales_USD")
    ]).sort("Day_of_Week_Num")

    day_of_week_quantity = sales_data.group_by(["Day_of_Week_Num", "Day_of_Week_Name"]).agg([
        pl.col("Quantity").sum().alias("Total_Quantity")
    ]).sort("Day_of_Week_Num")

    print(" Optimized geographic data prepared with SIMPLE discount calculation")
    print(" Discount calculation:")
    print("   • Total Without Discount = Unit_Price_USD × Quantity")
    print("   • Total With Discount = Line_Total_USD")
    print("   • Discount % = (Total Without Discount - Total With Discount) ÷ Total Without Discount × 100")

    return {
        "sales_data": sales_data,
        "monthly_trends": monthly_trends,
        "weekly_trends": weekly_trends,
        "monthly_country_trends": monthly_country_trends,
        "country_combined": country_combined,  # Combined data
        "city_combined": city_combined,        # Combined data
        "store_heatmap": store_heatmap,
        "day_of_week_sales": day_of_week_sales,
        "day_of_week_quantity": day_of_week_quantity
    }


def create_optimized_geographic_dashboard(data_dict):
    """Create optimized dashboard with combined visualizations"""
    print(" Creating optimized geographic dashboard...")

    # Convert all data to pandas for plotting
    monthly_trends_df = data_dict["monthly_trends"].to_pandas()
    weekly_trends_df = data_dict["weekly_trends"].to_pandas()
    monthly_country_df = data_dict["monthly_country_trends"].to_pandas()
    country_df = data_dict["country_combined"].to_pandas()  # Combined data
    city_df = data_dict["city_combined"].to_pandas()        # Combined data
    print(city_df.shape)
    store_df = data_dict["store_heatmap"].to_pandas()
    day_sales_df = data_dict["day_of_week_sales"].to_pandas()
    day_qty_df = data_dict["day_of_week_quantity"].to_pandas()

    # Create HTML content
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Optimized Geographic Analysis Dashboard</title>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        <style>
            body { font-family: Arial, sans-serif; margin: 20px; background-color: #f5f5f5; }
            .chart-container { background-color: white; margin: 20px 0; padding: 20px; border-radius: 10px; box-shadow: 0 2px 5px rgba(0,0,0,0.1); }
            .chart-title { font-size: 24px; font-weight: bold; text-align: center; margin-bottom: 20px; color: #333; }
            .dashboard-title { font-size: 36px; font-weight: bold; text-align: center; margin-bottom: 30px; color: #2c3e50; }
            .column-info { background-color: #e8f4f8; padding: 15px; margin: 10px 0; border-radius: 8px; font-size: 14px; }
        </style>
    </head>
    <body>
        <div class="dashboard-title"> Optimized Geographic Analysis Dashboard</div>
        <div class="column-info">
            <strong> Simple Discount Calculation:</strong><br>
            • <strong>Total Without Discount</strong>: Unit Price × Quantity<br>
            • <strong>Total With Discount</strong>: Line_Total_USD<br>
            • <strong>Discount %</strong>: (Total Without Discount - Total With Discount) ÷ Total Without Discount × 100
        </div>
    """

    # 1. Monthly Cumulative Trends
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(
        x=monthly_trends_df['Year_Month'],
        y=monthly_trends_df['Monthly_Sales_USD'],
        mode='lines+markers',
        name='Monthly Sales (USD)',
        line=dict(color='#1f77b4', width=3),
        marker=dict(size=8)
    ))
    fig1.update_layout(
        title=" Monthly Sales Trends (Cumulative)",
        xaxis_title="Month",
        yaxis_title="Sales (USD)",
        height=500,
        template="plotly_white"
    )
    fig1.update_xaxes(tickangle=45)

    # 2. Weekly Cumulative Trends
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(
        x=weekly_trends_df['Year_Week'],
        y=weekly_trends_df['Weekly_Sales_USD'],
        mode='lines+markers',
        name='Weekly Sales (USD)',
        line=dict(color='#ff7f0e', width=2),
        marker=dict(size=6)
    ))
    fig2.update_layout(
        title=" Weekly Sales Trends (Cumulative)",
        xaxis_title="Week",
        yaxis_title="Sales (USD)",
        height=500,
        template="plotly_white"
    )
    fig2.update_xaxes(tickangle=45)

    # 3. Monthly Trends by Country (Line Graph)
    fig3 = px.line(
        monthly_country_df,
        x="Year_Month",
        y="Monthly_Sales_USD",
        color="Country",
        title=" Monthly Sales Trends by Country",
        markers=True,
        height=600
    )
    fig3.update_xaxes(tickangle=45)
    fig3.update_layout(template="plotly_white")

    # 4. COMBINED Country Analysis - Sales & Quantity (bars) + Avg Realization (line)
    fig4 = make_subplots(specs=[[{"secondary_y": True}]])

    # Add Sales bars
    fig4.add_trace(
        go.Bar(
            x=country_df['Country'],
            y=country_df['Total_Sales_USD'],
            name='Total Sales (Line_Total_USD)',
            marker_color='lightblue',
            yaxis='y'
        ),
        secondary_y=False,
    )

    # Add Quantity bars
    fig4.add_trace(
        go.Bar(
            x=country_df['Country'],
            y=country_df['Total_Quantity'],
            name='Total Quantity',
            marker_color='lightgreen',
            yaxis='y'
        ),
        secondary_y=False,
    )

    # Add Average Realization line
    fig4.add_trace(
        go.Scatter(
            x=country_df['Country'],
            y=country_df['Avg_Realization_USD'],
            mode='lines+markers',
            name='Avg Realization (Line_Total_USD/Quantity)',
            line=dict(color='red', width=3),
            marker=dict(size=8),
            yaxis='y2'
        ),
        secondary_y=True,
    )

    fig4.update_xaxes(title_text="Country", tickangle=45)
    fig4.update_yaxes(title_text="Sales (USD) / Quantity", secondary_y=False)
    fig4.update_yaxes(title_text="Average Realization (USD)", secondary_y=True)
    fig4.update_layout(
        title_text="  Country Analysis: Sales, Quantity & Avg Realization",
        height=600,
        template="plotly_white"
    )

    # 5. COMBINED City Analysis - Sales & Quantity (bars) + Avg Realization (line)
    city_df['City_Country'] = city_df['City'] + ', ' + city_df['Country']
    fig5 = make_subplots(specs=[[{"secondary_y": True}]])

    # Add Sales bars
    fig5.add_trace(
        go.Bar(
            x=city_df['City_Country'],
            y=city_df['Total_Sales_USD'],
            name='Total Sales (Line_Total_USD)',
            marker_color='lightcoral',
            yaxis='y'
        ),
        secondary_y=False,
    )

    # Add Quantity bars
    fig5.add_trace(
        go.Bar(
            x=city_df['City_Country'],
            y=city_df['Total_Quantity'],
            name='Total Quantity',
            marker_color='lightpink',
            yaxis='y'
        ),
        secondary_y=False,
    )

    # Add Average Realization line
    fig5.add_trace(
        go.Scatter(
            x=city_df['City_Country'],
            y=city_df['Avg_Realization_USD'],
            mode='lines+markers',
            name='Avg Realization (Line_Total_USD/Quantity)',
            line=dict(color='orange', width=3),
            marker=dict(size=6),
            yaxis='y2'
        ),
        secondary_y=True,
    )

    fig5.update_xaxes(title_text="City, Country", tickangle=45)
    fig5.update_yaxes(title_text="Sales (USD) / Quantity", secondary_y=False)
    fig5.update_yaxes(title_text="Average Realization (USD)", secondary_y=True)
    fig5.update_layout(
        title_text="  City Analysis: Sales, Quantity & Avg Realization",
        height=700,
        template="plotly_white"
    )

    # 6. Store Heatmap (Scatter Map)
    store_clean = store_df.dropna(subset=['Latitude', 'Longitude'])
    fig6 = px.scatter_mapbox(
        store_clean,
        lat="Latitude",
        lon="Longitude",
        size="Total_Sales_USD",
        color="Total_Sales_USD",
        hover_name="Store Name",
        hover_data={
            "Country": True,
            "City": True,
            "Total_Sales_USD": ":,.0f",
            "Total_Quantity": ":,",
            "Total_Transactions": ":,"
        },
        color_continuous_scale="Plasma",
        size_max=50,
        zoom=1,
        title=" Store Sales Heatmap",
        height=700
    )
    fig6.update_layout(mapbox_style="open-street-map", template="plotly_white")

    # 7. Day of Week - Sales Analysis
    fig7 = go.Figure()
    fig7.add_trace(go.Bar(
        x=day_sales_df['Day_of_Week_Name'],
        y=day_sales_df['Total_Sales_USD'],
        name='Sales by Day',
        marker_color='skyblue'
    ))
    fig7.update_layout(
        title=" Sales by Day of Week",
        xaxis_title="Day of Week",
        yaxis_title="Total Sales (USD)",
        height=500,
        template="plotly_white"
    )

    # 8. Day of Week - Quantity Analysis
    fig8 = go.Figure()
    fig8.add_trace(go.Bar(
        x=day_qty_df['Day_of_Week_Name'],
        y=day_qty_df['Total_Quantity'],
        name='Quantity by Day',
        marker_color='lightseagreen'
    ))
    fig8.update_layout(
        title=" Quantity by Day of Week",
        xaxis_title="Day of Week",
        yaxis_title="Total Quantity",
        height=500,
        template="plotly_white"
    )

    # 9. SIMPLE Discount vs Sales by Country
    fig9 = px.scatter(
        country_df,
        x="Avg_Discount_Percent",
        y="Total_Sales_USD",
        text="Country",
        title=" Average Discount % vs Total Sales by Country<br><sub>Simple Calculation: (Unit Price × Quantity - Line_Total_USD) ÷ (Unit Price × Quantity) × 100</sub>",
        height=600
    )
    fig9.update_traces(textposition="top center")
    fig9.update_layout(
        xaxis_title="Average Discount (%)",
        yaxis_title="Total Sales (Line_Total_USD)",
        template="plotly_white"
    )

    # Convert all figures to HTML
    figures = [fig1, fig2, fig3, fig4, fig5, fig6, fig7, fig8, fig9]
    titles = [
        "Monthly Sales Trends (Cumulative)",
        "Weekly Sales Trends (Cumulative)",
        "Monthly Sales Trends by Country",
        "Combined Country Analysis: Sales, Quantity & Avg Realization",
        "Combined City Analysis: Sales, Quantity & Avg Realization",
        "Store Sales Heatmap",
        "Sales by Day of Week",
        "Quantity by Day of Week",
        "Simple Discount vs Sales by Country"
    ]

    for i, (fig, title) in enumerate(zip(figures, titles)):
        html_content += f"""
        <div class="chart-container">
            <div class="chart-title">{title}</div>
            <div id="chart{i+1}"></div>
        </div>
        """

    html_content += """
    <script>
    """

    # Add JavaScript for each plot
    for i, fig in enumerate(figures):
        plot_json = fig.to_json()
        html_content += f"""
        var plotData{i+1} = {plot_json};
        Plotly.newPlot('chart{i+1}', plotData{i+1}.data, plotData{i+1}.layout, {{responsive: true}});
        """

    html_content += """
    </script>
    </body>
    </html>
    """

    return html_content

def save_optimized_data_files(data_dict):
    """Save optimized combined data files"""
    print(" Saving optimized combined data files...")

    os.makedirs("geographic_analysis/data", exist_ok=True)

    # Save optimized datasets
    data_dict["monthly_trends"].write_csv("geographic_analysis/data/monthly_trends.csv")
    data_dict["weekly_trends"].write_csv("geographic_analysis/data/weekly_trends.csv")
    data_dict["monthly_country_trends"].write_csv("geographic_analysis/data/monthly_country_trends.csv")
    data_dict["country_combined"].write_csv("geographic_analysis/data/country_combined_analysis.csv")  # Combined
    data_dict["city_combined"].write_csv("geographic_analysis/data/city_combined_analysis.csv")        # Combined
    data_dict["store_heatmap"].write_csv("geographic_analysis/data/store_heatmap.csv")
    data_dict["day_of_week_sales"].write_csv("geographic_analysis/data/day_of_week_sales.csv")
    data_dict["day_of_week_quantity"].write_csv("geographic_analysis/data/day_of_week_quantity.csv")

    print(" Optimized data files saved with SIMPLE discount calculation!")

def run_optimized_geographic_analysis():
    """Run the optimized geographic analysis with simple discount calculation"""
    print(" Starting Optimized Geographic Analysis with SIMPLE Discount Calculation...")

    # Load data safely
    master_data = load_master_data_safely()
    if master_data is None:
        print(" Failed to load master data!")
        return None

    # Print column info for verification
    print(f"\n Master data shape: {master_data.shape}")
    print(" Key columns verified:")
    key_cols = ['Line_Total_USD', 'Unit Price', 'Quantity', 'Country', 'Transaction Type']
    for col in key_cols:
        if col in master_data.columns:
            print(f"    {col}")
        else:
            print(f"    {col} - MISSING!")

    # Prepare optimized data
    data_dict = prepare_optimized_geographic_data(master_data)
    if data_dict is None:
        print(" Failed to prepare data!")
        return None

    # Save optimized data files
    save_optimized_data_files(data_dict)

    # Create optimized dashboard
    html_content = create_optimized_geographic_dashboard(data_dict)

    # Create directory and save HTML
    os.makedirs("geographic_analysis", exist_ok=True)

    with open("geographic_analysis/optimized_geographic_dashboard.html", "w", encoding='utf-8') as f:
        f.write(html_content)

    # Print summary
    country_df = data_dict["country_combined"].to_pandas()
    city_df = data_dict["city_combined"].to_pandas()

    print("\n OPTIMIZED GEOGRAPHIC ANALYSIS COMPLETE!")
    print(f" Total Global Sales (Line_Total_USD): ${country_df['Total_Sales_USD'].sum():,.0f}")
    print(f" Total Quantity Sold: {country_df['Total_Quantity'].sum():,}")
    print(f" Countries Analyzed: {len(country_df)}")
    print(f" Cities Analyzed: {len(city_df)}")
    print(f" Global Avg Realization: ${country_df['Total_Sales_USD'].sum() / country_df['Total_Quantity'].sum():.2f}")
    print(f" Global Avg Discount: {((country_df['Total_Without_Discount_USD'].sum() - country_df['Total_Sales_USD'].sum()) / country_df['Total_Without_Discount_USD'].sum() * 100):.2f}%")

    print("\n Dashboard Created with SIMPLE Discount Calculation:")
    print("    geographic_analysis/optimized_geographic_dashboard.html")
    print("    geographic_analysis/data/ (Optimized CSV files)")

    print("\n  Discount Calculation Applied:")
    print("    Total Without Discount = Unit Price USD × Quantity")
    print("    Total With Discount = Line_Total_USD")
    print("    Discount % = (Total Without Discount - Total With Discount) ÷ Total Without Discount × 100")
    print("    Using actual column names from your data!")

    return data_dict

# Run the optimized analysis with simple discount calculation
results = run_optimized_geographic_analysis()


 Starting Optimized Geographic Analysis with SIMPLE Discount Calculation...
Loading master dataset safely...
 Master data with coordinates loaded: (6416029, 43)
 All required columns found!

 Master data shape: (6416029, 43)
 Key columns verified:
    Line_Total_USD
    Unit Price
    Quantity
    Country
    Transaction Type
 Preparing optimized geographic analysis data...
Sales data shape: (6077200, 43)
 Verifying required columns in sales data...
 Found column: Line_Total_USD
 Found column: Unit_Price_USD
 Found column: Quantity
 Found column: Country
 Found column: Date
 Creating optimized data aggregations with SIMPLE discount calculation...
Creating monthly trends...
Creating weekly trends...
Creating monthly country trends...
Creating country combined analysis...
Creating city combined analysis...
Creating store heatmap data...
Creating day of week analysis...
 Optimized geographic data prepared with SIMPLE discount calculation
 Discount calculation:
   • Total Without Discount 

In [ ]:
import polars as pl
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
import numpy as np
from datetime import datetime
import os

def load_master_data_for_pricing():
    """Load master data for pricing analysis"""
    print("Loading master data for pricing analysis...")

    try:
        if os.path.exists("data/master_transactions_with_coords.parquet"):
            master_data = pl.read_parquet("data/master_transactions_with_coords.parquet")
        else:
            master_data = pl.read_parquet("data/master_transactions.parquet")

        print(f"Master data loaded: {master_data.shape}")
        return master_data

    except Exception as e:
        print(f"Error loading master data: {e}")
        return None

def prepare_pricing_analysis_data(master_data):
    """Prepare comprehensive pricing and discount strategy analysis data"""
    print("Preparing pricing and discount strategy analysis data...")

    # Filter for sales only
    sales_data = master_data.filter(pl.col("Transaction Type") == "Sale")
    print(f"Sales data shape: {sales_data.shape}")

    # Add discount calculations
    sales_data = sales_data.with_columns([
        # Calculate discount amount and percentage
        (pl.col("Unit_Price_USD") * pl.col("Quantity") - pl.col("Line_Total_USD")).alias("Discount_Amount_USD"),
        ((pl.col("Unit_Price_USD") * pl.col("Quantity") - pl.col("Line_Total_USD")) /
         (pl.col("Unit_Price_USD") * pl.col("Quantity")) * 100).alias("Discount_Percent"),

        # Add time dimensions
        pl.col("Date").dt.strftime("%Y-%m").alias("Year_Month"),
        pl.col("Date").dt.quarter().alias("Quarter"),

        # Product identifier
        (pl.col("Product ID").cast(pl.Utf8) + "_" + pl.col("Size") + "_" + pl.col("Color")).alias("Product_SKU")
    ])

    print("Creating product-level aggregations...")

    # 1. Product Performance Matrix Data
    product_performance = sales_data.group_by([
        "Product ID", "Category", "Sub Category", "Description EN"
    ]).agg([
        # Revenue metrics
        pl.col("Line_Total_USD").sum().alias("Total_Revenue_USD"),
        pl.col("Quantity").sum().alias("Total_Quantity"),
        pl.col("Line_Total_USD").count().alias("Total_Transactions"),

        # Price metrics
        (pl.col("Line_Total_USD").sum() / pl.col("Quantity").sum()).alias("Avg_Unit_Price_USD"),

        # Discount metrics - WEIGHTED AVERAGE
        (pl.col("Discount_Amount_USD").sum() / (pl.col("Unit_Price_USD") * pl.col("Quantity")).sum() * 100).alias("Avg_Discount_Percent"),
        pl.col("Discount_Amount_USD").sum().alias("Total_Discount_USD"),

        # Customer metrics
        pl.col("Customer ID").n_unique().alias("Unique_Customers"),
        pl.col("Store ID").n_unique().alias("Stores_Available")
    ]).with_columns([
        # Calculate additional metrics
        (pl.col("Total_Revenue_USD") / pl.col("Total_Quantity")).alias("Revenue_Per_Unit"),
        (pl.col("Total_Discount_USD") / pl.col("Total_Quantity")).alias("Discount_Per_Unit"),
        (pl.col("Total_Revenue_USD") / pl.col("Unique_Customers")).alias("Revenue_Per_Customer")
    ])

    # Add percentile ranks for matrix positioning
    product_performance = product_performance.with_columns([
        pl.col("Total_Revenue_USD").rank(method="average").alias("Revenue_Rank"),
        pl.col("Avg_Discount_Percent").rank(method="average").alias("Discount_Rank")
    ]).with_columns([
        # Convert to percentile (0-1 scale)
        (pl.col("Revenue_Rank") / pl.col("Revenue_Rank").max()).alias("Revenue_Percentile_Rank"),
        (pl.col("Discount_Rank") / pl.col("Discount_Rank").max()).alias("Discount_Percentile_Rank")
    ]).sort("Total_Revenue_USD", descending=True)

    print("Creating price elasticity analysis...")

    # 2. Price Elasticity Analysis (Product + Size + Color level)
    price_elasticity = sales_data.group_by([
        "Product ID", "Size", "Color", "Category", "Sub Category"
    ]).agg([
        (pl.col("Line_Total_USD").sum() / pl.col("Quantity").sum()).alias("Avg_Unit_Price_USD"),  # Final price after discount
        pl.col("Quantity").sum().alias("Total_Quantity_Sold"),
        pl.col("Line_Total_USD").sum().alias("Total_Revenue_USD"),
        (pl.col("Discount_Amount_USD").sum() / (pl.col("Unit_Price_USD") * pl.col("Quantity")).sum() * 100).alias("Avg_Discount_Percent"),
        pl.col("Line_Total_USD").count().alias("Total_Transactions")
    ]).filter(
        # Filter for products with meaningful sales
        (pl.col("Total_Quantity_Sold") >= 10) &
        (pl.col("Total_Transactions") >= 5)
    ).sort("Total_Revenue_USD", descending=True)

    print("Creating discount effectiveness analysis...")

    # 3. Discount Effectiveness by Category
    discount_effectiveness = sales_data.group_by([
        "Category", "Sub Category"
    ]).agg([
        pl.col("Line_Total_USD").sum().alias("Total_Revenue_USD"),
        pl.col("Quantity").sum().alias("Total_Quantity"),
        (pl.col("Discount_Amount_USD").sum() / (pl.col("Unit_Price_USD") * pl.col("Quantity")).sum() * 100).alias("Avg_Discount_Percent"),
        (pl.col("Line_Total_USD").sum() / pl.col("Quantity").sum()).alias("Avg_Unit_Price_USD"),
        pl.col("Customer ID").n_unique().alias("Unique_Customers")
    ]).with_columns([
        (pl.col("Total_Revenue_USD") / pl.col("Total_Quantity")).alias("Revenue_Per_Unit"),
        (pl.col("Total_Revenue_USD") / pl.col("Unique_Customers")).alias("Revenue_Per_Customer")
    ]).sort("Total_Revenue_USD", descending=True)

    print("Creating discount bucket analysis...")

    # 4. Discount Bucket Analysis
    discount_buckets = sales_data.with_columns([
        pl.when(pl.col("Discount_Percent") <= 5)
        .then(pl.lit("0-5%"))
        .when(pl.col("Discount_Percent") <= 15)
        .then(pl.lit("6-15%"))
        .when(pl.col("Discount_Percent") <= 25)
        .then(pl.lit("16-25%"))
        .when(pl.col("Discount_Percent") <= 35)
        .then(pl.lit("26-35%"))
        .when(pl.col("Discount_Percent") <= 50)
        .then(pl.lit("36-50%"))
        .otherwise(pl.lit("50%+"))
        .alias("Discount_Bucket")
    ]).group_by("Discount_Bucket").agg([
        pl.col("Line_Total_USD").sum().alias("Total_Revenue_USD"),
        pl.col("Quantity").sum().alias("Total_Quantity"),
        pl.col("Line_Total_USD").count().alias("Total_Transactions"),
        pl.col("Customer ID").n_unique().alias("Unique_Customers")
    ]).with_columns([
        (pl.col("Total_Revenue_USD") / pl.col("Total_Quantity")).alias("Revenue_Per_Unit")
    ])

    print("Creating temporal pricing trends...")

    # 5. Monthly Pricing Trends
    monthly_pricing = sales_data.group_by("Year_Month").agg([
        (pl.col("Line_Total_USD").sum() / pl.col("Quantity").sum()).alias("Avg_Unit_Price_USD"),
        (pl.col("Discount_Amount_USD").sum() / (pl.col("Unit_Price_USD") * pl.col("Quantity")).sum() * 100).alias("Avg_Discount_Percent"),
        pl.col("Line_Total_USD").sum().alias("Total_Revenue_USD"),
        pl.col("Quantity").sum().alias("Total_Quantity")
    ]).sort("Year_Month")

    print("Pricing analysis data prepared successfully!")

    return {
        "sales_data": sales_data,
        "product_performance": product_performance,
        "price_elasticity": price_elasticity,
        "discount_effectiveness": discount_effectiveness,
        "discount_buckets": discount_buckets,
        "monthly_pricing": monthly_pricing
    }


def create_pricing_strategy_dashboard(data_dict):
    """Create comprehensive pricing and discount strategy dashboard"""
    print("Creating Pricing and Discount Strategy Dashboard...")

    # Convert to pandas for plotting
    product_perf_df = data_dict["product_performance"].to_pandas()
    price_elasticity_df = data_dict["price_elasticity"].to_pandas()
    discount_eff_df = data_dict["discount_effectiveness"].to_pandas()
    discount_buckets_df = data_dict["discount_buckets"].to_pandas()
    monthly_pricing_df = data_dict["monthly_pricing"].to_pandas()

    # Create HTML structure
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Pricing and Discount Strategy Optimization Dashboard</title>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        <style>
            body { font-family: Arial, sans-serif; margin: 20px; background-color: #f5f5f5; }
            .chart-container { background-color: white; margin: 20px 0; padding: 20px; border-radius: 10px; box-shadow: 0 2px 5px rgba(0,0,0,0.1); }
            .chart-title { font-size: 24px; font-weight: bold; text-align: center; margin-bottom: 20px; color: #333; }
            .dashboard-title { font-size: 36px; font-weight: bold; text-align: center; margin-bottom: 30px; color: #2c3e50; }
            .insights-box { background-color: #e8f4f8; padding: 15px; margin: 10px 0; border-radius: 8px; font-size: 14px; }
            .metric-highlight { background-color: #fff3cd; padding: 10px; margin: 5px 0; border-radius: 5px; font-weight: bold; }
        </style>
    </head>
    <body>
        <div class="dashboard-title">Pricing and Discount Strategy Optimization</div>

        <div class="insights-box">
            <strong>Key Insights:</strong><br>
            • <strong>Product Performance Matrix</strong>: Shows relationship between discount levels and revenue performance<br>
            • <strong>Price Elasticity</strong>: Analyzes how price changes affect quantity sold<br>
            • <strong>Discount Effectiveness</strong>: Measures ROI of discount strategies across categories<br>
            • <strong>Optimal Pricing</strong>: Identifies sweet spots for maximum profitability
        </div>
    """

    # 1. Product Performance Matrix (inspired by your chart)
    fig1 = px.scatter(
        product_perf_df,
        x="Revenue_Percentile_Rank",
        y="Discount_Percentile_Rank",
        size="Total_Revenue_USD",
        color="Total_Quantity",
        hover_name="Description EN",
        hover_data={
            "Category": True,
            "Sub Category": True,
            "Total_Revenue_USD": ":,.0f",
            "Total_Quantity": ":,",
            "Avg_Discount_Percent": ":.1f%",
            "Avg_Unit_Price_USD": ":.2f"
        },
        title="Product Performance Matrix: Revenue vs Discount Strategy",
        labels={
            "Revenue_Percentile_Rank": "Revenue Percentile Rank",
            "Discount_Percentile_Rank": "Discount Percentile Rank",
            "Total_Quantity": "Total Quantity"
        },
        color_continuous_scale="Viridis",
        height=700
    )

    # Add quadrant lines
    fig1.add_hline(y=0.5, line_dash="dash", line_color="gray", opacity=0.7)
    fig1.add_vline(x=0.5, line_dash="dash", line_color="gray", opacity=0.7)

    # Add quadrant labels
    fig1.add_annotation(x=0.25, y=0.75, text="High Discount<br>Low Revenue",
                       showarrow=False, font=dict(size=14, color="red"))
    fig1.add_annotation(x=0.75, y=0.75, text="High Discount<br>High Revenue",
                       showarrow=False, font=dict(size=14, color="orange"))
    fig1.add_annotation(x=0.25, y=0.25, text="Low Discount<br>Low Revenue",
                       showarrow=False, font=dict(size=14, color="gray"))
    fig1.add_annotation(x=0.75, y=0.25, text="Low Discount<br>High Revenue",
                       showarrow=False, font=dict(size=14, color="green"))

    fig1.update_layout(template="plotly_white")

    # 2. Product Price Elasticity (inspired by your chart)
    fig2 = px.scatter(
        price_elasticity_df,
        x="Avg_Unit_Price_USD",
        y="Total_Quantity_Sold",
        size="Total_Revenue_USD",
        color="Avg_Discount_Percent",
        hover_name="Product ID",
        hover_data={
            "Category": True,
            "Sub Category": True,
            "Size": True,
            "Color": True,
            "Total_Revenue_USD": ":,.0f",
            "Avg_Discount_Percent": ":.1f%"
        },
        title="Product Price Elasticity Analysis",
        labels={
            "Avg_Unit_Price_USD": "Average Unit Price (USD)",
            "Total_Quantity_Sold": "Total Quantity Sold",
            "Avg_Discount_Percent": "Avg Discount %"
        },
        color_continuous_scale="RdYlBu_r",
        height=700,
        log_y=True  # Log scale for better visualization
    )
    fig2.update_layout(template="plotly_white")

    # 3. Discount Effectiveness by Category
    fig3 = px.scatter(
        discount_eff_df,
        x="Avg_Discount_Percent",
        y="Revenue_Per_Unit",
        size="Total_Revenue_USD",
        color="Category",
        hover_name="Sub Category",
        hover_data={
            "Total_Revenue_USD": ":,.0f",
            "Total_Quantity": ":,",
            "Unique_Customers": ":,",
            "Avg_Unit_Price_USD": ":.2f"
        },
        title="Discount Effectiveness by Category",
        labels={
            "Avg_Discount_Percent": "Average Discount Percentage (%)",
            "Revenue_Per_Unit": "Revenue Per Unit (USD)"
        },
        height=600
    )
    fig3.update_layout(template="plotly_white")

    # 4. Discount Bucket Analysis
    discount_buckets_df['Discount_Order'] = discount_buckets_df['Discount_Bucket'].map({
        '0-5%': 1, '6-15%': 2, '16-25%': 3, '26-35%': 4, '36-50%': 5, '50%+': 6
    })
    discount_buckets_df = discount_buckets_df.sort_values('Discount_Order')

    fig4 = make_subplots(specs=[[{"secondary_y": True}]])

    # Add revenue bars
    fig4.add_trace(
        go.Bar(
            x=discount_buckets_df['Discount_Bucket'],
            y=discount_buckets_df['Total_Revenue_USD'],
            name='Total Revenue (USD)',
            marker_color='lightblue',
            yaxis='y'
        ),
        secondary_y=False,
    )

    # Add revenue per unit line
    fig4.add_trace(
        go.Scatter(
            x=discount_buckets_df['Discount_Bucket'],
            y=discount_buckets_df['Revenue_Per_Unit'],
            mode='lines+markers',
            name='Revenue Per Unit (USD)',
            line=dict(color='red', width=3),
            marker=dict(size=8),
            yaxis='y2'
        ),
        secondary_y=True,
    )

    fig4.update_xaxes(title_text="Discount Bucket")
    fig4.update_yaxes(title_text="Total Revenue (USD)", secondary_y=False)
    fig4.update_yaxes(title_text="Revenue Per Unit (USD)", secondary_y=True)
    fig4.update_layout(
        title_text="Revenue Performance by Discount Bucket",
        height=600,
        template="plotly_white"
    )

    # 5. Monthly Pricing Trends
    fig5 = make_subplots(specs=[[{"secondary_y": True}]])

    # Add average price line
    fig5.add_trace(
        go.Scatter(
            x=monthly_pricing_df['Year_Month'],
            y=monthly_pricing_df['Avg_Unit_Price_USD'],
            mode='lines+markers',
            name='Avg Unit Price (USD)',
            line=dict(color='blue', width=3),
            yaxis='y'
        ),
        secondary_y=False,
    )

    # Add discount percentage line
    fig5.add_trace(
        go.Scatter(
            x=monthly_pricing_df['Year_Month'],
            y=monthly_pricing_df['Avg_Discount_Percent'],
            mode='lines+markers',
            name='Avg Discount %',
            line=dict(color='red', width=3),
            yaxis='y2'
        ),
        secondary_y=True,
    )

    fig5.update_xaxes(title_text="Month", tickangle=45)
    fig5.update_yaxes(title_text="Average Unit Price (USD)", secondary_y=False)
    fig5.update_yaxes(title_text="Average Discount (%)", secondary_y=True)
    fig5.update_layout(
        title_text="Monthly Pricing and Discount Trends",
        height=600,
        template="plotly_white"
    )

    # 6. Category Performance Heatmap
    category_pivot = discount_eff_df.pivot_table(
        values='Revenue_Per_Unit',
        index='Category',
        columns='Sub Category',
        fill_value=0
    )

    fig6 = px.imshow(
        category_pivot.values,
        labels=dict(x="Sub Category", y="Category", color="Revenue Per Unit"),
        x=category_pivot.columns,
        y=category_pivot.index,
        title="Revenue Per Unit Heatmap by Category & Sub Category",
        color_continuous_scale="RdYlGn",
        height=600
    )
    fig6.update_layout(template="plotly_white")

    # 7. Price vs Quantity Correlation by Category
    fig7 = px.scatter(
        price_elasticity_df,
        x="Avg_Unit_Price_USD",
        y="Total_Quantity_Sold",
        color="Category",
        size="Total_Revenue_USD",
        hover_name="Product ID",
        title="Price vs Quantity Correlation by Category",
        labels={
            "Avg_Unit_Price_USD": "Average Unit Price (USD)",
            "Total_Quantity_Sold": "Total Quantity Sold"
        },
        height=600
    )
    fig7.update_layout(template="plotly_white")

    # Convert all figures to HTML
    figures = [fig1, fig2, fig3, fig4, fig5, fig6, fig7]
    titles = [
        "Product Performance Matrix: Revenue vs Discount Strategy",
        "Product Price Elasticity Analysis",
        "Discount Effectiveness by Category",
        "Revenue Performance by Discount Bucket",
        "Monthly Pricing and Discount Trends",
        "Revenue Per Unit Heatmap by Category & Sub Category",
        "Price vs Quantity Correlation by Category"
    ]

    for i, (fig, title) in enumerate(zip(figures, titles)):
        html_content += f"""
        <div class="chart-container">
            <div class="chart-title">{title}</div>
            <div id="chart{i+1}"></div>
        </div>
        """

    # Add key metrics summary
    total_revenue = product_perf_df['Total_Revenue_USD'].sum()
    avg_discount = product_perf_df['Avg_Discount_Percent'].mean()
    best_performing_category = discount_eff_df.loc[discount_eff_df['Revenue_Per_Unit'].idxmax(), 'Category']
    optimal_discount_bucket = discount_buckets_df.loc[discount_buckets_df['Revenue_Per_Unit'].idxmax(), 'Discount_Bucket']

    html_content += f"""
        <div class="insights-box">
            <strong>Key Performance Metrics:</strong><br>
            <div class="metric-highlight">Total Revenue: ${total_revenue:,.0f}</div>
            <div class="metric-highlight">Average Discount: {avg_discount:.1f}%</div>
            <div class="metric-highlight">Best Performing Category: {best_performing_category}</div>
            <div class="metric-highlight">Optimal Discount Range: {optimal_discount_bucket}</div>
        </div>
    """

    html_content += """
    <script>
    """

    # Add JavaScript for each plot
    for i, fig in enumerate(figures):
        plot_json = fig.to_json()
        html_content += f"""
        var plotData{i+1} = {plot_json};
        Plotly.newPlot('chart{i+1}', plotData{i+1}.data, plotData{i+1}.layout, {{responsive: true}});
        """

    html_content += """
    </script>
    </body>
    </html>
    """

    return html_content

def save_pricing_analysis_data(data_dict):
    """Save pricing analysis data files"""
    print("Saving pricing analysis data files...")

    os.makedirs("pricing_analysis/data", exist_ok=True)

    # Save all datasets
    data_dict["product_performance"].write_csv("pricing_analysis/data/product_performance_matrix.csv")
    data_dict["price_elasticity"].write_csv("pricing_analysis/data/price_elasticity_analysis.csv")
    data_dict["discount_effectiveness"].write_csv("pricing_analysis/data/discount_effectiveness.csv")
    data_dict["discount_buckets"].write_csv("pricing_analysis/data/discount_bucket_analysis.csv")
    data_dict["monthly_pricing"].write_csv("pricing_analysis/data/monthly_pricing_trends.csv")

    print("Pricing analysis data files saved successfully!")

def run_pricing_strategy_analysis():
    """Run comprehensive pricing and discount strategy analysis"""
    print("Starting Pricing and Discount Strategy Optimization Analysis...")

    # Load data
    master_data = load_master_data_for_pricing()
    if master_data is None:
        print("Failed to load master data!")
        return None

    # Prepare analysis data
    data_dict = prepare_pricing_analysis_data(master_data)
    if data_dict is None:
        print("Failed to prepare pricing data!")
        return None

    # Save data files
    save_pricing_analysis_data(data_dict)

    # Create dashboard
    html_content = create_pricing_strategy_dashboard(data_dict)

    # Create directory and save HTML
    os.makedirs("pricing_analysis", exist_ok=True)

    with open("pricing_analysis/pricing_strategy_dashboard.html", "w", encoding='utf-8') as f:
        f.write(html_content)

    # Print summary insights
    product_perf_df = data_dict["product_performance"].to_pandas()
    discount_buckets_df = data_dict["discount_buckets"].to_pandas()

    print("\nPRICING STRATEGY ANALYSIS COMPLETE!")
    print(f"Total Revenue Analyzed: ${product_perf_df['Total_Revenue_USD'].sum():,.0f}")
    print(f"Products Analyzed: {len(product_perf_df):,}")
    print(f"Average Discount Rate: {product_perf_df['Avg_Discount_Percent'].mean():.1f}%")
    print(f"Highest Revenue Product: {product_perf_df.iloc[0]['Description EN']}")

    # Find optimal discount bucket
    optimal_bucket = discount_buckets_df.loc[discount_buckets_df['Revenue_Per_Unit'].idxmax()]
    print(f"Optimal Discount Range: {optimal_bucket['Discount_Bucket']} (Revenue/Unit: ${optimal_bucket['Revenue_Per_Unit']:.2f})")

    print("\nDashboard and Data Files Created:")
    print("   pricing_analysis/pricing_strategy_dashboard.html")
    print("   pricing_analysis/data/ (CSV files for detailed analysis)")

    return data_dict

# Run the pricing strategy analysis
results = run_pricing_strategy_analysis()


Starting Pricing and Discount Strategy Optimization Analysis...
Loading master data for pricing analysis...
Master data loaded: (6416029, 41)
Preparing pricing and discount strategy analysis data...
Sales data shape: (6077200, 41)
Creating product-level aggregations...
Creating price elasticity analysis...
Creating discount effectiveness analysis...
Creating discount bucket analysis...
Creating temporal pricing trends...
Pricing analysis data prepared successfully!
Saving pricing analysis data files...
Pricing analysis data files saved successfully!
Creating Pricing and Discount Strategy Dashboard...

PRICING STRATEGY ANALYSIS COMPLETE!
Total Revenue Analyzed: $305,884,837
Products Analyzed: 17,940
Average Discount Rate: 8.9%
Highest Revenue Product: Men'S Blazer Of Light Fabric With Cut In Thin Stripes
Optimal Discount Range: 0-5% (Revenue/Unit: $52.59)

Dashboard and Data Files Created:
   pricing_analysis/pricing_strategy_dashboard.html
   pricing_analysis/data/ (CSV files for detai

## Workforce/Store Optimzation

In [ ]:
import polars as pl
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import os

def load_workforce_data():
    """Load data for workforce performance analysis"""
    print("Loading workforce data...")

    try:
        # Load master transactions with coordinates if available, otherwise use standard file
        if os.path.exists("data/master_transactions_with_coords.parquet"):
            master_data = pl.read_parquet("data/master_transactions_with_coords.parquet")
        else:
            master_data = pl.read_parquet("data/master_transactions.parquet")

        print(f"Master data loaded successfully: {master_data.shape}")
        return master_data

    except Exception as e:
        print(f"Error loading workforce data: {e}")
        return None

def prepare_separate_datasets(master_data):
    """Prepare separate, properly aggregated datasets for each visualization"""
    print("Preparing separate datasets for each visualization...")

    # Filter for sales transactions only
    sales_data = master_data.filter(pl.col("Transaction Type") == "Sale")
    print(f"Sales data shape: {sales_data.shape}")

    # Create output directory
    os.makedirs("workforce_analysis/data", exist_ok=True)

    # 1. STORE PERFORMANCE DATASET (For Fig1 & Fig2)
    print("Creating store performance dataset...")
    store_performance = sales_data.filter(
        pl.col("Store ID").is_not_null() &
        pl.col("Store Name").is_not_null() &
        pl.col("Country").is_not_null() &
        pl.col("City").is_not_null()
    ).group_by([
        "Store ID"
    ]).agg([
        pl.col("Store Name").first().alias("Store_Name"),
        pl.col("Country").first().alias("Country"),
        pl.col("City").first().alias("City"),
        pl.col("Line_Total_USD").sum().alias("Total_Revenue_USD"),
        pl.col("Quantity").sum().alias("Total_Quantity"),
        pl.col("Line_Total_USD").count().alias("Total_Transactions"),
        pl.col("Employee ID").n_unique().alias("Actual_Employees"),
        pl.col("Customer ID").n_unique().alias("Unique_Customers"),
        pl.col("Product ID").n_unique().alias("Unique_Products_Sold"),
    ]).with_columns([
        (pl.col("Total_Revenue_USD") / pl.col("Total_Transactions")).alias("Avg_Transaction_Value"),
        (pl.col("Total_Revenue_USD") / pl.col("Unique_Customers")).alias("Revenue_Per_Customer"),
        (pl.col("Total_Quantity") / pl.col("Total_Transactions")).alias("Items_Per_Transaction")
    ]).sort("Total_Revenue_USD", descending=True)

    print(f"Store Performance Dataset: {store_performance.shape}")
    store_performance.write_csv("workforce_analysis/data/store_performance.csv")

    # 2. STORE EFFICIENCY DATASET (For Fig3 - Efficiency Matrix)
    print("Creating store efficiency dataset...")
    store_efficiency = store_performance.with_columns([
        pl.col("Total_Revenue_USD").rank(method="average").alias("Revenue_Rank_Raw"),
        pl.col("Avg_Transaction_Value").rank(method="average").alias("Transaction_Value_Rank_Raw"),
        pl.col("Revenue_Per_Customer").rank(method="average").alias("Customer_Value_Rank_Raw")
    ]).with_columns([
        (pl.col("Revenue_Rank_Raw") / pl.col("Revenue_Rank_Raw").max()).alias("Revenue_Percentile"),
        (pl.col("Transaction_Value_Rank_Raw") / pl.col("Transaction_Value_Rank_Raw").max()).alias("Transaction_Value_Percentile"),
        (pl.col("Customer_Value_Rank_Raw") / pl.col("Customer_Value_Rank_Raw").max()).alias("Customer_Value_Percentile")
    ])

    print(f"Store Efficiency Dataset: {store_efficiency.shape}")
    store_efficiency.write_csv("workforce_analysis/data/store_efficiency.csv")

    # 3. EMPLOYEE PRODUCTIVITY DATASET (For Fig4)
    print("Creating employee productivity dataset...")
    employee_productivity = store_performance.with_columns([
        (pl.col("Total_Revenue_USD") / pl.col("Actual_Employees")).alias("Revenue_Per_Employee"),
        (pl.col("Total_Transactions") / pl.col("Actual_Employees")).alias("Transactions_Per_Employee"),
        (pl.col("Unique_Customers") / pl.col("Actual_Employees")).alias("Customers_Per_Employee")
    ])

    print(f"Employee Productivity Dataset: {employee_productivity.shape}")
    employee_productivity.write_csv("workforce_analysis/data/employee_productivity.csv")


    # 4. LOCATION PERFORMANCE DATASET (For Fig5) - FIXED
    print("Creating location performance dataset...")
    location_performance = store_performance.group_by([
        "Country", "City"
    ]).agg([
        pl.col("Store ID").count().alias("Number_of_Stores"),
        pl.col("Total_Revenue_USD").sum().alias("Total_Location_Revenue_USD"),
        pl.col("Total_Transactions").sum().alias("Total_Location_Transactions"),
        pl.col("Unique_Customers").sum().alias("Total_Location_Customers")
    ]).with_columns([
        (pl.col("Total_Location_Revenue_USD") / pl.col("Number_of_Stores")).alias("Avg_Revenue_Per_Store"),
        (pl.col("Total_Location_Transactions") / pl.col("Number_of_Stores")).alias("Avg_Transactions_Per_Store"),
        (pl.col("Total_Location_Customers") / pl.col("Number_of_Stores")).alias("Avg_Customers_Per_Store")
    ]).sort("Total_Location_Revenue_USD", descending=True)
    location_performance.write_csv("workforce_analysis/data/location_performance.csv")

    # 5. CUSTOMER EFFICIENCY DATASET (For Fig6)
    print("Creating customer efficiency dataset...")
    customer_efficiency = store_performance.select([
        "Store ID", "Store_Name", "Country", "City",
        "Unique_Customers", "Revenue_Per_Customer", "Total_Revenue_USD"
    ])

    print(f"Customer Efficiency Dataset: {customer_efficiency.shape}")
    customer_efficiency.write_csv("workforce_analysis/data/customer_efficiency.csv")

    print("All datasets created and saved successfully!")

    return {
        "store_performance": store_performance,
        "store_efficiency": store_efficiency,
        "employee_productivity": employee_productivity,
        "location_performance": location_performance,
        "customer_efficiency": customer_efficiency
    }

def create_workforce_dashboard_from_csv():
    """Create dashboard by loading separate CSV files for each visualization"""
    print("Creating dashboard from separate CSV datasets...")

    # Load each dataset separately
    store_perf_df = pd.read_csv("workforce_analysis/data/store_performance.csv")
    store_eff_df = pd.read_csv("workforce_analysis/data/store_efficiency.csv")
    employee_prod_df = pd.read_csv("workforce_analysis/data/employee_productivity.csv")
    location_perf_df = pd.read_csv("workforce_analysis/data/location_performance.csv")
    customer_eff_df = pd.read_csv("workforce_analysis/data/customer_efficiency.csv")

    print(f"Store Performance: {store_perf_df.shape}")
    print(f"Store Efficiency: {store_eff_df.shape}")
    print(f"Employee Productivity: {employee_prod_df.shape}")
    print(f"Location Performance: {location_perf_df.shape}")
    print(f"Customer Efficiency: {customer_eff_df.shape}")

    # Create HTML structure
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Workforce Optimization Dashboard</title>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        <style>
            body { font-family: Arial, sans-serif; margin: 20px; background-color: #f5f5f5; }
            .chart-container { background-color: white; margin: 20px 0; padding: 20px; border-radius: 10px; box-shadow: 0 2px 5px rgba(0,0,0,0.1); }
            .chart-title { font-size: 24px; font-weight: bold; text-align: center; margin-bottom: 20px; color: #333; }
            .dashboard-title { font-size: 36px; font-weight: bold; text-align: center; margin-bottom: 30px; color: #2c3e50; }
            .kpi-container { display: flex; justify-content: space-around; margin: 20px 0; }
            .kpi-box { background-color: #ffffff; padding: 20px; border-radius: 10px; text-align: center; box-shadow: 0 2px 5px rgba(0,0,0,0.1); min-width: 150px; }
            .kpi-value { font-size: 28px; font-weight: bold; color: #2c3e50; }
            .kpi-label { font-size: 14px; color: #7f8c8d; margin-top: 5px; }
            .insights-box { background-color: #e8f4f8; padding: 15px; margin: 10px 0; border-radius: 8px; font-size: 14px; }
        </style>
    </head>
    <body>
        <div class="dashboard-title">Workforce Optimization Dashboard</div>
    """

    # Calculate KPIs
    total_stores = len(store_perf_df)
    total_revenue = store_perf_df['Total_Revenue_USD'].sum()
    avg_revenue_per_store = store_perf_df['Total_Revenue_USD'].mean()
    avg_transaction_value = store_perf_df['Avg_Transaction_Value'].mean()
    total_employees_est = employee_prod_df['Actual_Employees'].sum()

    html_content += f"""
        <div class="kpi-container">
            <div class="kpi-box">
                <div class="kpi-value">{total_stores}</div>
                <div class="kpi-label">Total Stores</div>
            </div>
            <div class="kpi-box">
                <div class="kpi-value">${total_revenue:,.0f}</div>
                <div class="kpi-label">Total Revenue</div>
            </div>
            <div class="kpi-box">
                <div class="kpi-value">${avg_revenue_per_store:,.0f}</div>
                <div class="kpi-label">Avg Revenue/Store</div>
            </div>
            <div class="kpi-box">
                <div class="kpi-value">${avg_transaction_value:.2f}</div>
                <div class="kpi-label">Avg Transaction Value</div>
            </div>
            <div class="kpi-box">
                <div class="kpi-value">{total_employees_est}</div>
                <div class="kpi-label">Total Employees</div>
            </div>
        </div>
    """

    # Figure 1: Store Performance Matrix
    fig1 = px.scatter(
        store_perf_df.head(50),
        x="Total_Revenue_USD",
        y="Avg_Transaction_Value",
        size="Total_Transactions",
        color="Country",
        hover_name="Store_Name",
        hover_data={
            "City": True,
            "Total_Transactions": ":,",
            "Unique_Customers": ":,",
            "Revenue_Per_Customer": ":,.2f"
        },
        title="Store Performance: Revenue vs Transaction Value",
        labels={
            "Total_Revenue_USD": "Total Revenue (USD)",
            "Avg_Transaction_Value": "Average Transaction Value (USD)"
        },
        height=600
    )
    fig1.update_layout(template="plotly_white")

    # Figure 2: Top 20 Stores
    fig2 = px.bar(
        store_perf_df.head(20),
        x="Store_Name",
        y="Total_Revenue_USD",
        color="Country",
        title="Top 20 Store Performance by Revenue",
        labels={"Total_Revenue_USD": "Total Revenue (USD)"},
        height=600
    )
    fig2.update_xaxes(tickangle=45)
    fig2.update_layout(template="plotly_white")

    # Figure 3: Store Efficiency Matrix
    fig3 = px.scatter(
        store_eff_df,
        x="Revenue_Percentile",
        y="Customer_Value_Percentile",
        size="Total_Revenue_USD",
        color="Transaction_Value_Percentile",
        hover_name="Store_Name",
        hover_data={
            "Country": True,
            "City": True,
            "Total_Revenue_USD": ":,.0f",
            "Revenue_Per_Customer": ":,.2f"
        },
        title="Store Efficiency Matrix",
        labels={
            "Revenue_Percentile": "Revenue Performance (Percentile)",
            "Customer_Value_Percentile": "Customer Value Performance (Percentile)",
            "Transaction_Value_Percentile": "Transaction Value"
        },
        color_continuous_scale="RdYlBu_r",
        height=600,
        size_max=20
    )

    fig3.update_traces(
        marker=dict(sizemin=5, sizemode='diameter', line=dict(width=1, color='white'))
    )
    fig3.add_hline(y=0.5, line_dash="dash", line_color="gray", opacity=0.7)
    fig3.add_vline(x=0.5, line_dash="dash", line_color="gray", opacity=0.7)
    fig3.add_annotation(x=0.25, y=0.75, text="Low Revenue<br>High Customer Value",
                       showarrow=False, font=dict(size=10, color="gray"))
    fig3.add_annotation(x=0.75, y=0.75, text="High Revenue<br>High Customer Value",
                       showarrow=False, font=dict(size=10, color="gray"))
    fig3.add_annotation(x=0.25, y=0.25, text="Low Revenue<br>Low Customer Value",
                       showarrow=False, font=dict(size=10, color="gray"))
    fig3.add_annotation(x=0.75, y=0.25, text="High Revenue<br>Low Customer Value",
                       showarrow=False, font=dict(size=10, color="gray"))
    fig3.update_layout(template="plotly_white", showlegend=True,
                      coloraxis_colorbar=dict(title="Transaction Value Percentile", titleside="right"))


    # Figure 4: Employee Productivity
    fig4 = px.scatter(
        employee_prod_df.head(30),
        x="Revenue_Per_Employee",
        y="Transactions_Per_Employee",
        size="Total_Revenue_USD",
        color="Country",
        hover_name="Store_Name",
        hover_data={
            "City": True,
            "Actual_Employees": True,
            "Customers_Per_Employee": ":,.0f"
        },
        title="Employee Productivity Analysis",
        labels={
            "Revenue_Per_Employee": "Revenue Per Employee (USD)",
            "Transactions_Per_Employee": "Transactions Per Employee"
        },
        height=600
    )
    fig4.update_layout(template="plotly_white")

    # Figure 5: Location Performance
    fig5 = px.treemap(
        location_perf_df,
        path=['Country', 'City'],
        values='Total_Location_Revenue_USD',
        color='Avg_Revenue_Per_Store',
        hover_data={
            'Number_of_Stores': ':,',
            'Total_Location_Revenue_USD': ':,.0f',
            'Avg_Revenue_Per_Store': ':,.0f'
        },
        title='Location Performance by Revenue',
        color_continuous_scale='Viridis',
        height=600
    )
    fig5.update_layout(template="plotly_white")

    # Figure 6: Customer Efficiency
    fig6 = px.scatter(
        customer_eff_df,
        x="Unique_Customers",
        y="Revenue_Per_Customer",
        size="Total_Revenue_USD",
        color="Country",
        hover_name="Store_Name",
        title="Customer Efficiency: Volume vs Value",
        labels={
            "Unique_Customers": "Number of Unique Customers",
            "Revenue_Per_Customer": "Revenue Per Customer (USD)"
        },
        height=600
    )
    fig6.update_layout(template="plotly_white")

    # Add charts to HTML
    figures = [fig1, fig2, fig3, fig4, fig5, fig6]
    titles = [
        "Store Performance: Revenue vs Transaction Value",
        "Top 20 Store Performance by Revenue",
        "Store Efficiency Matrix",
        "Employee Productivity Analysis",
        "Location Performance by Revenue",
        "Customer Efficiency: Volume vs Value"
    ]

    for i, (fig, title) in enumerate(zip(figures, titles)):
        html_content += f"""
        <div class="chart-container">
            <div class="chart-title">{title}</div>
            <div id="chart{i+1}"></div>
        </div>
        """

    # Add insights
    best_store = store_perf_df.iloc[0]
    best_location = location_perf_df.iloc[0]
    avg_productivity = employee_prod_df['Revenue_Per_Employee'].mean()

    html_content += f"""
        <div class="insights-box">
            <strong>Key Workforce Insights:</strong><br>
            • <strong>Top Store:</strong> {best_store['Store_Name']} ({best_store['City']}, {best_store['Country']}) - ${best_store['Total_Revenue_USD']:,.0f}<br>
            • <strong>Best Location:</strong> {best_location['City']}, {best_location['Country']} ({best_location['Number_of_Stores']} stores)<br>
            • <strong>Avg Employee Productivity:</strong> ${avg_productivity:,.0f} revenue per employee<br>
            • <strong>Revenue Range:</strong> ${store_perf_df['Total_Revenue_USD'].min():,.0f} - ${store_perf_df['Total_Revenue_USD'].max():,.0f}
        </div>
    """

    html_content += "<script>"

    for i, fig in enumerate(figures):
        plot_json = fig.to_json()
        html_content += f"""
        var plotData{i+1} = {plot_json};
        Plotly.newPlot('chart{i+1}', plotData{i+1}.data, plotData{i+1}.layout, {{responsive: true}});
        """

    html_content += """
    </script>
    </body>
    </html>
    """

    return html_content

def run_workforce_analysis():
    """Run the complete workforce analysis pipeline"""
    print("Starting Workforce Analysis...")

    # Step 1: Load data
    master_data = load_workforce_data()
    if master_data is None:
        return None

    # Step 2: Prepare separate datasets and save to CSV
    datasets = prepare_separate_datasets(master_data)

    # Step 3: Create dashboard from CSV files
    html_content = create_workforce_dashboard_from_csv()

    # Step 4: Save dashboard
    with open("workforce_analysis/workforce_dashboard.html", "w", encoding='utf-8') as f:
        f.write(html_content)

    print("\nWorkforce Analysis Complete!")
    print("Files Created:")
    print("   Interactive Dashboard: workforce_analysis/workforce_dashboard.html")
    print("   Separate Data Files:")
    print("     - store_performance.csv")
    print("     - store_efficiency.csv")
    print("     - employee_productivity.csv")
    print("     - location_performance.csv")
    print("     - customer_efficiency.csv")

    return datasets

# Execute the analysis
results = run_workforce_analysis()


Starting Workforce Analysis...
Loading workforce data...
Master data loaded successfully: (6416029, 43)
Preparing separate datasets for each visualization...
Sales data shape: (6077200, 43)
Creating store performance dataset...
Store Performance Dataset: (35, 13)
Creating store efficiency dataset...
Store Efficiency Dataset: (35, 19)
Creating employee productivity dataset...
Employee Productivity Dataset: (35, 16)
Creating location performance dataset...
Creating customer efficiency dataset...
Customer Efficiency Dataset: (35, 7)
All datasets created and saved successfully!
Creating dashboard from separate CSV datasets...
Store Performance: (35, 13)
Store Efficiency: (35, 19)
Employee Productivity: (35, 16)
Location Performance: (35, 9)
Customer Efficiency: (35, 7)

Workforce Analysis Complete!
Files Created:
   Interactive Dashboard: workforce_analysis/workforce_dashboard.html
   Separate Data Files:
     - store_performance.csv
     - store_efficiency.csv
     - employee_productivity

Demographic Analysis

In [ ]:
import polars as pl
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import os
from datetime import datetime

def load_and_explore_data():
    """Safely load and explore the master data"""
    print("Loading Master Sales Data for Regional Demographics Analysis...")

    try:
        # Load the master dataset
        master_data = pl.read_parquet("data/master_transactions.parquet")
        print(f"Master data loaded successfully!")
        print(f"Shape: {master_data.shape[0]:,} rows × {master_data.shape[1]} columns")

        # Display column info with data types
        print(f"\nAvailable Columns with Data Types:")
        for i, (col, dtype) in enumerate(zip(master_data.columns, master_data.dtypes), 1):
            print(f"   {i:2d}. {col} ({dtype})")

        # Check key columns for regional analysis
        key_columns = ['Customer ID', 'City', 'Country', 'Gender', 'Date Of Birth', 'Line_Total_USD']
        print(f"\nChecking Key Columns for Regional Analysis:")
        for col in key_columns:
            if col in master_data.columns:
                dtype = master_data[col].dtype
                print(f"   {col} - Available ({dtype})")
            else:
                print(f"   {col} - Missing")

        # Show sample data for key columns
        if all(col in master_data.columns for col in ['City', 'Country', 'Customer ID']):
            print(f"\nSample Geographic Data:")
            sample_geo = master_data.select(['Customer ID', 'City', 'Country', 'Gender', 'Date Of Birth']).head(5)
            print(sample_geo)

        return master_data

    except Exception as e:
        print(f"Error loading data: {str(e)}")
        return None

def process_regional_data(master_data):
    """Process the data for regional demographics analysis"""
    print("Processing Data for Regional Demographics Analysis...")

    try:
        # Get current year for age calculation
        current_year = datetime.now().year

        # Check the data type of Date Of Birth column
        dob_dtype = master_data['Date Of Birth'].dtype
        print(f"   Date Of Birth column type: {dob_dtype}")

        # Process the data step by step
        print("   Processing dates and calculating ages...")

        # Handle Date Of Birth based on its actual data type
        if str(dob_dtype) == 'Date':
            # If it's already a Date type, extract year directly
            processed_data = master_data.with_columns([
                pl.when(pl.col("Date Of Birth").is_not_null())
                .then(pl.lit(current_year) - pl.col("Date Of Birth").dt.year())
                .otherwise(None)
                .alias("Age")
            ])
        elif 'Utf8' in str(dob_dtype) or 'String' in str(dob_dtype):
            # If it's a string type, parse it first
            processed_data = master_data.with_columns([
                pl.when(pl.col("Date Of Birth").is_not_null())
                .then(
                    pl.lit(current_year) -
                    pl.col("Date Of Birth").str.strptime(pl.Date, "%Y-%m-%d", strict=False).dt.year()
                )
                .otherwise(None)
                .alias("Age")
            ])
        else:
            # For other types, try to convert to string first then parse
            processed_data = master_data.with_columns([
                pl.when(pl.col("Date Of Birth").is_not_null())
                .then(
                    pl.lit(current_year) -
                    pl.col("Date Of Birth").cast(pl.Utf8).str.strptime(pl.Date, "%Y-%m-%d", strict=False).dt.year()
                )
                .otherwise(None)
                .alias("Age")
            ])

        print(f"   Age calculation completed!")

        # Add age groups
        print("   Creating age groups...")
        processed_data = processed_data.with_columns([
            pl.when(pl.col("Age").is_null()).then(pl.lit("Unknown"))
            .when(pl.col("Age") < 18).then(pl.lit("Under 18"))
            .when(pl.col("Age") < 25).then(pl.lit("18-24"))
            .when(pl.col("Age") < 35).then(pl.lit("25-34"))
            .when(pl.col("Age") < 45).then(pl.lit("35-44"))
            .when(pl.col("Age") < 55).then(pl.lit("45-54"))
            .when(pl.col("Age") < 65).then(pl.lit("55-64"))
            .otherwise(pl.lit("65+"))
            .alias("Age_Group")
        ])

        # Add spending categories based on Line_Total_USD
        print("   Creating spending categories...")
        processed_data = processed_data.with_columns([
            pl.when(pl.col("Line_Total_USD").is_null()).then(pl.lit("Unknown"))
            .when(pl.col("Line_Total_USD") < 50).then(pl.lit("Low Spender"))
            .when(pl.col("Line_Total_USD") < 150).then(pl.lit("Medium Spender"))
            .when(pl.col("Line_Total_USD") < 300).then(pl.lit("High Spender"))
            .otherwise(pl.lit("Premium Spender"))
            .alias("Spending_Category")
        ])

        # Add purchase categories based on Invoice_Total_USD
        print("   Creating purchase categories...")
        processed_data = processed_data.with_columns([
            pl.when(pl.col("Invoice_Total_USD").is_null()).then(pl.lit("Unknown"))
            .when(pl.col("Invoice_Total_USD") < 100).then(pl.lit("Small Purchase"))
            .when(pl.col("Invoice_Total_USD") < 500).then(pl.lit("Medium Purchase"))
            .when(pl.col("Invoice_Total_USD") < 1000).then(pl.lit("Large Purchase"))
            .otherwise(pl.lit("Premium Purchase"))
            .alias("Purchase_Category")
        ])

        print(f"Data processing completed!")
        print(f"Processed {processed_data.shape[0]:,} records")

        # Show processing summary
        print(f"\nProcessing Summary:")
        print(f"   Countries: {processed_data['Country'].n_unique()}")
        print(f"   Cities: {processed_data['City'].n_unique()}")
        print(f"   Unique Customers: {processed_data['Customer ID'].n_unique():,}")
        print(f"   Total Transactions: {processed_data.shape[0]:,}")

        # Show age distribution
        age_stats = processed_data.group_by("Age_Group").agg(pl.count().alias("Count")).sort("Count", descending=True)
        print(f"\nAge Group Distribution:")
        for row in age_stats.iter_rows():
            print(f"   {row[0]}: {row[1]:,} customers")

        return processed_data

    except Exception as e:
        print(f"Error processing data: {str(e)}")
        print(f"   Debug info: Date Of Birth dtype = {master_data['Date Of Birth'].dtype}")
        return None

def analyze_regional_demographics(data):
    """Analyze regional demographics"""
    print("Analyzing Regional Demographics...")

    try:
        # City-level analysis
        print("   Analyzing city-level demographics...")
        city_stats = data.group_by(["City", "Country"]).agg([
            pl.col("Customer ID").n_unique().alias("Unique_Customers"),
            pl.count().alias("Total_Transactions"),
            pl.col("Age").mean().alias("Avg_Age"),
            pl.col("Line_Total_USD").sum().alias("Total_Revenue_USD"),
            pl.col("Line_Total_USD").mean().alias("Avg_Transaction_Value_USD"),
            pl.col("Invoice_Total_USD").mean().alias("Avg_Invoice_Value_USD"),
            pl.col("Quantity").sum().alias("Total_Items_Sold")
        ]).with_columns([
            (pl.col("Total_Transactions") / pl.col("Unique_Customers")).alias("Transactions_Per_Customer"),
            (pl.col("Total_Revenue_USD") / pl.col("Unique_Customers")).alias("Revenue_Per_Customer")
        ]).sort("Total_Revenue_USD", descending=True)

        # Country-level analysis
        print("   Analyzing country-level demographics...")
        country_stats = data.group_by("Country").agg([
            pl.col("Customer ID").n_unique().alias("Unique_Customers"),
            pl.count().alias("Total_Transactions"),
            pl.col("City").n_unique().alias("Unique_Cities"),
            pl.col("Age").mean().alias("Avg_Age"),
            pl.col("Line_Total_USD").sum().alias("Total_Revenue_USD"),
            pl.col("Line_Total_USD").mean().alias("Avg_Transaction_Value_USD"),
            pl.col("Store ID").n_unique().alias("Unique_Stores")
        ]).sort("Total_Revenue_USD", descending=True)

        print(f"Regional analysis completed!")
        print(f"   {city_stats.shape[0]} cities analyzed")
        print(f"   {country_stats.shape[0]} countries analyzed")

        # Show top 5 cities
        print(f"\nTop 5 Cities by Revenue:")
        top_cities = city_stats.head(5)
        for row in top_cities.iter_rows():
            print(f"   {row[0]}, {row[1]}: ${row[4]:,.2f} USD ({row[2]:,} customers)")

        return city_stats, country_stats

    except Exception as e:
        print(f"Error in regional analysis: {str(e)}")
        return None, None

def save_analysis_data(data, city_stats, country_stats):
    """Save analysis data to CSV files"""
    print("Saving Analysis Data...")

    try:
        # Create output directory
        os.makedirs("regional_demographics", exist_ok=True)

        # Save processed data (sample)
        print("   Saving processed data sample...")
        data_sample = data.select([
            "Customer ID", "City", "Country", "Gender", "Age", "Age_Group",
            "Line_Total_USD", "Spending_Category", "Invoice_Total_USD", "Purchase_Category"
        ]).head(10000)  # Save first 10k records as sample
        data_sample.write_csv("regional_demographics/processed_data_sample.csv")

        # Save city statistics
        print("   Saving city statistics...")
        city_stats.write_csv("regional_demographics/city_statistics.csv")

        # Save country statistics
        print("   Saving country statistics...")
        country_stats.write_csv("regional_demographics/country_statistics.csv")

        # Create summary statistics
        print("   Creating summary statistics...")
        summary_stats = {
            'analysis_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'total_countries': country_stats.shape[0],
            'total_cities': city_stats.shape[0],
            'total_customers': data['Customer ID'].n_unique(),
            'total_transactions': data.shape[0],
            'total_revenue_usd': float(data['Line_Total_USD'].sum())
        }

        # Save summary as text file
        with open("regional_demographics/summary_statistics.txt", "w") as f:
            f.write("REGIONAL DEMOGRAPHICS ANALYSIS SUMMARY\n")
            f.write("=" * 50 + "\n\n")
            for key, value in summary_stats.items():
                f.write(f"{key}: {value}\n")

        print("All analysis data saved successfully!")
        return summary_stats

    except Exception as e:
        print(f"Error saving data: {str(e)}")
        return None

def create_regional_visualizations(data, city_stats, country_stats):
    """Create regional demographic visualizations"""
    print("Creating Regional Demographics Visualizations...")

    figures = []

    try:
        # 1. Top Cities by Revenue
        print("   Creating top cities by revenue chart...")
        top_20_cities = city_stats.head(20).to_pandas()
        top_20_cities['City_Country'] = top_20_cities['City'] + ', ' + top_20_cities['Country']

        fig_cities_revenue = px.bar(
            top_20_cities,
            x='Total_Revenue_USD',
            y='City_Country',
            orientation='h',
            title='Top 20 Cities by Total Revenue (USD)',
            color='Avg_Transaction_Value_USD',
            color_continuous_scale='Viridis',
            text='Total_Revenue_USD'
        )
        fig_cities_revenue.update_traces(texttemplate='$%{text:,.0f}', textposition='outside')
        fig_cities_revenue.update_layout(
            height=700,
            title_font_size=18,
            yaxis={'categoryorder': 'total ascending'},
            xaxis_title="Total Revenue (USD)"
        )
        figures.append(fig_cities_revenue)

        # 2. Country Distribution
        print("   Creating country distribution chart...")
        country_data = country_stats.to_pandas()

        fig_countries = px.pie(
            country_data,
            values='Unique_Customers',
            names='Country',
            title='Customer Distribution by Country',
            color_discrete_sequence=px.colors.qualitative.Set3
        )
        fig_countries.update_traces(textposition='inside', textinfo='percent+label')
        fig_countries.update_layout(height=500, title_font_size=18)
        figures.append(fig_countries)

        # 3. Customer Performance Scatter
        print("   Creating customer performance scatter plot...")
        fig_performance = px.scatter(
            top_20_cities,
            x='Unique_Customers',
            y='Total_Revenue_USD',
            size='Avg_Invoice_Value_USD',
            color='Revenue_Per_Customer',
            hover_name='City_Country',
            title='Customer Performance Metrics by City',
            labels={
                'Unique_Customers': 'Number of Unique Customers',
                'Total_Revenue_USD': 'Total Revenue (USD)',
                'Revenue_Per_Customer': 'Revenue per Customer (USD)'
            },
            color_continuous_scale='Viridis'
        )
        fig_performance.update_layout(height=600, title_font_size=18)
        figures.append(fig_performance)

        # 4. Age Distribution by Top Cities
        print("   Creating age distribution chart...")
        top_10_cities_list = city_stats.head(10).select(["City", "Country"]).to_pandas()

        age_city_data = []
        for _, row in top_10_cities_list.iterrows():
            city_data = data.filter(
                (pl.col("City") == row['City']) & (pl.col("Country") == row['Country'])
            )
            age_dist = city_data.group_by("Age_Group").agg(pl.count().alias("Count")).to_pandas()
            total_count = age_dist['Count'].sum()
            for _, age_row in age_dist.iterrows():
                percentage = round((age_row['Count'] / total_count) * 100)
                age_city_data.append({
                    'City_Country': f"{row['City']}, {row['Country']}",
                    'Age_Group': age_row['Age_Group'],
                    'Count': age_row['Count'],
                    'Percentage': f"{percentage}%"
                })

        if age_city_data:
            age_city_df = pl.DataFrame(age_city_data).to_pandas()

            fig_age_cities = px.bar(
                age_city_df,
                x='City_Country',
                y='Count',
                color='Age_Group',
                title='Age Distribution by Top 10 Cities',
                color_discrete_sequence=px.colors.qualitative.Pastel,
                text='Percentage'
            )
            fig_age_cities.update_traces(textposition='inside')
            fig_age_cities.update_layout(
                height=600,
                title_font_size=18,
                xaxis_tickangle=-45
            )
            figures.append(fig_age_cities)

        # 5. Gender Distribution
        print("   Creating gender distribution chart...")
        gender_city_data = []
        for _, row in top_10_cities_list.iterrows():
            city_data = data.filter(
                (pl.col("City") == row['City']) & (pl.col("Country") == row['Country'])
            )
            gender_dist = city_data.group_by("Gender").agg(pl.count().alias("Count")).to_pandas()
            total_count = gender_dist['Count'].sum()
            for _, gender_row in gender_dist.iterrows():
                percentage = round((gender_row['Count'] / total_count) * 100)
                gender_city_data.append({
                    'City_Country': f"{row['City']}, {row['Country']}",
                    'Gender': gender_row['Gender'],
                    'Count': gender_row['Count'],
                    'Percentage': f"{percentage}%"
                })

        if gender_city_data:
            gender_city_df = pl.DataFrame(gender_city_data).to_pandas()

            fig_gender_cities = px.bar(
                gender_city_df,
                x='City_Country',
                y='Count',
                color='Gender',
                title='Gender Distribution by Top 10 Cities',
                color_discrete_sequence=['#FF69B4', '#4169E1', '#32CD32'],
                text='Percentage'
            )
            fig_gender_cities.update_traces(textposition='inside')
            fig_gender_cities.update_layout(
                height=600,
                title_font_size=18,
                xaxis_tickangle=-45
            )
            figures.append(fig_gender_cities)

        # 6. Revenue Heatmap by Country and Age Group
        print("   Creating revenue heatmap...")

        # Get top 10 countries and create heatmap data
        top_countries = country_stats.head(10).select("Country").to_series().to_list()
        age_groups = ['18-24', '25-34', '35-44', '45-54', '55-64', '65+']

        heatmap_data = []
        heatmap_countries = []

        for country in top_countries:
            country_data = data.filter(pl.col("Country") == country)
            row_data = []
            for age_group in age_groups:
                revenue = country_data.filter(pl.col("Age_Group") == age_group)["Line_Total_USD"].sum()
                row_data.append(float(revenue))
            heatmap_data.append(row_data)
            heatmap_countries.append(country)

        fig_heatmap = go.Figure(data=go.Heatmap(
            z=heatmap_data,
            x=age_groups,
            y=heatmap_countries,
            colorscale='Viridis',
            text=[[f"${val:,.0f}" for val in row] for row in heatmap_data],
            texttemplate="%{text}",
            textfont={"size": 10}
        ))

        fig_heatmap.update_layout(
            title='Revenue Heatmap by Country and Age Group',
            title_font_size=18,
            height=500,
            xaxis_title="Age Group",
            yaxis_title="Country"
        )
        figures.append(fig_heatmap)

        print(f"Created {len(figures)} visualizations successfully!")
        return figures

    except Exception as e:
        print(f"Error creating visualizations: {str(e)}")
        return figures

def create_dashboard(figures, summary_stats):
    """Create HTML dashboard"""
    print("Creating Regional Demographics Dashboard...")

    try:
        html_content = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>Regional Demographics Analysis Dashboard</title>
            <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
            <style>
                body {{
                    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                    margin: 0;
                    padding: 20px;
                    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    min-height: 100vh;
                }}
                .container {{
                    max-width: 1400px;
                    margin: 0 auto;
                    background: white;
                    border-radius: 15px;
                    padding: 30px;
                    box-shadow: 0 20px 40px rgba(0,0,0,0.1);
                }}
                .header {{
                    text-align: center;
                    margin-bottom: 40px;
                    padding: 20px;
                    background: linear-gradient(135deg, #4CAF50, #45a049);
                    border-radius: 10px;
                    color: white;
                }}
                .stats-grid {{
                    display: grid;
                    grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
                    gap: 20px;
                    margin-bottom: 30px;
                }}
                .stat-card {{
                    background: #f8f9fa;
                    padding: 20px;
                    border-radius: 10px;
                    text-align: center;
                    border-left: 4px solid #4CAF50;
                }}
                .chart-container {{
                    margin: 30px 0;
                    padding: 20px;
                    border: 2px solid #e0e0e0;
                    border-radius: 10px;
                    background: #fafafa;
                }}
                .chart {{
                    width: 100%;
                    height: 600px;
                }}
                .footer {{
                    text-align: center;
                    margin-top: 40px;
                    padding: 20px;
                    background: #f5f5f5;
                    border-radius: 10px;
                    color: #666;
                }}
            </style>
        </head>
        <body>
            <div class="container">
                <div class="header">
                    <h1>Regional Demographics Analysis Dashboard</h1>
                    <p>Comprehensive Geographic Analysis of Customer Data</p>
                    <p>Generated on: {summary_stats['analysis_date']}</p>
                </div>

                <div class="stats-grid">
                    <div class="stat-card">
                        <h3>Countries</h3>
                        <h2>{summary_stats['total_countries']}</h2>
                    </div>
                    <div class="stat-card">
                        <h3>Cities</h3>
                        <h2>{summary_stats['total_cities']}</h2>
                    </div>
                    <div class="stat-card">
                        <h3>Customers</h3>
                        <h2>{summary_stats['total_customers']:,}</h2>
                    </div>
                    <div class="stat-card">
                        <h3>Transactions</h3>
                        <h2>{summary_stats['total_transactions']:,}</h2>
                    </div>
                    <div class="stat-card">
                        <h3>Total Revenue</h3>
                        <h2>${summary_stats['total_revenue_usd']:,.0f}</h2>
                    </div>
                </div>
        """

        # Add chart containers
        for i in range(len(figures)):
            html_content += f"""
                <div class="chart-container">
                    <div id="chart{i+1}" class="chart"></div>
                </div>
            """

        html_content += """
                <div class="footer">
                    <p>Regional Demographics Analysis | Based on Actual Transaction Data</p>
                </div>
            </div>
            <script>
        """

        # Add JavaScript for each plot
        for i, fig in enumerate(figures):
            plot_json = fig.to_json()
            html_content += f"""
            var plotData{i+1} = {plot_json};
            Plotly.newPlot('chart{i+1}', plotData{i+1}.data, plotData{i+1}.layout, {{responsive: true}});
            """

        html_content += """
            </script>
        </body>
        </html>
        """

        # Save dashboard
        with open("regional_demographics/regional_demographics_dashboard.html", "w", encoding='utf-8') as f:
            f.write(html_content)

        print("Dashboard created successfully!")
        return True

    except Exception as e:
        print(f"Error creating dashboard: {str(e)}")
        return False

def run_regional_demographics_analysis():
    """Run complete regional demographics analysis"""
    print("Starting Regional Demographics Analysis...")
    print("=" * 60)

    # Step 1: Load and explore data
    master_data = load_and_explore_data()
    if master_data is None:
        return None

    # Step 2: Process data
    processed_data = process_regional_data(master_data)
    if processed_data is None:
        return None

    # Step 3: Analyze regional demographics
    city_stats, country_stats = analyze_regional_demographics(processed_data)
    if city_stats is None or country_stats is None:
        return None

    # Step 4: Save analysis data
    summary_stats = save_analysis_data(processed_data, city_stats, country_stats)
    if summary_stats is None:
        return None

    # Step 5: Create visualizations
    figures = create_regional_visualizations(processed_data, city_stats, country_stats)

    # Step 6: Create dashboard
    dashboard_success = create_dashboard(figures, summary_stats)

    if dashboard_success:
        print("\n" + "=" * 60)
        print("REGIONAL DEMOGRAPHICS ANALYSIS COMPLETED!")
        print("=" * 60)
        print("\nFILES CREATED:")
        print("   regional_demographics/regional_demographics_dashboard.html")
        print("   regional_demographics/city_statistics.csv")
        print("   regional_demographics/country_statistics.csv")
        print("   regional_demographics/processed_data_sample.csv")
        print("   regional_demographics/summary_statistics.txt")

        print(f"\nANALYSIS SUMMARY:")
        print(f"   Countries Analyzed: {summary_stats['total_countries']}")
        print(f"   Cities Analyzed: {summary_stats['total_cities']}")
        print(f"   Unique Customers: {summary_stats['total_customers']:,}")
        print(f"   Total Transactions: {summary_stats['total_transactions']:,}")
        print(f"   Total Revenue: ${summary_stats['total_revenue_usd']:,.2f} USD")

        print(f"\nOpen 'regional_demographics/regional_demographics_dashboard.html' to view the complete analysis!")

        return {
            'processed_data': processed_data,
            'city_stats': city_stats,
            'country_stats': country_stats,
            'figures': figures,
            'summary_stats': summary_stats
        }
    else:
        return None

# Execute Regional Demographics Analysis
print("REGIONAL DEMOGRAPHICS INTEGRATION")
print("=" * 60)
regional_results = run_regional_demographics_analysis()

if regional_results:
    print("\nRegional Demographics Analysis completed successfully!")
else:
    print("\nRegional Demographics Analysis failed!")


REGIONAL DEMOGRAPHICS INTEGRATION
Starting Regional Demographics Analysis...
Loading Master Sales Data for Regional Demographics Analysis...
Master data loaded successfully!
Shape: 6,416,029 rows × 41 columns

Available Columns with Data Types:
    1. Invoice ID (String)
    2. Line (Int64)
    3. Customer ID (Int64)
    4. Product ID (Int64)
    5. Size (String)
    6. Color (String)
    7. Unit Price (Float64)
    8. Quantity (Int64)
    9. Date (Date)
   10. Discount (Float64)
   11. Line Total (Float64)
   12. Store ID (Int64)
   13. Employee ID (Int64)
   14. Currency (String)
   15. Currency Symbol (String)
   16. SKU (String)
   17. Transaction Type (String)
   18. Payment Method (String)
   19. Invoice Total (Float64)
   20. Exchange_Rate_to_USD (Float64)
   21. Unit_Price_USD (Float64)
   22. Line_Total_USD (Float64)
   23. Invoice_Total_USD (Float64)
   24. Category (String)
   25. Sub Category (String)
   26. Description EN (String)
   27. Color_right (String)
   28. Sizes (

/tmp/ipython-input-4111830008.py:142: DeprecationWarning:

`pl.count()` is deprecated. Please use `pl.len()` instead.




Age Group Distribution:
   18-24: 2,302,156 customers
   25-34: 1,669,823 customers
   35-44: 1,411,650 customers
   45-54: 669,500 customers
   55-64: 294,461 customers
   65+: 68,439 customers
Analyzing Regional Demographics...
   Analyzing city-level demographics...


/tmp/ipython-input-4111830008.py:163: DeprecationWarning:

`pl.count()` is deprecated. Please use `pl.len()` instead.



   Analyzing country-level demographics...


...

Regional analysis completed!
   35 cities analyzed
   7 countries analyzed

Top 5 Cities by Revenue:
   New York, United States: $29.97 USD (102,758 customers)
   Los Angeles, United States: $29.97 USD (94,317 customers)
   上海, China: $30.54 USD (84,955 customers)
   广州, China: $30.39 USD (92,209 customers)
   深圳, China: $30.53 USD (90,888 customers)
Saving Analysis Data...
   Saving processed data sample...
   Saving city statistics...
   Saving country statistics...
   Creating summary statistics...
All analysis data saved successfully!
Creating Regional Demographics Visualizations...
   Creating top cities by revenue chart...
   Creating country distribution chart...
   Creating customer performance scatter plot...
   Creating age distribution chart...


/tmp/ipython-input-4111830008.py:326: DeprecationWarning:

`pl.count()` is deprecated. Please use `pl.len()` instead.



   Creating gender distribution chart...


/tmp/ipython-input-4111830008.py:364: DeprecationWarning:

`pl.count()` is deprecated. Please use `pl.len()` instead.



   Creating revenue heatmap...
Created 6 visualizations successfully!
Creating Regional Demographics Dashboard...
Dashboard created successfully!

REGIONAL DEMOGRAPHICS ANALYSIS COMPLETED!

FILES CREATED:
   regional_demographics/regional_demographics_dashboard.html
   regional_demographics/city_statistics.csv
   regional_demographics/country_statistics.csv
   regional_demographics/processed_data_sample.csv
   regional_demographics/summary_statistics.txt

ANALYSIS SUMMARY:
   Countries Analyzed: 7
   Cities Analyzed: 35
   Unique Customers: 1,283,707
   Total Transactions: 6,416,029
   Total Revenue: $288,786,449.85 USD

Open 'regional_demographics/regional_demographics_dashboard.html' to view the complete analysis!

Regional Demographics Analysis completed successfully!


In [ ]:
import polars as pl
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
import numpy as np
from datetime import datetime, timedelta
import os

def load_master_data_for_demographics():
    """Load master data for demographics analysis"""
    print("Loading master data for demographics analysis...")

    try:
        if os.path.exists("data/master_transactions_with_coords.parquet"):
            master_data = pl.read_parquet("data/master_transactions_with_coords.parquet")
        else:
            master_data = pl.read_parquet("data/master_transactions.parquet")

        print(f"Master data loaded: {master_data.shape}")
        print(f"Available columns: {master_data.columns}")

        # Check for demographic columns
        demographic_cols = ['Date Of Birth', 'Gender', 'Age', 'Age Group', 'Occupation', 'Income Level']
        available_demo_cols = [col for col in demographic_cols if col in master_data.columns]
        print(f"Available demographic columns: {available_demo_cols}")

        return master_data

    except Exception as e:
        print(f"Error loading master data: {e}")
        return None

def prepare_demographics_analysis_data(master_data):
    """Prepare comprehensive demographics analysis data with age calculation"""
    print("Preparing demographics analysis data...")

    # Filter for sales only
    sales_data = master_data.filter(pl.col("Transaction Type") == "Sale")
    print(f"Sales data shape: {sales_data.shape}")

    # Calculate age from Date Of Birth
    current_date = datetime.now().date()
    print(f"Calculating age from Date Of Birth (current date: {current_date})")

    # Add age calculations and demographics
    sales_data = sales_data.with_columns([
        # Calculate age from Date Of Birth
        ((pl.lit(current_date) - pl.col("Date Of Birth")).dt.total_days() / 365.25).floor().cast(pl.Int32).alias("Age"),

        # Add time dimensions
        pl.col("Date").dt.strftime("%Y-%m").alias("Year_Month"),
        pl.col("Date").dt.quarter().alias("Quarter"),
        pl.col("Date").dt.weekday().alias("Day_of_Week"),
        pl.col("Date").dt.strftime("%A").alias("Day_Name"),

        # Calculate discount metrics
        (pl.col("Unit_Price_USD") * pl.col("Quantity") - pl.col("Line_Total_USD")).alias("Discount_Amount_USD"),
        ((pl.col("Unit_Price_USD") * pl.col("Quantity") - pl.col("Line_Total_USD")) /
         (pl.col("Unit_Price_USD") * pl.col("Quantity")) * 100).alias("Discount_Percent"),

        # Customer identifier for aggregation
        pl.col("Customer ID").alias("Customer_ID")
    ])

    # Create age groups
    sales_data = sales_data.with_columns([
        pl.when(pl.col("Age") < 18)
        .then(pl.lit("Under 18"))
        .when(pl.col("Age") < 25)
        .then(pl.lit("18-24"))
        .when(pl.col("Age") < 35)
        .then(pl.lit("25-34"))
        .when(pl.col("Age") < 45)
        .then(pl.lit("35-44"))
        .when(pl.col("Age") < 55)
        .then(pl.lit("45-54"))
        .when(pl.col("Age") < 65)
        .then(pl.lit("55-64"))
        .otherwise(pl.lit("65+"))
        .alias("Age_Group")
    ])

    # Check age calculation results
    age_stats = sales_data.select([
        pl.col("Age").min().alias("Min_Age"),
        pl.col("Age").max().alias("Max_Age"),
        pl.col("Age").mean().alias("Avg_Age"),
        # pl.col("Age_Group").value_counts().alias("Age_Group_Counts")
    ])

    print("Age calculation results:")
    print(f"Age range: {age_stats.select('Min_Age').item()} - {age_stats.select('Max_Age').item()}")
    print(f"Average age: {age_stats.select('Avg_Age').item():.1f}")

    # Show age group distribution
    age_group_dist = sales_data.group_by("Age_Group").agg(pl.count().alias("count")).sort("count", descending=True)
    print("Age group distribution:")
    for row in age_group_dist.iter_rows():
        print(f"   {row[0]}: {row[1]:,} transactions")

    print("Creating age-based demographics analysis...")

    # 1. Age Demographics Analysis
    age_demographics = sales_data.group_by(["Age", "Age_Group"]).agg([
        pl.col("Customer_ID").n_unique().alias("Unique_Customers"),
        pl.col("Line_Total_USD").sum().alias("Total_Revenue_USD"),
        pl.col("Quantity").sum().alias("Total_Quantity"),
        pl.col("Line_Total_USD").count().alias("Total_Transactions"),
        pl.col("Unit_Price_USD").mean().alias("Avg_Unit_Price_USD"),
        (pl.col("Discount_Amount_USD").sum() / (pl.col("Unit_Price_USD") * pl.col("Quantity")).sum() * 100).alias("Avg_Discount_Percent"),
        pl.col("Category").n_unique().alias("Categories_Purchased"),
        pl.col("Store ID").n_unique().alias("Stores_Visited")
    ]).with_columns([
        (pl.col("Total_Revenue_USD") / pl.col("Unique_Customers")).alias("Revenue_Per_Customer"),
        (pl.col("Total_Transactions") / pl.col("Unique_Customers")).alias("Transactions_Per_Customer"),
        (pl.col("Total_Revenue_USD") / pl.col("Total_Transactions")).alias("Average_Order_Value")
    ]).sort("Age")

    # 2. Age Group Summary (for easier visualization)
    age_group_demographics = sales_data.group_by("Age_Group").agg([
        pl.col("Customer_ID").n_unique().alias("Unique_Customers"),
        pl.col("Line_Total_USD").sum().alias("Total_Revenue_USD"),
        pl.col("Quantity").sum().alias("Total_Quantity"),
        pl.col("Line_Total_USD").count().alias("Total_Transactions"),
        pl.col("Unit_Price_USD").mean().alias("Avg_Unit_Price_USD"),
        (pl.col("Discount_Amount_USD").sum() / (pl.col("Unit_Price_USD") * pl.col("Quantity")).sum() * 100).alias("Avg_Discount_Percent"),
        pl.col("Category").n_unique().alias("Categories_Purchased"),
        pl.col("Store ID").n_unique().alias("Stores_Visited"),
        pl.col("Age").mean().alias("Average_Age")
    ]).with_columns([
        (pl.col("Total_Revenue_USD") / pl.col("Unique_Customers")).alias("Revenue_Per_Customer"),
        (pl.col("Total_Transactions") / pl.col("Unique_Customers")).alias("Transactions_Per_Customer"),
        (pl.col("Total_Revenue_USD") / pl.col("Total_Transactions")).alias("Average_Order_Value")
    ])

    # Sort age groups in logical order
    age_group_order = ["Under 18", "18-24", "25-34", "35-44", "45-54", "55-64", "65+"]
    age_group_demographics = age_group_demographics.with_columns([
        pl.col("Age_Group").map_elements(
            lambda x: age_group_order.index(x) if x in age_group_order else 999,
            return_dtype=pl.Int32
        ).alias("Age_Group_Order")
    ]).sort("Age_Group_Order").drop("Age_Group_Order")

    # 3. Gender Demographics Analysis
    if "Gender" in sales_data.columns:
        gender_demographics = sales_data.group_by("Gender").agg([
            pl.col("Customer_ID").n_unique().alias("Unique_Customers"),
            pl.col("Line_Total_USD").sum().alias("Total_Revenue_USD"),
            pl.col("Quantity").sum().alias("Total_Quantity"),
            pl.col("Line_Total_USD").count().alias("Total_Transactions"),
            pl.col("Unit_Price_USD").mean().alias("Avg_Unit_Price_USD"),
            (pl.col("Discount_Amount_USD").sum() / (pl.col("Unit_Price_USD") * pl.col("Quantity")).sum() * 100).alias("Avg_Discount_Percent"),
            pl.col("Category").n_unique().alias("Categories_Purchased"),
            pl.col("Store ID").n_unique().alias("Stores_Visited"),
            pl.col("Age").mean().alias("Average_Age")
        ]).with_columns([
            (pl.col("Total_Revenue_USD") / pl.col("Unique_Customers")).alias("Revenue_Per_Customer"),
            (pl.col("Total_Transactions") / pl.col("Unique_Customers")).alias("Transactions_Per_Customer"),
            (pl.col("Total_Revenue_USD") / pl.col("Total_Transactions")).alias("Average_Order_Value")
        ])
    else:
        print("Gender column not found - creating placeholder")
        gender_demographics = None

    print("Creating category preferences by demographics...")

    # 4. Age Group vs Category Analysis
    age_category_analysis = sales_data.group_by(["Age_Group", "Category"]).agg([
        pl.col("Line_Total_USD").sum().alias("Revenue_USD"),
        pl.col("Quantity").sum().alias("Quantity_Sold"),
        pl.col("Customer_ID").n_unique().alias("Unique_Customers"),
        pl.col("Line_Total_USD").count().alias("Transactions")
    ]).with_columns([
        (pl.col("Revenue_USD") / pl.col("Unique_Customers")).alias("Revenue_Per_Customer"),
        (pl.col("Revenue_USD") / pl.col("Transactions")).alias("Average_Order_Value")
    ]).sort(["Age_Group", "Revenue_USD"], descending=[False, True])

    # 5. Gender vs Category Analysis (if gender available)
    if "Gender" in sales_data.columns:
        gender_category_analysis = sales_data.group_by(["Gender", "Category"]).agg([
            pl.col("Line_Total_USD").sum().alias("Revenue_USD"),
            pl.col("Quantity").sum().alias("Quantity_Sold"),
            pl.col("Customer_ID").n_unique().alias("Unique_Customers"),
            pl.col("Line_Total_USD").count().alias("Transactions")
        ]).with_columns([
            (pl.col("Revenue_USD") / pl.col("Unique_Customers")).alias("Revenue_Per_Customer"),
            (pl.col("Revenue_USD") / pl.col("Transactions")).alias("Average_Order_Value")
        ]).sort(["Gender", "Revenue_USD"], descending=[False, True])
    else:
        gender_category_analysis = None

    print("Creating customer lifetime value by demographics...")

    # 6. Customer-Level Demographics with CLV
    customer_demographics_clv = sales_data.group_by([
        "Customer_ID", "Age", "Age_Group", "Gender" if "Gender" in sales_data.columns else pl.lit("Unknown").alias("Gender")
    ]).agg([
        pl.col("Line_Total_USD").sum().alias("Total_Spent_USD"),
        pl.col("Quantity").sum().alias("Total_Items"),
        pl.col("Line_Total_USD").count().alias("Total_Transactions"),
        pl.col("Date").min().alias("First_Purchase"),
        pl.col("Date").max().alias("Last_Purchase"),
        pl.col("Category").n_unique().alias("Categories_Explored"),
        pl.col("Store ID").n_unique().alias("Stores_Visited"),
        pl.col("Unit_Price_USD").mean().alias("Avg_Unit_Price"),
        (pl.col("Discount_Amount_USD").sum() / (pl.col("Unit_Price_USD") * pl.col("Quantity")).sum() * 100).alias("Avg_Discount_Percent")
    ]).with_columns([
        (pl.col("Total_Spent_USD") / pl.col("Total_Transactions")).alias("Average_Order_Value"),
        (pl.col("Last_Purchase") - pl.col("First_Purchase")).dt.total_days().alias("Customer_Lifespan_Days")
    ])

    # Convert to pandas for advanced CLV calculations
    customer_clv_df = customer_demographics_clv.to_pandas()
    customer_clv_df['Customer_Lifespan_Days'] = customer_clv_df['Customer_Lifespan_Days'].fillna(1).replace(0, 1)
    customer_clv_df['Monthly_Frequency'] = (customer_clv_df['Total_Transactions'] / customer_clv_df['Customer_Lifespan_Days'] * 30).fillna(1.0)

    # Simple CLV calculation
    customer_clv_df['Predicted_CLV'] = customer_clv_df['Average_Order_Value'] * customer_clv_df['Monthly_Frequency'] * 12
    customer_clv_df['Total_CLV'] = customer_clv_df['Total_Spent_USD'] + customer_clv_df['Predicted_CLV']

    print("Creating temporal demographics patterns...")

    # 7. Monthly Demographics Trends
    monthly_age_trends = sales_data.group_by(["Year_Month", "Age_Group"]).agg([
        pl.col("Line_Total_USD").sum().alias("Monthly_Revenue_USD"),
        pl.col("Customer_ID").n_unique().alias("Active_Customers"),
        pl.col("Line_Total_USD").count().alias("Transactions")
    ]).sort(["Year_Month", "Age_Group"])

    # 8. Day of Week Demographics Patterns
    dow_demographics = sales_data.group_by(["Day_Name", "Age_Group"]).agg([
        pl.col("Line_Total_USD").sum().alias("Revenue_USD"),
        pl.col("Customer_ID").n_unique().alias("Unique_Customers"),
        pl.col("Line_Total_USD").count().alias("Transactions")
    ])

    print("Creating discount sensitivity by demographics...")

    # 9. Discount Sensitivity by Demographics
    discount_sensitivity = sales_data.with_columns([
        pl.when(pl.col("Discount_Percent") <= 5)
        .then(pl.lit("0-5%"))
        .when(pl.col("Discount_Percent") <= 15)
        .then(pl.lit("6-15%"))
        .when(pl.col("Discount_Percent") <= 25)
        .then(pl.lit("16-25%"))
        .when(pl.col("Discount_Percent") <= 35)
        .then(pl.lit("26-35%"))
        .otherwise(pl.lit("35%+"))
        .alias("Discount_Bucket")
    ]).group_by(["Age_Group", "Discount_Bucket"]).agg([
        pl.col("Line_Total_USD").sum().alias("Revenue_USD"),
        pl.col("Customer_ID").n_unique().alias("Customers"),
        pl.col("Line_Total_USD").count().alias("Transactions")
    ])

    print("Demographics analysis data prepared successfully!")

    return {
        "sales_data": sales_data,
        "age_demographics": age_demographics,
        "age_group_demographics": age_group_demographics,
        "gender_demographics": gender_demographics,
        "age_category_analysis": age_category_analysis,
        "gender_category_analysis": gender_category_analysis,
        "customer_demographics_clv": pl.from_pandas(customer_clv_df),
        "monthly_age_trends": monthly_age_trends,
        "dow_demographics": dow_demographics,
        "discount_sensitivity": discount_sensitivity
    }

def create_demographics_dashboard(data_dict):
    """Create comprehensive demographics analysis dashboard"""
    print("Creating Demographics Analysis Dashboard...")

    # Convert to pandas for plotting
    age_demo_df = data_dict["age_group_demographics"].to_pandas()
    age_category_df = data_dict["age_category_analysis"].to_pandas()
    customer_clv_df = data_dict["customer_demographics_clv"].to_pandas()
    monthly_trends_df = data_dict["monthly_age_trends"].to_pandas()
    dow_demo_df = data_dict["dow_demographics"].to_pandas()
    discount_sens_df = data_dict["discount_sensitivity"].to_pandas()

    # Handle gender data
    if data_dict["gender_demographics"] is not None:
        gender_demo_df = data_dict["gender_demographics"].to_pandas()
        gender_category_df = data_dict["gender_category_analysis"].to_pandas()
    else:
        gender_demo_df = None
        gender_category_df = None

    # Create HTML structure
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Customer Demographics Analysis Dashboard</title>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        <style>
            body { font-family: Arial, sans-serif; margin: 20px; background-color: #f5f5f5; }
            .chart-container { background-color: white; margin: 20px 0; padding: 20px; border-radius: 10px; box-shadow: 0 2px 5px rgba(0,0,0,0.1); }
            .chart-title { font-size: 24px; font-weight: bold; text-align: center; margin-bottom: 20px; color: #333; }
            .dashboard-title { font-size: 36px; font-weight: bold; text-align: center; margin-bottom: 30px; color: #2c3e50; }
            .insights-box { background-color: #e8f4f8; padding: 15px; margin: 10px 0; border-radius: 8px; font-size: 14px; }
            .metric-highlight { background-color: #fff3cd; padding: 10px; margin: 5px 0; border-radius: 5px; font-weight: bold; }
            .demo-grid { display: grid; grid-template-columns: repeat(auto-fit, minmax(250px, 1fr)); gap: 15px; margin: 20px 0; }
            .demo-card { background-color: #f8f9fa; padding: 15px; border-radius: 8px; border-left: 4px solid #007bff; }
        </style>
    </head>
    <body>
        <div class="dashboard-title">Customer Demographics Analysis Dashboard</div>

        <div class="insights-box">
            <strong>Demographics Analysis Insights:</strong><br>
            • <strong>Age Distribution</strong>: Customer behavior patterns across different age groups<br>
            • <strong>Gender Preferences</strong>: Product and category preferences by gender<br>
            • <strong>Demographic CLV</strong>: Customer lifetime value analysis by demographics<br>
            • <strong>Purchase Patterns</strong>: Temporal and behavioral patterns by demographic segments<br>
            • <strong>Discount Sensitivity</strong>: How different demographics respond to pricing strategies
        </div>
    """

    # Create figures (same as before but without emojis in titles)
    # 1. Age Distribution Analysis
    fig1 = make_subplots(specs=[[{"secondary_y": True}]])

    fig1.add_trace(
        go.Bar(
            x=age_demo_df['Age_Group'],
            y=age_demo_df['Unique_Customers'],
            name='Number of Customers',
            marker_color='lightblue',
            yaxis='y'
        ),
        secondary_y=False,
    )

    fig1.add_trace(
        go.Scatter(
            x=age_demo_df['Age_Group'],
            y=age_demo_df['Revenue_Per_Customer'],
            mode='lines+markers',
            name='Revenue Per Customer (USD)',
            line=dict(color='red', width=3),
            marker=dict(size=8),
            yaxis='y2'
        ),
        secondary_y=True,
    )

    fig1.update_xaxes(title_text="Age Group")
    fig1.update_yaxes(title_text="Number of Customers", secondary_y=False)
    fig1.update_yaxes(title_text="Revenue Per Customer (USD)", secondary_y=True)
    fig1.update_layout(
        title_text="Customer Distribution and Revenue by Age Group",
        height=600,
        template="plotly_white"
    )

    # 2. Age vs Category Heatmap
    age_cat_pivot = age_category_df.pivot_table(
        values='Revenue_USD',
        index='Age_Group',
        columns='Category',
        fill_value=0
    )

    fig2 = px.imshow(
        age_cat_pivot.values,
        labels=dict(x="Category", y="Age Group", color="Revenue (USD)"),
        x=age_cat_pivot.columns,
        y=age_cat_pivot.index,
        title="Category Preferences by Age Group (Revenue Heatmap)",
        color_continuous_scale="Viridis",
        height=600
    )
    fig2.update_layout(template="plotly_white")

    # 3. Customer Lifetime Value by Age
    fig3 = px.box(
        customer_clv_df,
        x="Age_Group",
        y="Total_CLV",
        title="Customer Lifetime Value Distribution by Age Group",
        height=600
    )
    fig3.update_layout(template="plotly_white")

    # 4. Gender Analysis (if available)
    if gender_demo_df is not None:
        fig4 = px.pie(
            gender_demo_df,
            values='Unique_Customers',
            names='Gender',
            title="Customer Distribution by Gender",
            height=500
        )
        fig4.update_traces(textposition='inside', textinfo='percent+label')
        fig4.update_layout(template="plotly_white")
    else:
        fig4 = go.Figure()
        fig4.add_annotation(
            text="Gender data not available in dataset",
            xref="paper", yref="paper",
            x=0.5, y=0.5, xanchor='center', yanchor='middle',
            showarrow=False, font=dict(size=20)
        )
        fig4.update_layout(
            title="Gender Analysis - Data Not Available",
            height=500,
            template="plotly_white"
        )

    # 5. Monthly Trends by Age Group
    fig5 = px.line(
        monthly_trends_df,
        x="Year_Month",
        y="Monthly_Revenue_USD",
        color="Age_Group",
        title="Monthly Revenue Trends by Age Group",
        markers=True,
        height=600
    )
    fig5.update_xaxes(tickangle=45)
    fig5.update_layout(template="plotly_white")

    # 6. Day of Week Patterns by Age
    fig6 = px.bar(
        dow_demo_df,
        x="Day_Name",
        y="Revenue_USD",
        color="Age_Group",
        title="Daily Purchase Patterns by Age Group",
        height=600
    )
    fig6.update_layout(template="plotly_white")

    # 7. Discount Sensitivity by Age Group
    fig7 = px.bar(
        discount_sens_df,
        x="Age_Group",
        y="Revenue_USD",
        color="Discount_Bucket",
        title="Discount Sensitivity by Age Group",
        height=600
    )
    fig7.update_layout(template="plotly_white")

    # 8. Age vs Purchase Behavior Scatter
    fig8 = px.scatter(
        customer_clv_df,
        x="Average_Order_Value",
        y="Total_Transactions",
        color="Age_Group",
        size="Total_CLV",
        hover_data={
            "Customer_ID": True,
            "Total_Spent_USD": ":,.2f",
            "Categories_Explored": True,
            "Stores_Visited": True
        },
        title="Purchase Behavior: Order Value vs Frequency by Age",
        height=700
    )
    fig8.update_layout(template="plotly_white")

    # 9. Gender vs Category Analysis (if available)
    if gender_category_df is not None:
        fig9 = px.bar(
            gender_category_df,
            x="Category",
            y="Revenue_USD",
            color="Gender",
            title="Category Preferences by Gender",
            height=600
        )
        fig9.update_xaxes(tickangle=45)
        fig9.update_layout(template="plotly_white")
    else:
        fig9 = go.Figure()
        fig9.add_annotation(
            text="Gender category analysis not available",
            xref="paper", yref="paper",
            x=0.5, y=0.5, xanchor='center', yanchor='middle',
            showarrow=False, font=dict(size=20)
        )
        fig9.update_layout(
            title="Gender vs Category Analysis - Data Not Available",
            height=500,
            template="plotly_white"
        )

    # 10. Demographics Summary Metrics
    fig10 = go.Figure()

    fig10.add_trace(go.Bar(
        x=age_demo_df['Age_Group'],
        y=age_demo_df['Total_Revenue_USD'],
        name='Total Revenue',
        marker_color='skyblue'
    ))

    fig10.update_layout(
        title="Total Revenue by Age Group",
        xaxis_title="Age Group",
        yaxis_title="Total Revenue (USD)",
        height=500,
        template="plotly_white"
    )

    # Convert all figures to HTML
    figures = [fig1, fig2, fig3, fig4, fig5, fig6, fig7, fig8, fig9, fig10]
    titles = [
        "Customer Distribution and Revenue by Age Group",
        "Category Preferences by Age Group (Revenue Heatmap)",
        "Customer Lifetime Value Distribution by Age Group",
        "Customer Distribution by Gender",
        "Monthly Revenue Trends by Age Group",
        "Daily Purchase Patterns by Age Group",
        "Discount Sensitivity by Age Group",
        "Purchase Behavior: Order Value vs Frequency by Age",
        "Category Preferences by Gender",
        "Total Revenue by Age Group"
    ]

    for i, (fig, title) in enumerate(zip(figures, titles)):
        html_content += f"""
        <div class="chart-container">
            <div class="chart-title">{title}</div>
            <div id="chart{i+1}"></div>
        </div>
        """

    # Add demographic insights summary
    total_customers = age_demo_df['Unique_Customers'].sum()
    total_revenue = age_demo_df['Total_Revenue_USD'].sum()
    highest_clv_age = customer_clv_df.groupby('Age_Group')['Total_CLV'].mean().idxmax()
    most_active_age = age_demo_df.loc[age_demo_df['Transactions_Per_Customer'].idxmax(), 'Age_Group']

    html_content += f"""
        <div class="insights-box">
            <strong>Key Demographics Insights:</strong><br>
            <div class="metric-highlight">Total Customers Analyzed: {total_customers:,}</div>
            <div class="metric-highlight">Total Revenue: ${total_revenue:,.0f}</div>
            <div class="metric-highlight">Highest CLV Age Group: {highest_clv_age}</div>
            <div class="metric-highlight">Most Active Age Group: {most_active_age}</div>
        </div>

        <div class="chart-container">
            <div class="chart-title">Age Group Performance Summary</div>
            <div class="demo-grid">
    """

    # Add age group summary cards
    for _, row in age_demo_df.iterrows():
        html_content += f"""
            <div class="demo-card">
                <h3>{row['Age_Group']}</h3>
                <p><strong>Customers:</strong> {row['Unique_Customers']:,}</p>
                <p><strong>Revenue:</strong> ${row['Total_Revenue_USD']:,.0f}</p>
                <p><strong>Avg Order Value:</strong> ${row['Average_Order_Value']:.2f}</p>
                <p><strong>Revenue/Customer:</strong> ${row['Revenue_Per_Customer']:,.2f}</p>
                <p><strong>Avg Discount:</strong> {row['Avg_Discount_Percent']:.1f}%</p>
                <p><strong>Avg Age:</strong> {row['Average_Age']:.1f} years</p>
            </div>
        """

    html_content += """
            </div>
        </div>
    """

    html_content += """
    <script>
    """

    # Add JavaScript for each plot
    for i, fig in enumerate(figures):
        plot_json = fig.to_json()
        html_content += f"""
        var plotData{i+1} = {plot_json};
        Plotly.newPlot('chart{i+1}', plotData{i+1}.data, plotData{i+1}.layout, {{responsive: true}});
        """

    html_content += """
    </script>
    </body>
    </html>
    """

    return html_content


def save_demographics_analysis_data(data_dict):
    """Save demographics analysis data files"""
    print("Saving demographics analysis data files...")

    os.makedirs("demographics_analysis/data", exist_ok=True)

    # Save all datasets
    data_dict["age_demographics"].write_csv("demographics_analysis/data/age_demographics_detailed.csv")
    data_dict["age_group_demographics"].write_csv("demographics_analysis/data/age_group_demographics_summary.csv")
    data_dict["age_category_analysis"].write_csv("demographics_analysis/data/age_category_preferences.csv")
    data_dict["customer_demographics_clv"].write_csv("demographics_analysis/data/customer_demographics_clv.csv")
    data_dict["monthly_age_trends"].write_csv("demographics_analysis/data/monthly_age_trends.csv")
    data_dict["dow_demographics"].write_csv("demographics_analysis/data/day_of_week_demographics.csv")
    data_dict["discount_sensitivity"].write_csv("demographics_analysis/data/discount_sensitivity_by_age.csv")

    # Save gender data if available
    if data_dict["gender_demographics"] is not None:
        data_dict["gender_demographics"].write_csv("demographics_analysis/data/gender_demographics_analysis.csv")
        data_dict["gender_category_analysis"].write_csv("demographics_analysis/data/gender_category_preferences.csv")

    print("Demographics analysis data files saved!")

def run_demographics_analysis():
    """Run comprehensive demographics analysis"""
    print("Starting Customer Demographics Analysis...")

    # Load data
    master_data = load_master_data_for_demographics()
    if master_data is None:
        print("Failed to load master data!")
        return None

    # Prepare analysis data
    data_dict = prepare_demographics_analysis_data(master_data)
    if data_dict is None:
        print("Failed to prepare demographics data!")
        return None

    # Save data files
    save_demographics_analysis_data(data_dict)

    # Create dashboard
    html_content = create_demographics_dashboard(data_dict)

    # Create directory and save HTML
    os.makedirs("demographics_analysis", exist_ok=True)

    with open("demographics_analysis/demographics_analysis_dashboard.html", "w", encoding='utf-8') as f:
        f.write(html_content)

    # Print summary insights
    age_demo_df = data_dict["age_group_demographics"].to_pandas()
    customer_clv_df = data_dict["customer_demographics_clv"].to_pandas()

    print("\nCUSTOMER DEMOGRAPHICS ANALYSIS COMPLETE!")
    print(f"Total Customers Analyzed: {age_demo_df['Unique_Customers'].sum():,}")
    print(f"Total Revenue Analyzed: ${age_demo_df['Total_Revenue_USD'].sum():,.0f}")
    print(f"Age Groups Analyzed: {len(age_demo_df)}")

    # Age group insights
    top_age_group = age_demo_df.loc[age_demo_df['Total_Revenue_USD'].idxmax()]
    print(f"Top Revenue Age Group: {top_age_group['Age_Group']} (${top_age_group['Total_Revenue_USD']:,.0f})")

    highest_aov_age = age_demo_df.loc[age_demo_df['Average_Order_Value'].idxmax()]
    print(f"Highest AOV Age Group: {highest_aov_age['Age_Group']} (${highest_aov_age['Average_Order_Value']:.2f})")

    # CLV insights
    avg_clv_by_age = customer_clv_df.groupby('Age_Group')['Total_CLV'].mean()
    highest_clv_age = avg_clv_by_age.idxmax()
    print(f"Highest CLV Age Group: {highest_clv_age} (${avg_clv_by_age[highest_clv_age]:,.2f})")

    print("\nDashboard and Data Files Created:")
    print("   demographics_analysis/demographics_analysis_dashboard.html")
    print("   demographics_analysis/data/ (CSV files for detailed analysis)")

    return data_dict

def create_advanced_demographic_insights(data_dict):
    """Create advanced demographic insights and recommendations"""
    print("Generating advanced demographic insights...")

    # Convert key datasets to pandas for analysis
    age_demo_df = data_dict["age_group_demographics"].to_pandas()
    customer_clv_df = data_dict["customer_demographics_clv"].to_pandas()
    age_category_df = data_dict["age_category_analysis"].to_pandas()

    insights = {}

    # 1. Age Group Performance Analysis
    insights['age_performance'] = {}
    for _, row in age_demo_df.iterrows():
        age_group = row['Age_Group']
        insights['age_performance'][age_group] = {
            'customers': int(row['Unique_Customers']),
            'revenue': float(row['Total_Revenue_USD']),
            'revenue_per_customer': float(row['Revenue_Per_Customer']),
            'avg_order_value': float(row['Average_Order_Value']),
            'transactions_per_customer': float(row['Transactions_Per_Customer']),
            'discount_sensitivity': float(row['Avg_Discount_Percent']),
            'market_share': float(row['Unique_Customers'] / age_demo_df['Unique_Customers'].sum() * 100),
            'average_age': float(row['Average_Age'])
        }

    # 2. Category Preferences by Age
    insights['category_preferences'] = {}
    for age_group in age_category_df['Age_Group'].unique():
        age_data = age_category_df[age_category_df['Age_Group'] == age_group]
        top_categories = age_data.nlargest(3, 'Revenue_USD')

        insights['category_preferences'][age_group] = {
            'top_categories': top_categories[['Category', 'Revenue_USD']].to_dict('records'),
            'category_diversity': len(age_data),
            'total_revenue': float(age_data['Revenue_USD'].sum())
        }

    # 3. CLV Analysis by Demographics
    clv_by_age = customer_clv_df.groupby('Age_Group').agg({
        'Total_CLV': ['mean', 'median', 'std', 'count'],
        'Total_Spent_USD': 'mean',
        'Predicted_CLV': 'mean',
        'Average_Order_Value': 'mean',
        'Total_Transactions': 'mean'
    }).round(2)

    insights['clv_analysis'] = clv_by_age.to_dict()

    # 4. Strategic Recommendations
    insights['recommendations'] = {}

    # Find highest value age groups
    top_revenue_age = age_demo_df.loc[age_demo_df['Total_Revenue_USD'].idxmax(), 'Age_Group']
    top_clv_age = customer_clv_df.groupby('Age_Group')['Total_CLV'].mean().idxmax()
    highest_aov_age = age_demo_df.loc[age_demo_df['Average_Order_Value'].idxmax(), 'Age_Group']
    most_frequent_age = age_demo_df.loc[age_demo_df['Transactions_Per_Customer'].idxmax(), 'Age_Group']

    insights['recommendations'] = {
        'focus_segments': {
            'primary_revenue_driver': top_revenue_age,
            'highest_lifetime_value': top_clv_age,
            'premium_customers': highest_aov_age,
            'most_engaged': most_frequent_age
        },
        'strategies': {
            top_revenue_age: "Primary revenue driver - focus on retention and expansion",
            top_clv_age: "Highest CLV - invest in premium experiences and loyalty programs",
            highest_aov_age: "Premium segment - target with high-value products",
            most_frequent_age: "Most engaged - leverage for referrals and advocacy"
        }
    }

    # 5. Market Opportunities
    insights['opportunities'] = {
        'underperforming_segments': [],
        'growth_potential': [],
        'category_expansion': {}
    }

    # Identify underperforming segments (low revenue per customer)
    median_rpc = age_demo_df['Revenue_Per_Customer'].median()
    underperforming = age_demo_df[age_demo_df['Revenue_Per_Customer'] < median_rpc * 0.8]

    for _, row in underperforming.iterrows():
        insights['opportunities']['underperforming_segments'].append({
            'age_group': row['Age_Group'],
            'customers': int(row['Unique_Customers']),
            'current_rpc': float(row['Revenue_Per_Customer']),
            'potential_uplift': float(median_rpc - row['Revenue_Per_Customer'])
        })

    return insights

def generate_demographic_report(insights):
    """Generate a comprehensive demographic report"""
    print("Generating demographic insights report...")

    report = f"""

    Customer Demographics Analysis Report
    ====================================

    EXECUTIVE SUMMARY
    =================

    KEY FINDINGS:
    • Primary Revenue Driver: {insights['recommendations']['focus_segments']['primary_revenue_driver']}
    • Highest CLV Segment: {insights['recommendations']['focus_segments']['highest_lifetime_value']}
    • Premium Customer Group: {insights['recommendations']['focus_segments']['premium_customers']}
    • Most Engaged Segment: {insights['recommendations']['focus_segments']['most_engaged']}

    AGE GROUP PERFORMANCE ANALYSIS
    ==============================
    """

    # Add age group details
    for age_group, data in insights['age_performance'].items():
        report += f"""
    {age_group.upper()}:
       • Customers: {data['customers']:,} ({data['market_share']:.1f}% of total)
       • Average Age: {data['average_age']:.1f} years
       • Revenue: ${data['revenue']:,.0f}
       • Revenue per Customer: ${data['revenue_per_customer']:,.2f}
       • Average Order Value: ${data['avg_order_value']:.2f}
       • Purchase Frequency: {data['transactions_per_customer']:.1f} transactions/customer
       • Discount Sensitivity: {data['discount_sensitivity']:.1f}%
    """

    report += f"""

    CATEGORY PREFERENCES BY AGE GROUP
    =================================
    """

    # Add category preferences
    for age_group, prefs in insights['category_preferences'].items():
        report += f"""
    {age_group.upper()}:
       • Category Diversity: {prefs['category_diversity']} categories explored
       • Total Category Revenue: ${prefs['total_revenue']:,.0f}
       • Top 3 Categories:"""

        for i, cat in enumerate(prefs['top_categories'][:3], 1):
            report += f"""
         {i}. {cat['Category']}: ${cat['Revenue_USD']:,.0f}"""

    report += f"""

    STRATEGIC RECOMMENDATIONS
    =========================
    """

    for age_group, strategy in insights['recommendations']['strategies'].items():
        report += f"""
    • {age_group}: {strategy}
    """

    # Add opportunities section
    if insights['opportunities']['underperforming_segments']:
        report += f"""

    GROWTH OPPORTUNITIES
    ===================

    UNDERPERFORMING SEGMENTS (Revenue Enhancement Opportunities):
    """

        for segment in insights['opportunities']['underperforming_segments']:
            potential_revenue = segment['customers'] * segment['potential_uplift']
            report += f"""
    • {segment['age_group']}: {segment['customers']:,} customers
      Current RPC: ${segment['current_rpc']:.2f}
      Potential Uplift: ${segment['potential_uplift']:.2f}/customer
      Total Opportunity: ${potential_revenue:,.0f}
    """

    report += f"""

    END OF REPORT
    =============
    """

    return report

# Enhanced run function with insights
def run_complete_demographics_analysis():
    """Run complete demographics analysis with insights"""
    print("Starting Complete Customer Demographics Analysis...")

    # Run main analysis
    results = run_demographics_analysis()

    if results:
        print("\nGenerating Advanced Insights...")

        # Generate advanced insights
        insights = create_advanced_demographic_insights(results)

        # Generate comprehensive report
        report = generate_demographic_report(insights)

        # Save insights and report
        os.makedirs("demographics_analysis/reports", exist_ok=True)

        # Save report as text
        with open("demographics_analysis/reports/demographic_analysis_report.txt", "w", encoding='utf-8') as f:
            f.write(report)

        # Print the report
        print(report)

        print("\nADDITIONAL FILES CREATED:")
        print("   demographics_analysis/reports/demographic_analysis_report.txt")

        return results, insights

    else:
        return None, None

# Run the complete analysis
print("Executing Complete Demographics Analysis Pipeline...")
results, insights = run_complete_demographics_analysis()

if results and insights:
    print("\nDEMOGRAPHICS ANALYSIS PIPELINE COMPLETED SUCCESSFULLY!")
    print("\nANALYSIS OUTPUTS:")
    print("   Interactive Dashboard: demographics_analysis/demographics_analysis_dashboard.html")
    print("   Raw Data Files: demographics_analysis/data/")
    print("   Insights Report: demographics_analysis/reports/")
    print("   Strategic Recommendations: Available in report")
else:
    print("\nDemographics Analysis Pipeline failed!")
    print("Please check your data files and try again.")



Executing Complete Demographics Analysis Pipeline...
Starting Complete Customer Demographics Analysis...
Starting Customer Demographics Analysis...
Loading master data for demographics analysis...
Master data loaded: (6416029, 43)
Available columns: ['Invoice ID', 'Line', 'Customer ID', 'Product ID', 'Size', 'Color', 'Unit Price', 'Quantity', 'Date', 'Discount', 'Line Total', 'Store ID', 'Employee ID', 'Currency', 'Currency Symbol', 'SKU', 'Transaction Type', 'Payment Method', 'Invoice Total', 'Exchange_Rate_to_USD', 'Unit_Price_USD', 'Line_Total_USD', 'Invoice_Total_USD', 'Category', 'Sub Category', 'Description EN', 'Color_right', 'Sizes', 'Production Cost', 'Name', 'Email', 'Gender', 'Date Of Birth', 'Job Title', 'Country', 'City', 'Store Name', 'Number of Employees', 'Store ID_right', 'Name_right', 'Position', 'Latitude', 'Longitude']
Available demographic columns: ['Date Of Birth', 'Gender']
Preparing demographics analysis data...
Sales data shape: (6077200, 43)
Calculating age fr

/tmp/ipython-input-2420069177.py:98: DeprecationWarning:

`pl.count()` is deprecated. Please use `pl.len()` instead.



Age group distribution:
   18-24: 2,276,879 transactions
   25-34: 1,540,131 transactions
   35-44: 1,316,033 transactions
   45-54: 617,724 transactions
   55-64: 266,589 transactions
   65+: 59,844 transactions
Creating age-based demographics analysis...
Creating category preferences by demographics...
Creating customer lifetime value by demographics...
Creating temporal demographics patterns...
Creating discount sensitivity by demographics...
Demographics analysis data prepared successfully!
Saving demographics analysis data files...
Demographics analysis data files saved!
Creating Demographics Analysis Dashboard...

CUSTOMER DEMOGRAPHICS ANALYSIS COMPLETE!
Total Customers Analyzed: 1,283,707
Total Revenue Analyzed: $305,884,837
Age Groups Analyzed: 6
Top Revenue Age Group: 18-24 ($115,657,079)
Highest AOV Age Group: 65+ ($52.99)
Highest CLV Age Group: 25-34 ($7,455.27)

Dashboard and Data Files Created:
   demographics_analysis/demographics_analysis_dashboard.html
   demographics_a

## Customer Segmentation (RFM, CLV, Cohort)

In [ ]:
import polars as pl
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
import numpy as np
from datetime import datetime, timedelta
import os
from operator import attrgetter

def load_master_data_for_customer_segmentation():
    """Load master data for customer segmentation analysis"""
    print(" Loading master data for customer segmentation analysis...")

    try:
        if os.path.exists("data/master_transactions_with_coords.parquet"):
            master_data = pl.read_parquet("data/master_transactions_with_coords.parquet")
        else:
            master_data = pl.read_parquet("data/master_transactions.parquet")

        print(f" Master data loaded: {master_data.shape}")
        return master_data

    except Exception as e:
        print(f" Error loading master data: {e}")
        return None

def calculate_advanced_rfm_analysis(master_data):
    """Calculate advanced RFM analysis with additional customer metrics"""
    print("Calculating Advanced RFM Analysis...")

    # Filter for sales only
    sales_data = master_data.filter(pl.col("Transaction Type") == "Sale")

    # Get the latest date in the dataset as reference point
    latest_date = sales_data.select(pl.col("Date").max()).item()
    print(f"Analysis reference date: {latest_date}")

    # Calculate comprehensive RFM metrics
    # Calculate comprehensive RFM metrics
    rfm_data = sales_data.group_by("Customer ID").agg([
        # Core RFM metrics
        (pl.lit(latest_date) - pl.col("Date").max()).dt.total_days().alias("Recency_Days"),
        pl.col("Invoice ID").n_unique().alias("Frequency"),
        pl.col("Line_Total_USD").sum().alias("Monetary_Value"),

        # Additional customer behavior metrics
        pl.col("Date").min().alias("First_Purchase_Date"),
        pl.col("Date").max().alias("Last_Purchase_Date"),
        pl.col("Quantity").sum().alias("Total_Items_Purchased"),
        pl.col("Product ID").n_unique().alias("Unique_Products_Purchased"),
        pl.col("Category").n_unique().alias("Unique_Categories_Purchased"),
        pl.col("Store ID").n_unique().alias("Stores_Visited"),
        pl.col("Line_Total_USD").mean().alias("Average_Order_Value"),
        (pl.col("Line_Total_USD").sum() / pl.col("Quantity").sum()).alias("Average_Unit_Price"),

        # Seasonal behavior (simplified)
        pl.col("Date").dt.quarter().first().alias("First_Quarter"),
        pl.col("Date").dt.weekday().first().alias("First_Weekday"),
        pl.col("Date").dt.month().first().alias("First_Month"),

        # Product preferences
        pl.col("Category").first().alias("First_Category"),  # Changed from mode() to first()
        pl.col("Sub Category").first().alias("First_Sub_Category")  # Changed from mode() to first()
    ])


    # Convert to pandas for advanced calculations
    rfm_pandas = rfm_data.to_pandas()

    # Calculate customer lifetime and additional metrics
    rfm_pandas['Customer_Lifetime_Days'] = (rfm_pandas['Last_Purchase_Date'] - rfm_pandas['First_Purchase_Date']).dt.days
    rfm_pandas['Customer_Lifetime_Days'] = rfm_pandas['Customer_Lifetime_Days'].fillna(1).replace(0, 1)

    # Advanced behavioral metrics
    rfm_pandas['Monthly_Purchase_Frequency'] = (rfm_pandas['Frequency'] / rfm_pandas['Customer_Lifetime_Days'] * 30).fillna(1.0)
    rfm_pandas['Items_Per_Transaction'] = rfm_pandas['Total_Items_Purchased'] / rfm_pandas['Frequency']
    rfm_pandas['Categories_Per_Transaction'] = rfm_pandas['Unique_Categories_Purchased'] / rfm_pandas['Frequency']
    rfm_pandas['Store_Loyalty_Score'] = 1 / rfm_pandas['Stores_Visited']  # Higher = more loyal to specific stores
    rfm_pandas['Product_Diversity_Score'] = rfm_pandas['Unique_Products_Purchased'] / rfm_pandas['Frequency']

    # Calculate percentile-based RFM scores (1-5 scale)
    rfm_pandas['R_Score'] = pd.qcut(rfm_pandas['Recency_Days'], 5, labels=[5,4,3,2,1], duplicates='drop')  # Lower recency = higher score
    rfm_pandas['F_Score'] = pd.qcut(rfm_pandas['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5], duplicates='drop')
    rfm_pandas['M_Score'] = pd.qcut(rfm_pandas['Monetary_Value'], 5, labels=[1,2,3,4,5], duplicates='drop')

    # Handle any NaN values in scores
    rfm_pandas['R_Score'] = rfm_pandas['R_Score'].fillna(3)
    rfm_pandas['F_Score'] = rfm_pandas['F_Score'].fillna(3)
    rfm_pandas['M_Score'] = rfm_pandas['M_Score'].fillna(3)

    # Create RFM segment string
    rfm_pandas['RFM_Score'] = rfm_pandas['R_Score'].astype(str) + rfm_pandas['F_Score'].astype(str) + rfm_pandas['M_Score'].astype(str)

    print(f" Advanced RFM analysis completed for {len(rfm_pandas):,} customers")

    return rfm_pandas

def create_customer_segments(rfm_data):
    """Create detailed customer segments based on RFM and behavioral data - OPTIMIZED"""
    print(" Creating detailed customer segments...")

    # Convert R_Score, F_Score, M_Score to integers for faster operations
    rfm_data['R_Score'] = rfm_data['R_Score'].astype(int)
    rfm_data['F_Score'] = rfm_data['F_Score'].astype(int)
    rfm_data['M_Score'] = rfm_data['M_Score'].astype(int)

    # Vectorized RFM segmentation using numpy conditions
    conditions = [
        # Champions: High value, frequent, recent
        (rfm_data['R_Score'] >= 4) & (rfm_data['F_Score'] >= 4) & (rfm_data['M_Score'] >= 4),

        # Loyal Customers: High frequency, good monetary
        (rfm_data['F_Score'] >= 4) & (rfm_data['M_Score'] >= 3),

        # Potential Loyalists: Recent customers with good frequency
        (rfm_data['R_Score'] >= 3) & (rfm_data['F_Score'] >= 2) & (rfm_data['F_Score'] <= 3),

        # New Customers: Recent but low frequency
        (rfm_data['R_Score'] >= 4) & (rfm_data['F_Score'] <= 2),

        # Promising: Recent customers with potential
        (rfm_data['R_Score'] >= 3) & (rfm_data['F_Score'] <= 2) & (rfm_data['M_Score'] >= 2),

        # Need Attention: Above average recency, frequency & monetary
        (rfm_data['R_Score'] >= 2) & (rfm_data['F_Score'] >= 2) & (rfm_data['M_Score'] >= 2),

        # About to Sleep: Below average recency but good frequency
        (rfm_data['R_Score'] <= 2) & (rfm_data['F_Score'] >= 3),

        # At Risk: Good customers who haven't purchased recently
        (rfm_data['R_Score'] <= 2) & (rfm_data['F_Score'] >= 2) & (rfm_data['M_Score'] >= 3),

        # Cannot Lose Them: High value but low recency and frequency
        (rfm_data['F_Score'] >= 4) & (rfm_data['M_Score'] >= 4),

        # Hibernating: Low recency, frequency & monetary
        (rfm_data['R_Score'] <= 2) & (rfm_data['F_Score'] <= 2) & (rfm_data['M_Score'] <= 2)
    ]

    choices = [
        "Champions", "Loyal Customers", "Potential Loyalists", "New Customers",
        "Promising", "Need Attention", "About to Sleep", "At Risk",
        "Cannot Lose Them", "Hibernating"
    ]


    rfm_data['Customer_Segment'] = np.select(conditions, choices, default="Lost")

    # Vectorized behavioral segmentation
    aov_threshold = rfm_data['Average_Order_Value'].quantile(0.75)
    freq_threshold = rfm_data['Monthly_Purchase_Frequency'].quantile(0.75)
    cat_threshold = rfm_data['Unique_Categories_Purchased'].quantile(0.75)
    loyalty_threshold = rfm_data['Store_Loyalty_Score'].quantile(0.75)

    behavioral_conditions = [
        (rfm_data['Average_Order_Value'] >= aov_threshold) & (rfm_data['Monthly_Purchase_Frequency'] >= freq_threshold),
        (rfm_data['Average_Order_Value'] >= aov_threshold) & (rfm_data['Monthly_Purchase_Frequency'] < freq_threshold),
        rfm_data['Monthly_Purchase_Frequency'] >= freq_threshold,
        rfm_data['Unique_Categories_Purchased'] >= cat_threshold,
        rfm_data['Store_Loyalty_Score'] >= loyalty_threshold
    ]

    behavioral_choices = [
        "High Value Frequent", "High Value Occasional", "Frequent Shoppers",
        "Category Explorers", "Store Loyalists"
    ]

    rfm_data['Behavioral_Segment'] = np.select(behavioral_conditions, behavioral_choices, default="Regular Customers")

    # Vectorized value segmentation
    high_value = rfm_data['Monetary_Value'].quantile(0.8)
    medium_value = rfm_data['Monetary_Value'].quantile(0.5)
    high_freq = rfm_data['Frequency'].quantile(0.7)

    value_conditions = [
        (rfm_data['Monetary_Value'] >= high_value) & (rfm_data['Frequency'] >= high_freq),
        (rfm_data['Monetary_Value'] >= high_value) & (rfm_data['Frequency'] < high_freq),
        (rfm_data['Monetary_Value'] >= medium_value) & (rfm_data['Frequency'] >= high_freq),
        (rfm_data['Monetary_Value'] >= medium_value) & (rfm_data['Frequency'] < high_freq),
        (rfm_data['Monetary_Value'] < medium_value) & (rfm_data['Frequency'] >= high_freq)
    ]

    value_choices = [
        "VIP Customers", "Big Spenders", "Loyal Regulars",
        "Medium Value", "Frequent Low Spenders"
    ]

    rfm_data['Value_Segment'] = np.select(value_conditions, value_choices, default="Low Value")

    print(f" Customer segmentation completed")
    print(f" RFM Segments: {rfm_data['Customer_Segment'].nunique()}")
    print(f" Behavioral Segments: {rfm_data['Behavioral_Segment'].nunique()}")
    print(f" Value Segments: {rfm_data['Value_Segment'].nunique()}")

    return rfm_data


def calculate_customer_lifetime_value(rfm_data):
    """Calculate Customer Lifetime Value (CLV) using computational methods"""
    print(" Calculating Customer Lifetime Value...")

    # Historical CLV (what customer has already spent)
    rfm_data['Historical_CLV'] = rfm_data['Monetary_Value']

    # Predicted CLV based on purchase patterns
    # Method 1: Simple frequency-based prediction
    rfm_data['Avg_Days_Between_Purchases'] = rfm_data['Customer_Lifetime_Days'] / (rfm_data['Frequency'] - 1)
    rfm_data['Avg_Days_Between_Purchases'] = rfm_data['Avg_Days_Between_Purchases'].fillna(rfm_data['Customer_Lifetime_Days'])

    # Estimate future purchases in next 12 months
    rfm_data['Estimated_Future_Purchases'] = np.where(
        rfm_data['Avg_Days_Between_Purchases'] > 0,
        365 / rfm_data['Avg_Days_Between_Purchases'],
        rfm_data['Monthly_Purchase_Frequency'] * 12
    )

    # Predicted CLV (conservative estimate)
    rfm_data['Predicted_CLV'] = rfm_data['Average_Order_Value'] * rfm_data['Estimated_Future_Purchases']

    # Total CLV (Historical + Predicted)
    rfm_data['Total_CLV'] = rfm_data['Historical_CLV'] + rfm_data['Predicted_CLV']

    # CLV segments based on total CLV
    rfm_data['CLV_Segment'] = pd.qcut(
        rfm_data['Total_CLV'],
        q=5,
        labels=['Very Low CLV', 'Low CLV', 'Medium CLV', 'High CLV', 'Very High CLV'],
        duplicates='drop'
    )

    # Customer risk assessment based on recency and frequency trends
    def risk_assessment(row):
        recency = row['Recency_Days']
        avg_gap = row['Avg_Days_Between_Purchases']

        if recency > avg_gap * 2:
            return "High Risk"
        elif recency > avg_gap * 1.5:
            return "Medium Risk"
        else:
            return "Low Risk"

    rfm_data['Churn_Risk'] = rfm_data.apply(risk_assessment, axis=1)

    print(" CLV calculation completed")

    return rfm_data

def create_cohort_analysis(master_data):
    """Create customer cohort analysis"""
    print(" Creating cohort analysis...")

    # Filter for sales only
    sales_data = master_data.filter(pl.col("Transaction Type") == "Sale")

    # Convert to pandas for cohort analysis
    df = sales_data.select([
        "Customer ID", "Date", "Line_Total_USD"
    ]).to_pandas()

    # Get customer's first purchase date
    df['Order_Period'] = df['Date'].dt.to_period('M')
    df['Cohort_Group'] = df.groupby('Customer ID')['Date'].transform('min').dt.to_period('M')

    # Calculate period number
    df['Period_Number'] = (df['Order_Period'] - df['Cohort_Group']).apply(attrgetter('n'))

    # Create cohort table for customer retention
    cohort_data = df.groupby(['Cohort_Group', 'Period_Number'])['Customer ID'].nunique().reset_index()
    cohort_counts = cohort_data.pivot(index='Cohort_Group', columns='Period_Number', values='Customer ID')

    # Calculate cohort sizes (first month customers)
    cohort_sizes = df.groupby('Cohort_Group')['Customer ID'].nunique()
    cohort_table = cohort_counts.divide(cohort_sizes, axis=0)

    # Revenue cohort analysis
    revenue_cohort_data = df.groupby(['Cohort_Group', 'Period_Number'])['Line_Total_USD'].sum().reset_index()
    revenue_cohort_table = revenue_cohort_data.pivot(index='Cohort_Group', columns='Period_Number', values='Line_Total_USD')

    print(" Cohort analysis completed")

    return cohort_table, cohort_counts, revenue_cohort_table

def analyze_customer_journey(master_data):
    """Analyze customer journey and purchase patterns"""
    print(" Analyzing customer journey patterns...")

    # Filter for sales only
    sales_data = master_data.filter(pl.col("Transaction Type") == "Sale")

    # Customer journey metrics
    journey_analysis = sales_data.group_by("Customer ID").agg([
        pl.col("Date").min().alias("First_Purchase"),
        pl.col("Date").max().alias("Last_Purchase"),
        pl.col("Invoice ID").n_unique().alias("Total_Transactions"),
        pl.col("Line_Total_USD").sum().alias("Total_Spent"),
        pl.col("Store ID").n_unique().alias("Stores_Visited"),
        pl.col("Category").n_unique().alias("Categories_Explored"),

        # Purchase evolution
        pl.col("Line_Total_USD").first().alias("First_Purchase_Amount"),
        pl.col("Line_Total_USD").last().alias("Last_Purchase_Amount"),
        pl.col("Line_Total_USD").mean().alias("Average_Purchase_Amount"),
        pl.col("Line_Total_USD").max().alias("Highest_Purchase_Amount")
    ]).to_pandas()

    # Calculate journey metrics
    journey_analysis['Customer_Lifespan_Days'] = (journey_analysis['Last_Purchase'] - journey_analysis['First_Purchase']).dt.days
    journey_analysis['Purchase_Growth'] = ((journey_analysis['Last_Purchase_Amount'] - journey_analysis['First_Purchase_Amount']) /
                                         journey_analysis['First_Purchase_Amount'] * 100).fillna(0)

    # Journey stages
    def journey_stage(row):
        transactions = row['Total_Transactions']
        lifespan = row['Customer_Lifespan_Days']

        if transactions == 1:
            return "One-Time Buyer"
        elif transactions <= 3 and lifespan <= 90:
            return "Early Stage"
        elif transactions <= 5 and lifespan <= 180:
            return "Developing"
        elif transactions <= 10:
            return "Established"
        else:
            return "Mature"

    journey_analysis['Journey_Stage'] = journey_analysis.apply(journey_stage, axis=1)

    print(" Customer journey analysis completed")

    return journey_analysis

def create_customer_segmentation_dashboard(rfm_data, cohort_table, revenue_cohort_table, journey_analysis):
    """Create comprehensive customer segmentation dashboard"""
    print(" Creating Customer Segmentation Dashboard...")

    # Create HTML structure
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Advanced Customer Segmentation Analysis Dashboard</title>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        <style>
            body { font-family: Arial, sans-serif; margin: 20px; background-color: #f5f5f5; }
            .chart-container { background-color: white; margin: 20px 0; padding: 20px; border-radius: 10px; box-shadow: 0 2px 5px rgba(0,0,0,0.1); }
            .chart-title { font-size: 24px; font-weight: bold; text-align: center; margin-bottom: 20px; color: #333; }
            .dashboard-title { font-size: 36px; font-weight: bold; text-align: center; margin-bottom: 30px; color: #2c3e50; }
            .insights-box { background-color: #e8f4f8; padding: 15px; margin: 10px 0; border-radius: 8px; font-size: 14px; }
            .metric-highlight { background-color: #fff3cd; padding: 10px; margin: 5px 0; border-radius: 5px; font-weight: bold; }
            .segment-summary { display: grid; grid-template-columns: repeat(auto-fit, minmax(300px, 1fr)); gap: 20px; margin: 20px 0; }
            .segment-card { background-color: #f8f9fa; padding: 15px; border-radius: 8px; border-left: 4px solid #007bff; }
        </style>
    </head>
    <body>
        <div class="dashboard-title"> Advanced Customer Segmentation Analysis</div>

        <div class="insights-box">
            <strong> Customer Segmentation Insights:</strong><br>
            • <strong>RFM Analysis</strong>: Segments customers based on Recency, Frequency, and Monetary value<br>
            • <strong>Behavioral Segmentation</strong>: Groups customers by shopping patterns and preferences<br>
            • <strong>Customer Lifetime Value</strong>: Predicts future value and identifies high-value customers<br>
            • <strong>Cohort Analysis</strong>: Tracks customer retention and revenue patterns over time<br>
            • <strong>Journey Analysis</strong>: Maps customer evolution and purchase progression
        </div>
    """

    # 1. RFM Analysis 3D Scatter Plot
    fig1 = px.scatter_3d(
        rfm_data,
        x='Recency_Days',
        y='Frequency',
        z='Monetary_Value',
        color='Customer_Segment',
        size='Total_CLV',
        hover_name='Customer ID',
        hover_data={
            'Average_Order_Value': ':.2f',
            'Total_Items_Purchased': ':,',
            'Stores_Visited': True,
            'CLV_Segment': True
        },
        title=" 3D RFM Analysis: Customer Segmentation",
        labels={
            'Recency_Days': 'Recency (Days)',
            'Frequency': 'Purchase Frequency',
            'Monetary_Value': 'Total Spent (USD)'
        },
        height=700
    )
    fig1.update_layout(template="plotly_white")

    # 2. Customer Segment Distribution
    segment_counts = rfm_data['Customer_Segment'].value_counts()
    fig2 = px.pie(
        values=segment_counts.values,
        names=segment_counts.index,
        title=" Customer Segment Distribution",
        height=600
    )
    fig2.update_traces(textposition='inside', textinfo='percent+label')
    fig2.update_layout(template="plotly_white")

    # 3. CLV Analysis by Segment
    clv_by_segment = rfm_data.groupby('Customer_Segment').agg({
        'Total_CLV': ['mean', 'sum', 'count'],
        'Historical_CLV': 'mean',
        'Predicted_CLV': 'mean'
    }).round(2)

    fig3 = px.bar(
        x=clv_by_segment.index,
        y=clv_by_segment[('Total_CLV', 'mean')],
        title=" Average Customer Lifetime Value by Segment",
        labels={'x': 'Customer Segment', 'y': 'Average CLV (USD)'},
        height=600
    )
    fig3.update_layout(template="plotly_white", xaxis_tickangle=45)

    # 4. Behavioral Segmentation
    behavioral_counts = rfm_data['Behavioral_Segment'].value_counts()
    fig4 = px.bar(
        x=behavioral_counts.index,
        y=behavioral_counts.values,
        title=" Behavioral Segmentation Distribution",
        labels={'x': 'Behavioral Segment', 'y': 'Number of Customers'},
        height=600
    )
    fig4.update_layout(template="plotly_white", xaxis_tickangle=45)

    # 5. Customer Journey Stage Analysis
    journey_counts = journey_analysis['Journey_Stage'].value_counts()
    fig5 = px.funnel(
        x=journey_counts.values,
        y=journey_counts.index,
        title=" Customer Journey Stage Distribution",
        height=600
    )
    fig5.update_layout(template="plotly_white")

    # 6. Cohort Retention Heatmap
    fig6 = px.imshow(
        cohort_table.iloc[:12, :12].values,  # Show first 12 months of first 12 cohorts
        labels=dict(x="Period", y="Cohort Month", color="Retention Rate"),
        x=[f"Month {i}" for i in range(12)],
        y=[str(cohort)[:7] for cohort in cohort_table.index[:12]],
        title=" Customer Retention Cohort Analysis",
        color_continuous_scale="RdYlGn",
        height=600
    )
    fig6.update_layout(template="plotly_white")

    # 7. Revenue Cohort Analysis
    fig7 = px.imshow(
        revenue_cohort_table.iloc[:12, :12].fillna(0).values,
        labels=dict(x="Period", y="Cohort Month", color="Revenue (USD)"),
        x=[f"Month {i}" for i in range(12)],
        y=[str(cohort)[:7] for cohort in revenue_cohort_table.index[:12]],
        title=" Revenue Cohort Analysis",
        color_continuous_scale="Blues",
        height=600
    )
    fig7.update_layout(template="plotly_white")

    # 8. Customer Value vs Risk Matrix
    fig8 = px.scatter(
        rfm_data,
        x='Total_CLV',
        y='Recency_Days',
        color='Churn_Risk',
        size='Frequency',
        hover_name='Customer ID',
        hover_data={
            'Customer_Segment': True,
            'Average_Order_Value': ':.2f',
            'Monthly_Purchase_Frequency': ':.2f'
        },
        title=" Customer Value vs Churn Risk Matrix",
        labels={
            'Total_CLV': 'Customer Lifetime Value (USD)',
            'Recency_Days': 'Days Since Last Purchase'
        },
        height=700
    )
    fig8.update_layout(template="plotly_white")

    # 9. Purchase Behavior Analysis
    fig9 = px.scatter(
        rfm_data,
        x='Average_Order_Value',
        y='Monthly_Purchase_Frequency',
        color='Value_Segment',
        size='Total_Items_Purchased',
        hover_name='Customer ID',
        title=" Purchase Behavior Analysis",
        labels={
            'Average_Order_Value': 'Average Order Value (USD)',
            'Monthly_Purchase_Frequency': 'Monthly Purchase Frequency'
        },
        height=600
    )
    fig9.update_layout(template="plotly_white")

    # 10. Customer Loyalty Analysis
    fig10 = px.scatter(
        rfm_data,
        x='Store_Loyalty_Score',
        y='Product_Diversity_Score',
        color='Behavioral_Segment',
        size='Monetary_Value',
        hover_name='Customer ID',
        title=" Customer Loyalty vs Product Diversity",
        labels={
            'Store_Loyalty_Score': 'Store Loyalty Score',
            'Product_Diversity_Score': 'Product Diversity Score'
        },
        height=600
    )
    fig10.update_layout(template="plotly_white")

    # Convert all figures to HTML
    figures = [fig1, fig2, fig3, fig4, fig5, fig6, fig7, fig8, fig9, fig10]
    titles = [
        "3D RFM Analysis: Customer Segmentation",
        "Customer Segment Distribution",
        "Average Customer Lifetime Value by Segment",
        "Behavioral Segmentation Distribution",
        "Customer Journey Stage Distribution",
        "Customer Retention Cohort Analysis",
        "Revenue Cohort Analysis",
        "Customer Value vs Churn Risk Matrix",
        "Purchase Behavior Analysis",
        "Customer Loyalty vs Product Diversity"
    ]

    for i, (fig, title) in enumerate(zip(figures, titles)):
        html_content += f"""
        <div class="chart-container">
            <div class="chart-title">{title}</div>
            <div id="chart{i+1}"></div>
        </div>
        """

    # Add segment summary cards
    segment_summary = rfm_data.groupby('Customer_Segment').agg({
        'Customer ID': 'count',
        'Total_CLV': 'mean',
        'Average_Order_Value': 'mean',
        'Monthly_Purchase_Frequency': 'mean',
        'Recency_Days': 'mean'
    }).round(2)

    html_content += """
        <div class="chart-container">
            <div class="chart-title"> Customer Segment Summary</div>
            <div class="segment-summary">
    """

    for segment in segment_summary.index:
        data = segment_summary.loc[segment]
        html_content += f"""
            <div class="segment-card">
                <h3>{segment}</h3>
                <p><strong>Customers:</strong> {data['Customer ID']:,}</p>
                <p><strong>Avg CLV:</strong> ${data['Total_CLV']:,.2f}</p>
                <p><strong>Avg Order Value:</strong> ${data['Average_Order_Value']:,.2f}</p>
                <p><strong>Monthly Frequency:</strong> {data['Monthly_Purchase_Frequency']:.2f}</p>
                <p><strong>Avg Recency:</strong> {data['Recency_Days']:.0f} days</p>
            </div>
        """

    html_content += """
            </div>
        </div>
    """

    # Add key insights
    total_customers = len(rfm_data)
    total_clv = rfm_data['Total_CLV'].sum()
    avg_clv = rfm_data['Total_CLV'].mean()
    champions = len(rfm_data[rfm_data['Customer_Segment'] == 'Champions'])
    at_risk = len(rfm_data[rfm_data['Customer_Segment'] == 'At Risk'])

    html_content += f"""
        <div class="insights-box">
            <strong> Key Customer Insights:</strong><br>
            <div class="metric-highlight"> Total Customers Analyzed: {total_customers:,}</div>
            <div class="metric-highlight"> Total Customer Lifetime Value: ${total_clv:,.0f}</div>
            <div class="metric-highlight"> Average CLV per Customer: ${avg_clv:,.2f}</div>
            <div class="metric-highlight"> Champion Customers: {champions:,} ({champions/total_customers*100:.1f}%)</div>
            <div class="metric-highlight"> At-Risk Customers: {at_risk:,} ({at_risk/total_customers*100:.1f}%)</div>
        </div>
    """

    html_content += """
    <script>
    """

    # Add JavaScript for each plot
    for i, fig in enumerate(figures):
        plot_json = fig.to_json()
        html_content += f"""
        var plotData{i+1} = {plot_json};
        Plotly.newPlot('chart{i+1}',
        plotData{i+1}.data, plotData{i+1}.layout, {{responsive: true}});
        """

    html_content += """
    </script>
    </body>
    </html>
    """

    return html_content


In [ ]:

def save_customer_segmentation_data(rfm_data, cohort_table, revenue_cohort_table, journey_analysis):
    """Save customer segmentation analysis data files"""
    print(" Saving customer segmentation analysis data files...")

    os.makedirs("customer_segmentation/data", exist_ok=True)

    # Save main datasets
    rfm_data.to_csv("customer_segmentation/data/rfm_customer_analysis.csv", index=False)
    cohort_table.to_csv("customer_segmentation/data/customer_retention_cohorts.csv")
    revenue_cohort_table.to_csv("customer_segmentation/data/revenue_cohorts.csv")
    journey_analysis.to_csv("customer_segmentation/data/customer_journey_analysis.csv", index=False)

    # Create segment-specific files
    for segment in rfm_data['Customer_Segment'].unique():
        segment_data = rfm_data[rfm_data['Customer_Segment'] == segment]
        filename = f"customer_segmentation/data/segment_{segment.lower().replace(' ', '_')}.csv"
        segment_data.to_csv(filename, index=False)

    # Create summary reports
    segment_summary = rfm_data.groupby('Customer_Segment').agg({
        'Customer ID': 'count',
        'Total_CLV': ['mean', 'sum', 'std'],
        'Historical_CLV': 'mean',
        'Predicted_CLV': 'mean',
        'Average_Order_Value': 'mean',
        'Monthly_Purchase_Frequency': 'mean',
        'Recency_Days': 'mean',
        'Frequency': 'mean',
        'Monetary_Value': 'mean',
        'Total_Items_Purchased': 'mean',
        'Unique_Products_Purchased': 'mean',
        'Stores_Visited': 'mean'
    }).round(2)

    segment_summary.to_csv("customer_segmentation/data/segment_summary_report.csv")

    # CLV analysis by segment
    clv_analysis = rfm_data.groupby(['Customer_Segment', 'CLV_Segment']).size().unstack(fill_value=0)
    clv_analysis.to_csv("customer_segmentation/data/clv_by_segment_analysis.csv")

    # Risk analysis
    risk_analysis = rfm_data.groupby(['Customer_Segment', 'Churn_Risk']).size().unstack(fill_value=0)
    risk_analysis.to_csv("customer_segmentation/data/churn_risk_by_segment.csv")

    print(" Customer segmentation data files saved!")

def generate_customer_insights_report(rfm_data, journey_analysis):
    """Generate detailed customer insights and recommendations"""
    print(" Generating customer insights and recommendations...")

    insights = {
        'segment_insights': {},
        'clv_insights': {},
        'risk_insights': {},
        'behavioral_insights': {},
        'recommendations': {}
    }

    # Segment-specific insights
    for segment in rfm_data['Customer_Segment'].unique():
        segment_data = rfm_data[rfm_data['Customer_Segment'] == segment]

        insights['segment_insights'][segment] = {
            'count': len(segment_data),
            'percentage': len(segment_data) / len(rfm_data) * 100,
            'avg_clv': segment_data['Total_CLV'].mean(),
            'avg_aov': segment_data['Average_Order_Value'].mean(),
            'avg_frequency': segment_data['Monthly_Purchase_Frequency'].mean(),
            'avg_recency': segment_data['Recency_Days'].mean(),
            'revenue_contribution': segment_data['Monetary_Value'].sum() / rfm_data['Monetary_Value'].sum() * 100
        }

    # CLV insights
    high_clv_customers = rfm_data[rfm_data['CLV_Segment'].isin(['High CLV', 'Very High CLV'])]
    insights['clv_insights'] = {
        'high_clv_count': len(high_clv_customers),
        'high_clv_percentage': len(high_clv_customers) / len(rfm_data) * 100,
        'high_clv_revenue_share': high_clv_customers['Monetary_Value'].sum() / rfm_data['Monetary_Value'].sum() * 100,
        'avg_clv_all': rfm_data['Total_CLV'].mean(),
        'top_10_percent_clv': rfm_data.nlargest(int(len(rfm_data) * 0.1), 'Total_CLV')['Total_CLV'].mean()
    }

    # Risk insights
    at_risk_customers = rfm_data[rfm_data['Churn_Risk'] == 'High Risk']
    insights['risk_insights'] = {
        'high_risk_count': len(at_risk_customers),
        'high_risk_percentage': len(at_risk_customers) / len(rfm_data) * 100,
        'potential_revenue_loss': at_risk_customers['Predicted_CLV'].sum(),
        'avg_recency_at_risk': at_risk_customers['Recency_Days'].mean()
    }

    # Behavioral insights
    insights['behavioral_insights'] = {
        'multi_category_shoppers': len(rfm_data[rfm_data['Unique_Categories_Purchased'] > 3]) / len(rfm_data) * 100,
        'store_loyalists': len(rfm_data[rfm_data['Stores_Visited'] == 1]) / len(rfm_data) * 100,
        'frequent_shoppers': len(rfm_data[rfm_data['Monthly_Purchase_Frequency'] > 2]) / len(rfm_data) * 100,
        'high_aov_customers': len(rfm_data[rfm_data['Average_Order_Value'] > rfm_data['Average_Order_Value'].quantile(0.8)]) / len(rfm_data) * 100
    }

    # Generate recommendations
    insights['recommendations'] = {
        'champions': "Focus on retention programs, VIP experiences, and referral incentives",
        'loyal_customers': "Reward loyalty with exclusive offers and early access to new products",
        'potential_loyalists': "Nurture with personalized recommendations and loyalty programs",
        'new_customers': "Onboard with welcome series and product education",
        'at_risk': "Immediate re-engagement campaigns with special offers",
        'cannot_lose_them': "Win-back campaigns with premium service recovery",
        'hibernating': "Reactivation campaigns with significant incentives",
        'lost': "Final win-back attempt or remove from active marketing"
    }

    return insights

def run_customer_segmentation_analysis():
    """Run comprehensive customer segmentation analysis"""
    print(" Starting Advanced Customer Segmentation Analysis...")

    # Load data
    master_data = load_master_data_for_customer_segmentation()
    if master_data is None:
        print(" Failed to load master data!")
        return None

    # Calculate RFM analysis
    print("\n Step 1: Calculating RFM Analysis...")
    rfm_data = calculate_advanced_rfm_analysis(master_data)

    # Create customer segments
    print("\n Step 2: Creating Customer Segments...")
    rfm_data = create_customer_segments(rfm_data)

    # Calculate CLV
    print("\n Step 3: Calculating Customer Lifetime Value...")
    rfm_data = calculate_customer_lifetime_value(rfm_data)

    # Cohort analysis
    print("\n Step 4: Performing Cohort Analysis...")
    cohort_table, cohort_counts, revenue_cohort_table = create_cohort_analysis(master_data)

    # Customer journey analysis
    print("\n Step 5: Analyzing Customer Journey...")
    journey_analysis = analyze_customer_journey(master_data)

    # Generate insights
    print("\n Step 6: Generating Insights and Recommendations...")
    insights = generate_customer_insights_report(rfm_data, journey_analysis)

    # Save data files
    print("\n Step 7: Saving Analysis Data...")
    save_customer_segmentation_data(rfm_data, cohort_table, revenue_cohort_table, journey_analysis)

    # Create dashboard
    print("\n Step 8: Creating Interactive Dashboard...")
    html_content = create_customer_segmentation_dashboard(rfm_data, cohort_table, revenue_cohort_table, journey_analysis)

    # Create directory and save HTML
    os.makedirs("customer_segmentation", exist_ok=True)

    with open("customer_segmentation/customer_segmentation_dashboard.html", "w", encoding='utf-8') as f:
        f.write(html_content)

    # Print comprehensive summary
    print("\n" + "="*80)
    print(" CUSTOMER SEGMENTATION ANALYSIS COMPLETE!")
    print("="*80)

    print(f"\n CUSTOMER OVERVIEW:")
    print(f"    Total Customers Analyzed: {len(rfm_data):,}")
    print(f"    Total Revenue: ${rfm_data['Monetary_Value'].sum():,.2f}")
    print(f"    Average CLV: ${rfm_data['Total_CLV'].mean():,.2f}")
    print(f"    Average Order Value: ${rfm_data['Average_Order_Value'].mean():.2f}")

    print(f"\n SEGMENT DISTRIBUTION:")
    segment_dist = rfm_data['Customer_Segment'].value_counts()
    for segment, count in segment_dist.head(5).items():
        percentage = count / len(rfm_data) * 100
        print(f"   • {segment}: {count:,} customers ({percentage:.1f}%)")

    print(f"\n HIGH-VALUE CUSTOMERS:")
    champions = rfm_data[rfm_data['Customer_Segment'] == 'Champions']
    print(f"    Champions: {len(champions):,} customers")
    print(f"  Champions Revenue: ${champions['Monetary_Value'].sum():,.2f}")
    print(f"   Champions CLV: ${champions['Total_CLV'].mean():,.2f}")

    print(f"\n AT-RISK CUSTOMERS:")
    at_risk = rfm_data[rfm_data['Churn_Risk'] == 'High Risk']
    print(f"    High Risk: {len(at_risk):,} customers ({len(at_risk)/len(rfm_data)*100:.1f}%)")
    print(f"    Potential Revenue Loss: ${at_risk['Predicted_CLV'].sum():,.2f}")

    print(f"\n BEHAVIORAL INSIGHTS:")
    print(f"  Multi-Category Shoppers: {insights['behavioral_insights']['multi_category_shoppers']:.1f}%")
    print(f"  Store Loyalists: {insights['behavioral_insights']['store_loyalists']:.1f}%")
    print(f"  Frequent Shoppers: {insights['behavioral_insights']['frequent_shoppers']:.1f}%")

    print(f"\n FILES CREATED:")
    print(f"   customer_segmentation/customer_segmentation_dashboard.html")
    print(f"   customer_segmentation/data/ (Multiple CSV files)")
    print(f"  Segment-specific customer lists")
    print(f"   Summary reports and analysis")

    print(f"\n TOP RECOMMENDATIONS:")
    print(f"   1. Focus retention efforts on {len(champions):,} Champion customers")
    print(f"   2. Immediate re-engagement for {len(at_risk):,} at-risk customers")
    print(f"   3. Develop loyalty programs for Potential Loyalists")
    print(f"   4. Create win-back campaigns for hibernating customers")

    return {
        'rfm_data': rfm_data,
        'cohort_table': cohort_table,
        'revenue_cohort_table': revenue_cohort_table,
        'journey_analysis': journey_analysis,
        'insights': insights
    }

# Run the customer segmentation analysis
print(" Initializing Advanced Customer Segmentation Analysis...")
results = run_customer_segmentation_analysis()

if results:
    print("\n Analysis completed successfully!")
    print("Open the dashboard to explore detailed customer insights and segments")
else:
    print("\n Analysis failed. Please check the data and try again.")


 Initializing Advanced Customer Segmentation Analysis...
 Starting Advanced Customer Segmentation Analysis...
 Loading master data for customer segmentation analysis...
 Master data loaded: (6416029, 41)

 Step 1: Calculating RFM Analysis...
Calculating Advanced RFM Analysis...
Analysis reference date: 2025-03-18
 Advanced RFM analysis completed for 1,283,707 customers

 Step 2: Creating Customer Segments...
 Creating detailed customer segments...
 Customer segmentation completed
 RFM Segments: 10
 Behavioral Segments: 6
 Value Segments: 6

 Step 3: Calculating Customer Lifetime Value...
 Calculating Customer Lifetime Value...
 CLV calculation completed

 Step 4: Performing Cohort Analysis...
 Creating cohort analysis...
 Cohort analysis completed

 Step 5: Analyzing Customer Journey...
 Analyzing customer journey patterns...
 Customer journey analysis completed

 Step 6: Generating Insights and Recommendations...
 Generating customer insights and recommendations...

 Step 7: Saving An

/tmp/ipython-input-892873010.py:38: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



 Customer segmentation data files saved!

 Step 8: Creating Interactive Dashboard...
 Creating Customer Segmentation Dashboard...

 CUSTOMER SEGMENTATION ANALYSIS COMPLETE!

 CUSTOMER OVERVIEW:
    Total Customers Analyzed: 1,283,707
    Total Revenue: $305,884,836.55
    Average CLV: $410.86
    Average Order Value: $50.16

 SEGMENT DISTRIBUTION:
   • Potential Loyalists: 268,689 customers (20.9%)
   • Champions: 250,673 customers (19.5%)
   • Loyal Customers: 241,294 customers (18.8%)
   • Hibernating: 229,024 customers (17.8%)
   • Need Attention: 126,280 customers (9.8%)

 HIGH-VALUE CUSTOMERS:
    Champions: 250,673 customers
  Champions Revenue: $126,623,570.15
   Champions CLV: $696.92

 AT-RISK CUSTOMERS:
    High Risk: 203,138 customers (15.8%)
    Potential Revenue Loss: $123,512,118.84

 BEHAVIORAL INSIGHTS:
  Multi-Category Shoppers: 0.0%
  Store Loyalists: 75.5%
  Frequent Shoppers: 28.3%

 FILES CREATED:
   customer_segmentation/customer_segmentation_dashboard.html
   cus

In [ ]:
# Add this line before loading your data
import gc; gc.collect(); print(f"🧹 Memory cleared: {gc.collect()} objects")

🧹 Memory cleared: 0 objects


## Statistical Analysis (Correlation & Regression Analysis)

Qty vs Sales (Employee, custoner, product etc)


In [ ]:
import polars as pl
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import os

def load_master_data():
    """Load master transaction data"""
    print("Loading Master Data...")
    master_data = pl.read_parquet("data/master_transactions.parquet")
    sales_data = master_data.filter(pl.col("Transaction Type") == "Sale")
    print(f"Loaded {sales_data.shape[0]:,} sales transactions")
    return sales_data

def prepare_analysis_datasets(sales_data):
    """Prepare datasets for analysis"""
    print("Preparing Analysis Datasets...")

    # Create output directory
    os.makedirs("price_elasticity_analysis", exist_ok=True)

    # 1. Price Elasticity by Country
    print("  Computing price elasticity...")
    countries = sales_data["Country"].unique().to_list()
    elasticity_results = []

    for country in countries:
        country_data = sales_data.filter(pl.col("Country") == country)
        df = country_data.select(["Unit_Price_USD", "Quantity"]).to_pandas().dropna()
        df = df[(df['Unit_Price_USD'] > 0) & (df['Quantity'] > 0)]

        if len(df) < 50:
            continue

        # Calculate elasticity using price deciles
        df['Price_Decile'] = pd.qcut(df['Unit_Price_USD'], q=10, labels=False, duplicates='drop')
        price_demand = df.groupby('Price_Decile').agg({
            'Unit_Price_USD': 'mean',
            'Quantity': 'sum'
        }).reset_index()

        if len(price_demand) < 5:
            continue

        # Log-log regression for elasticity
        log_price = np.log(price_demand['Unit_Price_USD'])
        log_qty = np.log(price_demand['Quantity'])
        mask = np.isfinite(log_price) & np.isfinite(log_qty)

        if mask.sum() >= 3:
            from scipy import stats
            slope, _, _, _, _ = stats.linregress(log_price[mask], log_qty[mask])
            elasticity_results.append({
                'Country': country,
                'Price_Elasticity': round(abs(slope), 4)
            })

    elasticity_df = pl.DataFrame(elasticity_results)
    elasticity_df.write_csv("price_elasticity_analysis/price_elasticity_data.csv")

    # 2. Sales vs Quantity Analysis Datasets
    datasets = {}

    # Employee Analysis
    datasets['employee'] = sales_data.group_by("Employee ID").agg([
        pl.col("Line_Total_USD").sum().alias("Sales"),
        pl.col("Quantity").sum().alias("Qty")
    ]).filter(pl.col("Sales") > 0)

    # Country Analysis
    datasets['country'] = sales_data.group_by("Country").agg([
        pl.col("Line_Total_USD").sum().alias("Sales"),
        pl.col("Quantity").sum().alias("Qty"),
        pl.col("Employee ID").n_unique().alias("Staff_Count")
    ])

    # City Analysis
    datasets['city'] = sales_data.group_by(["City", "Country"]).agg([
        pl.col("Line_Total_USD").sum().alias("Sales"),
        pl.col("Quantity").sum().alias("Qty")
    ])

    # Customer Analysis
    datasets['customer'] = sales_data.group_by("Customer ID").agg([
        pl.col("Line_Total_USD").sum().alias("Sales"),
        pl.col("Quantity").sum().alias("Qty")
    ]).filter(pl.col("Sales") > 0)

    # Product Analysis
    datasets['product'] = sales_data.group_by("Name").agg([
        pl.col("Line_Total_USD").sum().alias("Sales"),
        pl.col("Quantity").sum().alias("Qty")
    ]).filter(pl.col("Sales") > 0)

    # Save all datasets
    for name, data in datasets.items():
        data.write_csv(f"price_elasticity_analysis/{name}_analysis_data.csv")

    # 3. Multiple Regression Dataset (Country level)
    multiple_reg_data = datasets['country'].filter(pl.col("Staff_Count") > 0)
    multiple_reg_data.write_csv("price_elasticity_analysis/multiple_regression_data.csv")

    print(f"Prepared {len(datasets)} analysis datasets + elasticity + multiple regression")

def perform_regression_analysis():
    """Perform regression analysis from saved datasets"""
    print("Performing Regression Analysis...")

    ols_summary_data = []

    # Analysis types
    analysis_types = ['employee', 'country', 'city', 'customer', 'product']

    for analysis_type in analysis_types:
        # Load data
        df = pl.read_csv(f"price_elasticity_analysis/{analysis_type}_analysis_data.csv").to_pandas()
        df = df.dropna()
        df = df[(df['Sales'] > 0) & (df['Qty'] > 0)]

        if len(df) < 3:
            continue

        # OLS Regression: Sales = coefficient * Qty + intercept
        X = df['Qty'].values
        y = df['Sales'].values
        X_ols = sm.add_constant(X)
        model = sm.OLS(y, X_ols).fit()

        # Extract statistics
        intercept = model.params[0]
        coefficient = model.params[1]
        r2 = model.rsquared
        p_value = model.pvalues[1]

        # Add to OLS summary
        ols_summary_data.append({
            'Analysis_Type': analysis_type.title(),
            'Equation': f"Sales = {coefficient:.1f} × Qty + {intercept:.0f}",
            'R_Squared': f"{r2:.4f}",
            'P_Value': f"{p_value:.6f}",
            'Coefficient': f"{coefficient:.2f}",
            'Intercept': f"{intercept:.2f}",
            'Data_Points': len(df)
        })

    # Multiple Regression
    df_multi = pl.read_csv("price_elasticity_analysis/multiple_regression_data.csv").to_pandas().dropna()

    if len(df_multi) >= 3 and 'Staff_Count' in df_multi.columns:
        X_multi = df_multi[['Staff_Count', 'Qty']].values
        y_multi = df_multi['Sales'].values
        X_multi_ols = sm.add_constant(X_multi)
        model_multi = sm.OLS(y_multi, X_multi_ols).fit()

        intercept_multi = model_multi.params[0]
        coef_staff = model_multi.params[1]
        coef_qty = model_multi.params[2]
        r2_multi = model_multi.rsquared
        p_value_multi = model_multi.f_pvalue

        # Add to OLS summary
        ols_summary_data.append({
            'Analysis_Type': 'Multiple Regression',
            'Equation': f"Sales = {coef_staff:.1f} × Staff + {coef_qty:.1f} × Qty + {intercept_multi:.0f}",
            'R_Squared': f"{r2_multi:.4f}",
            'P_Value': f"{p_value_multi:.6f}",
            'Coefficient': f"Staff:{coef_staff:.1f}, Qty:{coef_qty:.1f}",
            'Intercept': f"{intercept_multi:.2f}",
            'Data_Points': len(df_multi)
        })

    # Save OLS Summary
    ols_summary_df = pl.DataFrame(ols_summary_data)
    ols_summary_df.write_csv("price_elasticity_analysis/ols_summary.csv")

    print("Regression analysis completed")

def create_dashboard():
    """Create dashboard using same logic as workforce productivity"""
    print("Creating Dashboard...")

    # Load data from CSV files
    elasticity_df = pl.read_csv("price_elasticity_analysis/price_elasticity_data.csv").to_pandas()
    ols_summary_df = pl.read_csv("price_elasticity_analysis/ols_summary.csv").to_pandas()

    # Create individual figures first (same as workforce dashboard)
    figures = []

    # 1. Price Elasticity Chart
    fig1 = px.bar(
        elasticity_df,
        x='Country',
        y='Price_Elasticity',
        title='Price Elasticity by Country',
        text='Price_Elasticity'
    )
    fig1.update_traces(texttemplate='%{text:.4f}', textposition='outside')
    fig1.update_layout(height=400)
    figures.append(fig1)

    # 2. OLS Summary Table
    fig2 = go.Figure(data=[go.Table(
        header=dict(values=list(ols_summary_df.columns),
                   fill_color='lightblue',
                   align='center'),
        cells=dict(values=[ols_summary_df[col] for col in ols_summary_df.columns],
                  fill_color='white',
                  align='center')
    )])
    fig2.update_layout(title="OLS Summary Statistics", height=400)
    figures.append(fig2)

    # 3-7. Sales vs Quantity Regression Charts
    analysis_types = ['employee', 'country', 'city', 'customer', 'product']

    for analysis_type in analysis_types:
        try:
            df = pl.read_csv(f"price_elasticity_analysis/{analysis_type}_analysis_data.csv").to_pandas()
            df = df.dropna()
            df = df[(df['Sales'] > 0) & (df['Qty'] > 0)]

            if len(df) < 3:
                continue

            # Calculate regression
            X = df['Qty'].values
            y = df['Sales'].values
            X_ols = sm.add_constant(X)
            model = sm.OLS(y, X_ols).fit()
            y_pred = model.params[0] + model.params[1] * X

            # Calculate medians
            sales_median = df['Sales'].median()
            qty_median = df['Qty'].median()

            # Create scatter plot
            fig = go.Figure()

            # Add scatter points
            fig.add_trace(go.Scatter(
                x=df['Qty'],
                y=df['Sales'],
                mode='markers',
                name='Data Points',
                marker=dict(size=6, opacity=0.6)
            ))

            # Add regression line
            fig.add_trace(go.Scatter(
                x=df['Qty'],
                y=y_pred,
                mode='lines',
                name='Regression Line',
                line=dict(color='red', width=2)
            ))

            # Add median lines
            fig.add_shape(
                type="line",
                x0=qty_median, y0=df['Sales'].min(),
                x1=qty_median, y1=df['Sales'].max(),
                line=dict(color="gray", width=1, dash="dash")
            )

            fig.add_shape(
                type="line",
                x0=df['Qty'].min(), y0=sales_median,
                x1=df['Qty'].max(), y1=sales_median,
                line=dict(color="gray", width=1, dash="dash")
            )

            # Add equation annotation
            equation = f"Sales = {model.params[1]:.1f} × Qty + {model.params[0]:.0f}"
            r2 = model.rsquared

            fig.add_annotation(
                x=0.05, y=0.95,
                xref="paper", yref="paper",
                text=f"{equation}<br>R² = {r2:.3f}",
                showarrow=False,
                font=dict(size=12),
                bgcolor="rgba(255,255,255,0.8)",
                bordercolor="gray",
                borderwidth=1
            )

            fig.update_layout(
                title=f'{analysis_type.title()}: Sales vs Quantity',
                xaxis_title='Quantity',
                yaxis_title='Sales (USD)',
                height=400
            )

            figures.append(fig)

        except Exception as e:
            print(f"  Could not create chart for {analysis_type}: {e}")
            continue

    # 8. Multiple Regression 3D
    try:
        multiple_reg_data = pl.read_csv("price_elasticity_analysis/multiple_regression_data.csv").to_pandas()

        if len(multiple_reg_data) >= 3:
            fig3d = go.Figure(data=[go.Scatter3d(
                x=multiple_reg_data['Staff_Count'],
                y=multiple_reg_data['Qty'],
                z=multiple_reg_data['Sales'],
                mode='markers+text',
                text=multiple_reg_data['Country'] if 'Country' in multiple_reg_data.columns else None,
                marker=dict(
                    size=8,
                    color=multiple_reg_data['Sales'],
                    colorscale='Viridis',
                    showscale=True
                )
            )])

            fig3d.update_layout(
                title='Multiple Regression: Sales = f(Staff Count, Quantity)',
                scene=dict(
                    xaxis_title='Staff Count',
                    yaxis_title='Quantity',
                    zaxis_title='Sales (USD)'
                ),
                height=500
            )

            figures.append(fig3d)
    except:
        pass

    # Combine all figures into dashboard (same as workforce dashboard)
    dashboard_html = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Price Elasticity & Regression Analysis Dashboard</title>
        <style>
            body { font-family: Arial, sans-serif; margin: 20px; }
            .chart-container { margin-bottom: 30px; }
            h1 { text-align: center; color: #2E86AB; }
        </style>
    </head>
    <body>
        <h1>Price Elasticity & Sales-Quantity Regression Analysis Dashboard</h1>
    """

    # Add each figure to HTML
    for i, fig in enumerate(figures):
        chart_html = fig.to_html(include_plotlyjs='inline' if i == 0 else False, div_id=f"chart_{i}")
        dashboard_html += f'<div class="chart-container">{chart_html}</div>'

    dashboard_html += """
    </body>
    </html>
    """

    # Save dashboard
    with open("price_elasticity_analysis/price_elasticity_dashboard.html", "w", encoding='utf-8') as f:
        f.write(dashboard_html)

    print("Dashboard saved as: price_elasticity_dashboard.html")

    # Return first figure for display
    return figures[0] if figures else None

def main():
    """Main analysis pipeline"""
    print("PRICE ELASTICITY & REGRESSION ANALYSIS")
    print("=" * 60)

    # Step 1: Load master data
    sales_data = load_master_data()

    # Step 2: Prepare analysis datasets and save to CSV
    prepare_analysis_datasets(sales_data)

    # Step 3: Perform regression analysis from CSV data
    perform_regression_analysis()

    # Step 4: Create dashboard from CSV data
    dashboard_fig = create_dashboard()

    # Step 5: Display first chart
    if dashboard_fig:
        print("\nDisplaying Sample Chart...")
        dashboard_fig.show()

    print("\nANALYSIS COMPLETED!")
    print("Results saved in: price_elasticity_analysis/")
    print("Dashboard: price_elasticity_dashboard.html")


main()


PRICE ELASTICITY & REGRESSION ANALYSIS
Loading Master Data...
Loaded 6,077,200 sales transactions
Preparing Analysis Datasets...
  Computing price elasticity...
Prepared 5 analysis datasets + elasticity + multiple regression
Performing Regression Analysis...
Regression analysis completed
Creating Dashboard...
Dashboard saved as: price_elasticity_dashboard.html

Displaying Sample Chart...



ANALYSIS COMPLETED!
Results saved in: price_elasticity_analysis/
Dashboard: price_elasticity_dashboard.html


In [ ]:
import polars as pl
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import os

def load_master_data():
    """Load master transaction data"""
    print(" Loading Master Data...")
    master_data = pl.read_parquet("data/master_transactions.parquet")
    sales_data = master_data.filter(pl.col("Transaction Type") == "Sale")
    print(f"✓ Loaded {sales_data.shape[0]:,} sales transactions")
    return sales_data

def prepare_correlation_data(sales_data):
    """Prepare data for correlation analysis - aggregated by Store ID"""
    print(" Preparing Correlation Dataset by Store ID...")

    # Create output directory
    os.makedirs("correlation_analysis_results", exist_ok=True)

    # Aggregate by Store ID
    correlation_data = sales_data.group_by("Store ID").agg([
        pl.col("Line_Total_USD").sum().alias("Total_Sales"),
        pl.col("Employee ID").n_unique().alias("Staff_Count"),
        pl.col("Quantity").sum().alias("Total_Quantity")
    ]).filter(
        (pl.col("Total_Sales") > 0) &
        (pl.col("Staff_Count") > 0) &
        (pl.col("Total_Quantity") > 0)
    )

    # Save dataset
    correlation_data.write_csv("correlation_analysis_results/correlation_data.csv")

    print(f"✓ Dataset prepared: {correlation_data.shape[0]} stores")
    print("✓ Variables: Total_Sales, Staff_Count, Total_Quantity")
    return correlation_data

def run_correlation_analysis():
    """Run correlation analysis between key variables"""
    print(" Running Correlation Analysis...")

    # Load data
    df = pl.read_csv("correlation_analysis_results/correlation_data.csv").to_pandas()
    df = df.dropna()

    print(f"✓ Analysis dataset: {len(df)} stores")

    # Calculate correlations with p-values
    corr_staff_sales, p_staff_sales = pearsonr(df['Staff_Count'], df['Total_Sales'])
    corr_staff_quantity, p_staff_quantity = pearsonr(df['Staff_Count'], df['Total_Quantity'])
    corr_quantity_sales, p_quantity_sales = pearsonr(df['Total_Quantity'], df['Total_Sales'])

    # Print correlation results
    print("\n" + "="*80)
    print("CORRELATION ANALYSIS RESULTS")
    print("="*80)

    print(f"\n CORRELATION COEFFICIENTS:")
    print(f"   Staff Count vs Total Sales:    r = {corr_staff_sales:.3f} (p = {p_staff_sales:.3f})")
    print(f"   Staff Count vs Total Quantity: r = {corr_staff_quantity:.3f} (p = {p_staff_quantity:.3f})")
    print(f"   Total Quantity vs Total Sales: r = {corr_quantity_sales:.3f} (p = {p_quantity_sales:.3f})")

    # Interpretation
    print(f"\n STATISTICAL SIGNIFICANCE (α = 0.05):")
    print(f"   Staff vs Sales:    {'Significant' if p_staff_sales < 0.05 else 'Not Significant'}")
    print(f"   Staff vs Quantity: {'Significant' if p_staff_quantity < 0.05 else 'Not Significant'}")
    print(f"   Quantity vs Sales: {'Significant' if p_quantity_sales < 0.05 else 'Not Significant'}")

    # Correlation strength interpretation
    def interpret_correlation(r):
        abs_r = abs(r)
        if abs_r >= 0.7:
            return "Strong"
        elif abs_r >= 0.3:
            return "Moderate"
        elif abs_r >= 0.1:
            return "Weak"
        else:
            return "Very Weak"

    print(f"\n CORRELATION STRENGTH:")
    print(f"   Staff vs Sales:    {interpret_correlation(corr_staff_sales)} ({corr_staff_sales:.3f})")
    print(f"   Staff vs Quantity: {interpret_correlation(corr_staff_quantity)} ({corr_staff_quantity:.3f})")
    print(f"   Quantity vs Sales: {interpret_correlation(corr_quantity_sales)} ({corr_quantity_sales:.3f})")

    # Create correlation matrix
    correlation_matrix = df[['Staff_Count', 'Total_Sales', 'Total_Quantity']].corr()

    print(f"\n CORRELATION MATRIX:")
    print(correlation_matrix.round(3))

    # Save results
    results_summary = {
        'Correlation_Pair': ['Staff_Count vs Total_Sales', 'Staff_Count vs Total_Quantity', 'Total_Quantity vs Total_Sales'],
        'Correlation_Coefficient': [corr_staff_sales, corr_staff_quantity, corr_quantity_sales],
        'P_Value': [p_staff_sales, p_staff_quantity, p_quantity_sales],
        'Significance': [p_staff_sales < 0.05, p_staff_quantity < 0.05, p_quantity_sales < 0.05],
        'Strength': [interpret_correlation(corr_staff_sales), interpret_correlation(corr_staff_quantity), interpret_correlation(corr_quantity_sales)]
    }

    results_df = pd.DataFrame(results_summary)
    results_df.to_csv("correlation_analysis_results/correlation_results.csv", index=False)

    # Save correlation matrix
    correlation_matrix.to_csv("correlation_analysis_results/correlation_matrix.csv")

    # Save detailed summary
    with open("correlation_analysis_results/correlation_summary.txt", "w") as f:
        f.write("CORRELATION ANALYSIS SUMMARY\n")
        f.write("="*50 + "\n\n")
        f.write(f"Dataset: {len(df)} stores analyzed\n\n")
        f.write("CORRELATION RESULTS:\n")
        f.write(f"Staff Count vs Total Sales:    r = {corr_staff_sales:.3f} (p = {p_staff_sales:.3f}) - {interpret_correlation(corr_staff_sales)}\n")
        f.write(f"Staff Count vs Total Quantity: r = {corr_staff_quantity:.3f} (p = {p_staff_quantity:.3f}) - {interpret_correlation(corr_staff_quantity)}\n")
        f.write(f"Total Quantity vs Total Sales: r = {corr_quantity_sales:.3f} (p = {p_quantity_sales:.3f}) - {interpret_correlation(corr_quantity_sales)}\n\n")
        f.write("CORRELATION MATRIX:\n")
        f.write(str(correlation_matrix.round(3)))

    # Print summary statistics
    print(f"\n DATASET SUMMARY:")
    print(f"   Stores analyzed: {len(df)}")
    print(f"   Total Sales range: ${df['Total_Sales'].min():,.0f} - ${df['Total_Sales'].max():,.0f}")
    print(f"   Staff Count range: {df['Staff_Count'].min()} - {df['Staff_Count'].max()}")
    print(f"   Total Quantity range: {df['Total_Quantity'].min():,.0f} - {df['Total_Quantity'].max():,.0f}")

    print(f"\n✓ Results saved to: correlation_analysis_results/")
    print(f"✓ Summary: correlation_summary.txt")
    print(f"✓ Results: correlation_results.csv")
    print(f"✓ Matrix: correlation_matrix.csv")

    return {
        'staff_sales_corr': corr_staff_sales,
        'staff_sales_p': p_staff_sales,
        'staff_quantity_corr': corr_staff_quantity,
        'staff_quantity_p': p_staff_quantity,
        'quantity_sales_corr': corr_quantity_sales,
        'quantity_sales_p': p_quantity_sales,
        'correlation_matrix': correlation_matrix,
        'data': df
    }

def main():
    """Main function - Correlation analysis by Store ID"""
    print(" CORRELATION ANALYSIS BY STORE")
    print("Variables: Staff_Count, Total_Sales, Total_Quantity")
    print("Aggregation Level: Store ID")
    print("="*70)

    # Load data
    sales_data = load_master_data()

    # Prepare correlation dataset by Store ID
    correlation_data = prepare_correlation_data(sales_data)

    # Run correlation analysis
    results = run_correlation_analysis()

    print("\n CORRELATION ANALYSIS COMPLETED!")

    return results

main()

 CORRELATION ANALYSIS BY STORE
Variables: Staff_Count, Total_Sales, Total_Quantity
Aggregation Level: Store ID
 Loading Master Data...
✓ Loaded 6,077,200 sales transactions
 Preparing Correlation Dataset by Store ID...
✓ Dataset prepared: 35 stores
✓ Variables: Total_Sales, Staff_Count, Total_Quantity
 Running Correlation Analysis...
✓ Analysis dataset: 35 stores

CORRELATION ANALYSIS RESULTS

 CORRELATION COEFFICIENTS:
   Staff Count vs Total Sales:    r = 0.328 (p = 0.054)
   Staff Count vs Total Quantity: r = 0.302 (p = 0.078)
   Total Quantity vs Total Sales: r = 0.994 (p = 0.000)

 STATISTICAL SIGNIFICANCE (α = 0.05):
   Staff vs Sales:    Not Significant
   Staff vs Quantity: Not Significant
   Quantity vs Sales: Significant

 CORRELATION STRENGTH:
   Staff vs Sales:    Moderate (0.328)
   Staff vs Quantity: Moderate (0.302)
   Quantity vs Sales: Strong (0.994)

 CORRELATION MATRIX:
                Staff_Count  Total_Sales  Total_Quantity
Staff_Count           1.000        0.328 

{'staff_sales_corr': np.float64(0.32814064237136814),
 'staff_sales_p': np.float64(0.05429575349629119),
 'staff_quantity_corr': np.float64(0.302315623709834),
 'staff_quantity_p': np.float64(0.07754123765764515),
 'quantity_sales_corr': np.float64(0.9944932462471471),
 'quantity_sales_p': np.float64(6.511329730628192e-34),
 'correlation_matrix':                 Staff_Count  Total_Sales  Total_Quantity
 Staff_Count        1.000000     0.328141        0.302316
 Total_Sales        0.328141     1.000000        0.994493
 Total_Quantity     0.302316     0.994493        1.000000,
 'data':     Store ID   Total_Sales  Staff_Count  Total_Quantity
 0         28  3.370185e+06            6           73413
 1         30  5.766159e+06            8          125296
 2         12  7.749670e+06            9          169491
 3         10  1.025459e+07            9          207591
 4         27  4.143102e+06            8           91110
 5         33  3.212839e+06            8           70218
 6          

In [ ]:
import polars as pl
import pandas as pd
import statsmodels.api as sm
import os

def load_master_data():
    """Load master transaction data"""
    print(" Loading Master Data...")
    master_data = pl.read_parquet("data/master_transactions.parquet")
    sales_data = master_data.filter(pl.col("Transaction Type") == "Sale")
    print(f"✓ Loaded {sales_data.shape[0]:,} sales transactions")
    return sales_data

def prepare_regression_data(sales_data):
    """Prepare data for multiple regression - aggregated by Store ID"""
    print(" Preparing Regression Dataset by Store ID...")

    # Create output directory
    os.makedirs("ols_regression_results", exist_ok=True)

    # Aggregate by Store ID
    regression_data = sales_data.group_by("Store ID").agg([
        pl.col("Line_Total_USD").sum().alias("Total_Sales"),
        pl.col("Employee ID").n_unique().alias("Staff_Count"),
        pl.col("Quantity").sum().alias("Total_Quantity")
    ]).filter(
        (pl.col("Total_Sales") > 0) &
        (pl.col("Staff_Count") > 0) &
        (pl.col("Total_Quantity") > 0)
    )

    # Save dataset
    regression_data.write_csv("ols_regression_results/regression_data.csv")

    print(f"✓ Dataset prepared: {regression_data.shape[0]} stores")
    print("✓ Variables: Total_Sales, Staff_Count, Total_Quantity")
    return regression_data

def run_ols_regression():
    """Run OLS regression and display results table"""
    print(" Running OLS Regression...")

    # Load data
    df = pl.read_csv("ols_regression_results/regression_data.csv").to_pandas()
    df = df.dropna()

    print(f"✓ Analysis dataset: {len(df)} stores")
    print(f"✓ Variables: {list(df.columns)}")

    # Define variables
    Y = df['Total_Sales']  # Dependent variable
    X = df[['Staff_Count', 'Total_Quantity']]  # Independent variables

    # Add constant (intercept)
    X = sm.add_constant(X)

    # Fit OLS model
    model = sm.OLS(Y, X).fit()

    # Print the regression results table
    print("\n" + "="*80)
    print("OLS REGRESSION RESULTS")
    print("Total_Sales = β₀ + β₁×Staff_Count + β₂×Total_Quantity + ε")
    print("="*80)
    print(model.summary())

    # Save results to file
    with open("ols_regression_results/regression_summary.txt", "w") as f:
        f.write("OLS REGRESSION RESULTS\n")
        f.write("Total_Sales = β₀ + β₁×Staff_Count + β₂×Total_Quantity + ε\n")
        f.write("="*80 + "\n")
        f.write(str(model.summary()))

    # Save coefficients table
    results_df = pd.DataFrame({
        'Variable': ['const', 'Staff_Count', 'Total_Quantity'],
        'Coefficient': model.params.values,
        'Std_Error': model.bse.values,
        't_Statistic': model.tvalues.values,
        'P_Value': model.pvalues.values,
        'Conf_Int_Lower': model.conf_int()[0].values,
        'Conf_Int_Upper': model.conf_int()[1].values
    })

    results_df.to_csv("ols_regression_results/coefficients_table.csv", index=False)

    # Print summary statistics
    print(f"\n DATASET SUMMARY:")
    print(f"   Stores analyzed: {len(df)}")
    print(f"   Total Sales range: ${df['Total_Sales'].min():,.0f} - ${df['Total_Sales'].max():,.0f}")
    print(f"   Staff Count range: {df['Staff_Count'].min()} - {df['Staff_Count'].max()}")
    print(f"   Total Quantity range: {df['Total_Quantity'].min():,.0f} - {df['Total_Quantity'].max():,.0f}")

    print(f"\n✓ Results saved to: ols_regression_results/")
    print(f"✓ Summary: regression_summary.txt")
    print(f"✓ Coefficients: coefficients_table.csv")

    return model

def main():
    """Main function - OLS regression by Store ID"""
    print(" OLS REGRESSION ANALYSIS BY STORE")
    print("Model: Total_Sales = β₀ + β₁×Staff_Count + β₂×Total_Quantity + ε")
    print("Aggregation Level: Store ID")
    print("="*70)

    # Load data
    sales_data = load_master_data()

    # Prepare regression dataset by Store ID
    regression_data = prepare_regression_data(sales_data)

    # Run OLS regression and display table
    model = run_ols_regression()

    print("\n OLS REGRESSION COMPLETED!")

main()


 OLS REGRESSION ANALYSIS BY STORE
Model: Total_Sales = β₀ + β₁×Staff_Count + β₂×Total_Quantity + ε
Aggregation Level: Store ID
 Loading Master Data...
✓ Loaded 6,077,200 sales transactions
 Preparing Regression Dataset by Store ID...
✓ Dataset prepared: 35 stores
✓ Variables: Total_Sales, Staff_Count, Total_Quantity
 Running OLS Regression...
✓ Analysis dataset: 35 stores
✓ Variables: ['Store ID', 'Total_Sales', 'Staff_Count', 'Total_Quantity']

OLS REGRESSION RESULTS
Total_Sales = β₀ + β₁×Staff_Count + β₂×Total_Quantity + ε
                            OLS Regression Results                            
Dep. Variable:            Total_Sales   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                     1560.
Date:                Sun, 03 Aug 2025   Prob (F-statistic):           1.27e-32
Time:                        18:20:53   Log-Likelihood:                -51

Descriptive stats

In [ ]:
def perform_descriptive_statistics(discounts_pl, products_pl, employees_pl, stores_pl, customers_pl, transactions_pl):
    """
    Perform comprehensive descriptive statistics on all datasets
    """
    print("\n" + "="*60)
    print("PERFORMING DESCRIPTIVE STATISTICS ANALYSIS")
    print("="*60)

    import os
    from datetime import datetime

    # Create output directory
    stats_dir = "descriptive_statistics"
    os.makedirs(stats_dir, exist_ok=True)

    stats_summary = {}

    # 1. DISCOUNTS DATASET ANALYSIS
    print("\nDISCOUNTS Dataset Analysis:")
    print("-" * 40)

    discounts_stats = {
        'total_records': discounts_pl.shape[0],
        'total_columns': discounts_pl.shape[1],
        'columns': discounts_pl.columns,
        'data_types': dict(zip(discounts_pl.columns, [str(dtype) for dtype in discounts_pl.dtypes])),
        'null_counts': discounts_pl.null_count().to_pandas().iloc[0].to_dict(),
        'memory_usage_mb': discounts_pl.estimated_size('mb')
    }

    # Numeric columns analysis for discounts
    numeric_cols = discounts_pl.select(pl.col(pl.NUMERIC_DTYPES)).columns
    if numeric_cols:
        discounts_stats['numeric_summary'] = discounts_pl.select(numeric_cols).describe().to_pandas().to_dict()

    stats_summary['discounts'] = discounts_stats

    print(f"   Records: {discounts_stats['total_records']:,}")
    print(f"   Columns: {discounts_stats['total_columns']}")
    print(f"   Memory: {discounts_stats['memory_usage_mb']:.2f} MB")

    # 2. PRODUCTS DATASET ANALYSIS
    print("\nPRODUCTS Dataset Analysis:")
    print("-" * 40)

    products_stats = {
        'total_records': products_pl.shape[0],
        'total_columns': products_pl.shape[1],
        'columns': products_pl.columns,
        'data_types': dict(zip(products_pl.columns, [str(dtype) for dtype in products_pl.dtypes])),
        'null_counts': products_pl.null_count().to_pandas().iloc[0].to_dict(),
        'memory_usage_mb': products_pl.estimated_size('mb')
    }

    # Unique categories/brands analysis
    if 'Category' in products_pl.columns:
        products_stats['unique_categories'] = products_pl['Category'].n_unique()
        products_stats['category_distribution'] = products_pl.group_by('Category').len().sort('len', descending=True).to_pandas().to_dict('records')

    # Price analysis if available
    price_cols = [col for col in products_pl.columns if 'price' in col.lower() or 'cost' in col.lower()]
    if price_cols:
        products_stats['price_summary'] = products_pl.select(price_cols).describe().to_pandas().to_dict()

    stats_summary['products'] = products_stats

    print(f"   Records: {products_stats['total_records']:,}")
    print(f"   Columns: {products_stats['total_columns']}")
    print(f"   Memory: {products_stats['memory_usage_mb']:.2f} MB")
    if 'unique_categories' in products_stats:
        print(f"   Categories: {products_stats['unique_categories']}")

    # 3. EMPLOYEES DATASET ANALYSIS
    print("\nEMPLOYEES Dataset Analysis:")
    print("-" * 40)

    employees_stats = {
        'total_records': employees_pl.shape[0],
        'total_columns': employees_pl.shape[1],
        'columns': employees_pl.columns,
        'data_types': dict(zip(employees_pl.columns, [str(dtype) for dtype in employees_pl.dtypes])),
        'null_counts': employees_pl.null_count().to_pandas().iloc[0].to_dict(),
        'memory_usage_mb': employees_pl.estimated_size('mb')
    }

    # Department/Role analysis if available
    dept_cols = [col for col in employees_pl.columns if any(keyword in col.lower() for keyword in ['department', 'role', 'position', 'title'])]
    for col in dept_cols:
        employees_stats[f'{col}_distribution'] = employees_pl.group_by(col).len().sort('len', descending=True).to_pandas().to_dict('records')

    stats_summary['employees'] = employees_stats

    print(f"   Records: {employees_stats['total_records']:,}")
    print(f"   Columns: {employees_stats['total_columns']}")
    print(f"   Memory: {employees_stats['memory_usage_mb']:.2f} MB")

    # 4. STORES DATASET ANALYSIS
    print("\nSTORES Dataset Analysis:")
    print("-" * 40)

    stores_stats = {
        'total_records': stores_pl.shape[0],
        'total_columns': stores_pl.shape[1],
        'columns': stores_pl.columns,
        'data_types': dict(zip(stores_pl.columns, [str(dtype) for dtype in stores_pl.dtypes])),
        'null_counts': stores_pl.null_count().to_pandas().iloc[0].to_dict(),
        'memory_usage_mb': stores_pl.estimated_size('mb')
    }

    # Location analysis if available
    location_cols = [col for col in stores_pl.columns if any(keyword in col.lower() for keyword in ['city', 'state', 'country', 'region'])]
    for col in location_cols:
        if col in stores_pl.columns:
            stores_stats[f'{col}_distribution'] = stores_pl.group_by(col).len().sort('len', descending=True).to_pandas().to_dict('records')

    stats_summary['stores'] = stores_stats

    print(f"   Records: {stores_stats['total_records']:,}")
    print(f"   Columns: {stores_stats['total_columns']}")
    print(f"   Memory: {stores_stats['memory_usage_mb']:.2f} MB")

    # 5. CUSTOMERS DATASET ANALYSIS
    print("\nCUSTOMERS Dataset Analysis:")
    print("-" * 40)

    customers_stats = {
        'total_records': customers_pl.shape[0],
        'total_columns': customers_pl.shape[1],
        'columns': customers_pl.columns,
        'data_types': dict(zip(customers_pl.columns, [str(dtype) for dtype in customers_pl.dtypes])),
        'null_counts': customers_pl.null_count().to_pandas().iloc[0].to_dict(),
        'memory_usage_mb': customers_pl.estimated_size('mb')
    }

    # Demographics analysis
    demo_cols = [col for col in customers_pl.columns if any(keyword in col.lower() for keyword in ['gender', 'age', 'city', 'state', 'country'])]
    for col in demo_cols:
        if col in customers_pl.columns:
            customers_stats[f'{col}_distribution'] = customers_pl.group_by(col).len().sort('len', descending=True).head(10).to_pandas().to_dict('records')

    stats_summary['customers'] = customers_stats

    print(f"   Records: {customers_stats['total_records']:,}")
    print(f"   Columns: {customers_stats['total_columns']}")
    print(f"   Memory: {customers_stats['memory_usage_mb']:.2f} MB")

    # 6. TRANSACTIONS DATASET ANALYSIS (Most Important)
    print("\nTRANSACTIONS Dataset Analysis:")
    print("-" * 40)

    transactions_stats = {
        'total_records': transactions_pl.shape[0],
        'total_columns': transactions_pl.shape[1],
        'columns': transactions_pl.columns,
        'data_types': dict(zip(transactions_pl.columns, [str(dtype) for dtype in transactions_pl.dtypes])),
        'null_counts': transactions_pl.null_count().to_pandas().iloc[0].to_dict(),
        'memory_usage_mb': transactions_pl.estimated_size('mb')
    }

    # Financial metrics
    financial_cols = [col for col in transactions_pl.columns if any(keyword in col.lower() for keyword in ['amount', 'total', 'price', 'cost', 'revenue', 'quantity'])]
    if financial_cols:
        transactions_stats['financial_summary'] = transactions_pl.select(financial_cols).describe().to_pandas().to_dict()

    # Date range analysis
    date_cols = [col for col in transactions_pl.columns if any(keyword in col.lower() for keyword in ['date', 'time'])]
    for col in date_cols:
        if col in transactions_pl.columns:
            try:
                date_stats = transactions_pl.select([
                    pl.col(col).min().alias('min_date'),
                    pl.col(col).max().alias('max_date'),
                    pl.col(col).n_unique().alias('unique_dates')
                ]).to_pandas().iloc[0].to_dict()
                transactions_stats[f'{col}_range'] = date_stats
            except:
                pass

    # Unique counts for key columns
    key_cols = ['Customer ID', 'Product ID', 'Store ID', 'Employee ID']
    for col in key_cols:
        if col in transactions_pl.columns:
            transactions_stats[f'unique_{col.lower().replace(" ", "_")}'] = transactions_pl[col].n_unique()

    stats_summary['transactions'] = transactions_stats

    print(f"   Records: {transactions_stats['total_records']:,}")
    print(f"   Columns: {transactions_stats['total_columns']}")
    print(f"   Memory: {transactions_stats['memory_usage_mb']:.2f} MB")

    # 7. OVERALL SUMMARY
    print("\nOVERALL DATA SUMMARY:")
    print("-" * 40)

    total_records = sum([stats['total_records'] for stats in stats_summary.values()])
    total_memory = sum([stats['memory_usage_mb'] for stats in stats_summary.values()])

    overall_stats = {
        'total_datasets': len(stats_summary),
        'total_records_across_all': total_records,
        'total_memory_usage_mb': total_memory,
        'analysis_timestamp': datetime.now().isoformat()
    }

    stats_summary['overall'] = overall_stats

    print(f"   Total Datasets: {overall_stats['total_datasets']}")
    print(f"   Total Records: {overall_stats['total_records_across_all']:,}")
    print(f"   Total Memory: {overall_stats['total_memory_usage_mb']:.2f} MB")

    # 8. SAVE RESULTS
    print(f"\nSaving descriptive statistics...")

    # Save as JSON
    import json
    with open(f"{stats_dir}/descriptive_statistics_summary.json", 'w') as f:
        json.dump(stats_summary, f, indent=2, default=str)

    # Save detailed CSV reports for each dataset
    for dataset_name, stats in stats_summary.items():
        if dataset_name != 'overall':
            stats_df = pd.DataFrame([stats])
            stats_df.to_csv(f"{stats_dir}/{dataset_name}_statistics.csv", index=False)

    print(f"Descriptive statistics completed!")
    print(f"Results saved in: {stats_dir}/")
    print("="*60)

    return stats_summary

# Usage in your main function:
def load_data_and_analyze():
    """Load data and perform descriptive statistics"""

    # Load data (your existing function)
    discounts_pl, products_pl, employees_pl, stores_pl, customers_pl, transactions_pl = load_data_efficiently()

    # Perform descriptive statistics
    stats_summary = perform_descriptive_statistics(
        discounts_pl, products_pl, employees_pl, stores_pl, customers_pl, transactions_pl
    )

    return discounts_pl, products_pl, employees_pl, stores_pl, customers_pl, transactions_pl, stats_summary

load_data_and_analyze()


 Loading data efficiently...
Loading discounts...
Loading products...
Loading employees...
Loading stores...
Loading customers...
Loading transactions (this may take a moment)...
Intial Transactions loaded: 6416827 rows
Final Transactions loaded: 6416029 rows
Converting to Polars for better performance...
 Data loaded successfully!
Transactions shape: (6416029, 19)

PERFORMING DESCRIPTIVE STATISTICS ANALYSIS

DISCOUNTS Dataset Analysis:
----------------------------------------
   Records: 181
   Columns: 6
   Memory: 0.02 MB

PRODUCTS Dataset Analysis:
----------------------------------------
   Records: 17,940
   Columns: 12
   Memory: 4.67 MB
   Categories: 3

EMPLOYEES Dataset Analysis:
----------------------------------------
   Records: 404
   Columns: 4
   Memory: 0.02 MB

STORES Dataset Analysis:
----------------------------------------
   Records: 35
   Columns: 8
   Memory: 0.00 MB

CUSTOMERS Dataset Analysis:
----------------------------------------


/tmp/ipython-input-3133635659.py:32: DeprecationWarning: `NUMERIC_DTYPES` is deprecated. Define your own data type groups or use the `polars.selectors` module for selecting columns of a certain data type.
  numeric_cols = discounts_pl.select(pl.col(pl.NUMERIC_DTYPES)).columns


   Records: 1,643,306
   Columns: 9
   Memory: 169.71 MB

TRANSACTIONS Dataset Analysis:
----------------------------------------
   Records: 6,416,029
   Columns: 19
   Memory: 936.52 MB

OVERALL DATA SUMMARY:
----------------------------------------
   Total Datasets: 6
   Total Records: 8,077,895
   Total Memory: 1110.94 MB

Saving descriptive statistics...
Descriptive statistics completed!
Results saved in: descriptive_statistics/


(shape: (181, 6)
 ┌────────────┬────────────┬─────────┬─────────────────────┬───────────┬────────────────────────────┐
 │ Start      ┆ End        ┆ Discont ┆ Description         ┆ Category  ┆ Sub Category               │
 │ ---        ┆ ---        ┆ ---     ┆ ---                 ┆ ---       ┆ ---                        │
 │ str        ┆ str        ┆ f64     ┆ str                 ┆ str       ┆ str                        │
 ╞════════════╪════════════╪═════════╪═════════════════════╪═══════════╪════════════════════════════╡
 │ 2020-01-01 ┆ 2020-01-10 ┆ 0.4     ┆ 40% discount during ┆ Feminine  ┆ Coats and Blazers          │
 │            ┆            ┆         ┆ our New Ye…         ┆           ┆                            │
 │ 2020-01-01 ┆ 2020-01-10 ┆ 0.4     ┆ 40% discount during ┆ Feminine  ┆ Sweaters and Knitwear      │
 │            ┆            ┆         ┆ our New Ye…         ┆           ┆                            │
 │ 2020-01-01 ┆ 2020-01-10 ┆ 0.4     ┆ 40% discount during ┆ Masc